# Import

In [ ]:
import platform
import sys
print(f"개발환경 (OS) : {platform.platform()}")
print(f"Python Version : {sys.version}")
# T4 GPU, 고용량 RAM 설정

개발환경 (OS) : Linux-6.1.123+-x86_64-with-glibc2.35
Python Version : 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]


In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# LightGBM을 위한 GPU driver 설치 코드
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!sudo apt install nvidia-driver-460 nvidia-cuda-toolkit clinfo
!apt-get update --fix-missing
!pip install -q  lightgbm==4.1.0 \
  --config-settings=cmake.define.USE_GPU=ON \
  --config-settings=cmake.define.OpenCL_INCLUDE_DIR="/usr/local/cuda/include/" \
  --config-settings=cmake.define.OpenCL_LIBRARY="/usr/local/cuda/lib64/libOpenCL.so"

# catboost 설치
!pip install catboost

# !pip install imbalanced-learn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
clinfo is already the newest version (3.0.21.02.21-1).
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 libnvidia-compute-510 : Depends: libnvidia-compute-535 but it is not installable
 nvidia-cuda-dev : Breaks: libcuda1 (< 495)
                   Recommends: libnvcuvid1 but it is not installable
E: Unable to correct problems, you have held broken packages.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archi

In [ ]:
import sklearn
from sklearn.utils.class_weight import compute_class_weight
import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, LGBMClassifier

In [ ]:
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"lightgbm version: {lgb.__version__}")
print(f"catboost version: {catboost.__version__}")
print(f"xgboost version: {xgb.__version__}")
print(f"sklearn version: {sklearn.__version__}")
print(f"imblearn version: {imblearn.__version__}")

pandas version: 2.2.2
numpy version: 2.0.2
lightgbm version: 4.1.0
catboost version: 1.2.8
xgboost version: 2.1.4
sklearn version: 1.6.1
imblearn version: 0.13.0


# Data load & Preprocessing (1) - train data

## 회원정보 (customer)

- customer_train_df 생성

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"] # train 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"

            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"

            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

customer_train_07 is loaded from /content/drive/MyDrive/Data/train/1.회원정보/201807_train_회원정보.parquet
customer_train_08 is loaded from /content/drive/MyDrive/Data/train/1.회원정보/201808_train_회원정보.parquet
customer_train_09 is loaded from /content/drive/MyDrive/Data/train/1.회원정보/201809_train_회원정보.parquet
customer_train_10 is loaded from /content/drive/MyDrive/Data/train/1.회원정보/201810_train_회원정보.parquet
customer_train_11 is loaded from /content/drive/MyDrive/Data/train/1.회원정보/201811_train_회원정보.parquet
customer_train_12 is loaded from /content/drive/MyDrive/Data/train/1.회원정보/201812_train_회원정보.parquet


93

In [ ]:
# 데이터 유형별 설정
info_categories = ["customer"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

customer_train_df = train_dfs["customer_train_df"]
gc.collect()

customer_train_df is created with shape: (2400000, 78)


0

- 전처리

In [ ]:
##### 1. 결측치 처리
# '_1순위신용체크구분', '_2순위신용체크구분’
#    → 결측치 '기타'로 채우기
#    → '_1순위신용체크구분_인코딩', '_2순위신용체크구분_인코딩’
customer_train_df['_1순위신용체크구분'] = customer_train_df['_1순위신용체크구분'].fillna('기타')
customer_train_df['_2순위신용체크구분'] = customer_train_df['_2순위신용체크구분'].fillna('기타')
mapping = {'신용': 1, '체크': 0, '기타': -1}
customer_train_df['1순위신용체크구분_인코딩'] = customer_train_df['_1순위신용체크구분'].map(mapping)
customer_train_df['2순위신용체크구분_인코딩'] = customer_train_df['_2순위신용체크구분'].map(mapping)
customer_train_df.drop(columns=['_1순위신용체크구분','_2순위신용체크구분'], inplace=True)

# 가입통신회사코드 → 가입통신회사_S사여부
customer_train_df['가입통신회사_S사여부'] = (customer_train_df['가입통신회사코드'] == 'S사').astype(int)
customer_train_df = customer_train_df.drop(columns=['가입통신회사코드'])

# 직장시도명 → 직장_수도권여부
customer_train_df['직장_수도권여부'] = customer_train_df['직장시도명'].isin(['서울', '경기']).astype(int)
customer_train_df = customer_train_df.drop(columns=['직장시도명'])

# 결측치 많은 변수 삭제!
customer_train_df = customer_train_df.drop(columns=['최종카드발급일자','최종유효년월_신용_이용가능', '최종유효년월_신용_이용'])


##### 2. 자료형 변환
# 거주시도명 → 거주지_수도권여부
customer_train_df['거주지_수도권여부'] = customer_train_df['거주시도명'].isin(['서울', '경기']).astype(int)
customer_train_df = customer_train_df.drop(columns=['거주시도명'])

# 연회비발생카드수_B0M → 연회비발생카드수_B0M_이진
customer_train_df['연회비발생카드수_B0M_이진'] = customer_train_df['연회비발생카드수_B0M'].isin(['1개이상']).astype(int)
customer_train_df = customer_train_df.drop(columns=['연회비발생카드수_B0M'])

# 데이터 다 동일한 변수 4개 삭제!
columns_drop = ['상품관련면제카드수_B0M','임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']
customer_train_df = customer_train_df.drop(columns=columns_drop)

# Life_Stage: 파생변수(Life_Stage_자녀성장_여부) 생성 후 변수 삭제
customer_train_df['Life_Stage_자녀성장_여부'] = customer_train_df['Life_Stage'].isin(['자녀성장(1)', '자녀성장(2)']).astype(int)
customer_train_df = customer_train_df.drop(columns=['Life_Stage'])

# 연령에서 숫자만 추출 후 int형으로 변환
customer_train_df['연령'] = customer_train_df['연령'].str.extract(r'(\d+)').astype(float).astype('Int64')

##### 3.상관관계 분석
# 데이터 다 동일한 변수 4개 삭제
columns_drop = ['이용금액_R3M_체크_가족','연회비할인카드수_B0M','할인금액_기본연회비_B0M','할인금액_제휴연회비_B0M']
customer_train_df = customer_train_df.drop(columns=columns_drop)

##### 4. 변수 별로 확인
# 4개의 변수 삭제
# '입회일자_신용' 삭제 (입회경과개월수_신용 있음)
# 데이터 다 동일한 변수 '이용카드수_체크_가족' 삭제
# '청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M’삭제 (의미 같은 변수 있음)
columns_drop = ['입회일자_신용','이용카드수_체크_가족','청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M']
customer_train_df = customer_train_df.drop(columns=columns_drop)

- 파일 저장

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in customer_train_df.select_dtypes(include='int64').columns:
    if customer_train_df[col].max() < 2_147_483_647:
        customer_train_df[col] = customer_train_df[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in customer_train_df.select_dtypes(include='float64').columns:
    customer_train_df[col] = customer_train_df[col].astype('float32')

#customer_train_df = customer_train_df.drop(columns=['Segment'])
customer_train_df.to_parquet("/content/drive/MyDrive/Dacon/customer_train_cleaned.parquet")

## 신용정보 (credit)

- credit_with_seg 생성

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"] # train 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"

            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"

            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

credit_train_07 is loaded from /content/drive/MyDrive/Data/train/2.신용정보/201807_train_신용정보.parquet
credit_train_08 is loaded from /content/drive/MyDrive/Data/train/2.신용정보/201808_train_신용정보.parquet
credit_train_09 is loaded from /content/drive/MyDrive/Data/train/2.신용정보/201809_train_신용정보.parquet
credit_train_10 is loaded from /content/drive/MyDrive/Data/train/2.신용정보/201810_train_신용정보.parquet
credit_train_11 is loaded from /content/drive/MyDrive/Data/train/2.신용정보/201811_train_신용정보.parquet
credit_train_12 is loaded from /content/drive/MyDrive/Data/train/2.신용정보/201812_train_신용정보.parquet


19

In [ ]:
# 데이터 유형별 설정
info_categories = ["credit"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

credit_train_df = train_dfs["credit_train_df"]
gc.collect()

credit_train_df is created with shape: (2400000, 42)


0

In [ ]:
customer_train_df = pd.read_parquet('/content/drive/MyDrive/Dacon/customer_train_cleaned.parquet')
customer_train_df_seg = customer_train_df[['기준년월','ID','Segment']]

credit_with_seg = customer_train_df_seg.merge(credit_train_df, on=['기준년월', 'ID'], how='left') # segment 변수 추가
#credit_with_seg.to_parquet('/content/drive/MyDrive/DACON/rawdata/credit_with_seg.parquet')

- 전처리

In [ ]:
##### 1. 결측치 처리
# 'RV신청일자' 변수 삭제
credit_with_seg = credit_with_seg.drop(columns=['RV신청일자'])

# 'RV전환가능여부' → 'RV전환불가능여부'
credit_with_seg['RV_전환가능여부_이진'] = (credit_with_seg['RV전환가능여부'] == 'N').astype(int)
credit_with_seg = credit_with_seg.drop(columns='RV전환가능여부')

##### 2. 자료형 변환
# 자발한도감액횟수_R12M: '0회' → 0, '1회' → 1, '2회' → 2
credit_with_seg['자발한도감액횟수_R12M'] = (
    credit_with_seg['자발한도감액횟수_R12M']
    .str.replace('회', '', regex=False)  # '회' 제거
    .astype(int)  # 바로 int로 변환!
)

#‘한도증액횟수_R12M’ → '한도증액_R12M_여부': '0회' → 0, '1회이상' → 1
credit_with_seg['한도증액_R12M_여부'] = credit_with_seg['한도증액횟수_R12M'].map({
    '0회': 0,
    '1회이상': 1
    }).astype(int)
credit_with_seg.drop(columns=['한도증액횟수_R12M'], inplace=True)

# '카드론동의여부': 'Y' → 1, 'N' → 0
credit_with_seg['카드론동의여부'] = credit_with_seg['카드론동의여부'].map({
    'Y': 1,
    'N': 0
    }).astype(int)

# '한도심사요청건수' → '한도심사요청여부': '0회' → 0, '1회이상' → 1
credit_with_seg['한도심사요청여부'] = credit_with_seg['한도심사요청건수'].map({
    '0회': 0,
    '1회이상': 1
    }).astype(int)
credit_with_seg.drop(columns=['한도심사요청건수'], inplace=True)

In [ ]:
##### 3. 상관관계 분석
# 데이터 모두 동일한 변수 1개 삭제
credit_with_seg = credit_with_seg.drop(columns=['시장연체상환여부_R3M'])

##### 4. 변수 별로 확인
# 'RV실사용여부' 변수 추가
credit_with_seg['RV실사용여부'] = (credit_with_seg['RV약정청구율'] > 0).astype(int)

# '강제한도감액횟수_2회이상여부' 변수 추가
credit_with_seg['강제한도감액횟수_2회이상여부'] = (credit_with_seg['강제한도감액횟수_R12M'] > 1).astype(int)

# '강제한도감액금액_R12M_3이상여부' 변수 추가
credit_with_seg['강제한도감액금액_R12M_3이상여부'] = (credit_with_seg['강제한도감액금액_R12M'] > 2).astype(int)

# '상향가능CA한도금액_1여부' 변수 추가
credit_with_seg['상향가능CA한도금액_1여부'] = (credit_with_seg['상향가능CA한도금액'] == 1).astype(int)

# '카드이용한도금액_A수준복합' 변수 추가
a_limit = credit_with_seg[credit_with_seg['Segment'] == 'A']['카드이용한도금액_B1M']
q1 = a_limit.quantile(0.25)
q3 = a_limit.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
a_no_outlier = a_limit[(a_limit >= lower_bound) & (a_limit <= upper_bound)]
a_B1M = a_no_outlier.min()
print(f"A의 이상치 제외 후 최솟값: {a_B1M:,}")  # 170,344

a_limit = credit_with_seg[credit_with_seg['Segment'] == 'A']['카드이용한도금액_B2M']
q1 = a_limit.quantile(0.25)
q3 = a_limit.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
a_no_outlier = a_limit[(a_limit >= lower_bound) & (a_limit <= upper_bound)]
a_B2M = a_no_outlier.min()
print(f"A의 이상치 제외 후 최솟값: {a_B2M:,}")  # 170,344

def classify_dual_limit(row):
    b1 = row['카드이용한도금액_B1M'] >= a_B1M
    b2 = row['카드이용한도금액_B2M'] >= a_B2M
    if b1 and b2:
        return 2     # 둘 다 170,344 이상
    elif b1 or b2:
        return 1     # 하나만 170,344 이상
    else:
        return 0    # 둘 다 170,344 미만

credit_with_seg['카드이용한도금액_A수준복합'] = credit_with_seg.apply(classify_dual_limit, axis=1)

A의 이상치 제외 후 최솟값: 170,344
A의 이상치 제외 후 최솟값: 170,344


- 파일 저장

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in credit_with_seg.select_dtypes(include='int64').columns:
    if credit_with_seg[col].max() < 2_147_483_647:
        credit_with_seg[col] = credit_with_seg[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in credit_with_seg.select_dtypes(include='float64').columns:
    credit_with_seg[col] = credit_with_seg[col].astype('float32')

In [ ]:
credit_with_seg = credit_with_seg.drop(columns=['Segment'])
credit_with_seg.to_parquet("/content/drive/MyDrive/Dacon/credit_train_cleaned.parquet")

## 승인매출정보 (sales)

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- sales_with_seg 생성

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"] # train 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            #file_path = f"./{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"

            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"

            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

sales_train_07 is loaded from /content/drive/MyDrive/Data/train/3.승인매출정보/201807_train_승인매출정보.parquet
sales_train_08 is loaded from /content/drive/MyDrive/Data/train/3.승인매출정보/201808_train_승인매출정보.parquet
sales_train_09 is loaded from /content/drive/MyDrive/Data/train/3.승인매출정보/201809_train_승인매출정보.parquet
sales_train_10 is loaded from /content/drive/MyDrive/Data/train/3.승인매출정보/201810_train_승인매출정보.parquet
sales_train_11 is loaded from /content/drive/MyDrive/Data/train/3.승인매출정보/201811_train_승인매출정보.parquet
sales_train_12 is loaded from /content/drive/MyDrive/Data/train/3.승인매출정보/201812_train_승인매출정보.parquet


90

In [ ]:
# 데이터 유형별 설정
info_categories = ["sales"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

sales_train_df = train_dfs["sales_train_df"]
gc.collect()

sales_train_df is created with shape: (2400000, 406)


0

In [ ]:
customer_train_df = pd.read_parquet('/content/drive/MyDrive/Dacon/customer_train_cleaned.parquet')
customer_train_df_seg = customer_train_df[['기준년월','ID','Segment']]

sales_with_seg = customer_train_df_seg.merge(sales_train_df, on=['기준년월', 'ID'], how='left') # segment 변수 추가

del sales_train_df
del customer_train_df
del customer_train_df_seg
gc.collect()

0

- 전처리

### 1. 결측치 처리

In [ ]:
# '_n순위@@업종' 변수(총 15개 변수) 결측치는 '없음'으로 채우기
upjong_cols = [
    '_1순위업종','_2순위업종','_3순위업종',
    '_1순위쇼핑업종','_2순위쇼핑업종','_3순위쇼핑업종',
    '_1순위교통업종','_2순위교통업종','_3순위교통업종',
    '_1순위여유업종','_2순위여유업종','_3순위여유업종',
    '_1순위납부업종','_2순위납부업종','_3순위납부업종'
]

sales_with_seg[upjong_cols] = sales_with_seg[upjong_cols].fillna('없음')

In [ ]:
# '_n순위업종_AB' 변수 추가 (상관계수 -0.3030)

'''
- 1순위 업종이 쇼핑	+1
- 2순위 업종이 사교활동	+1
- 2순위 업종이 교육	+0.5
- 2순위 업종이 의료	+1.5 (A, B에서 더 강하게 나타나므로)
- 3순위 업종이 사교활동	+1
- 3순위 업종이 의료	+1.5 (A, B에서 더 강하게 나타나므로)

- 2순위 업종이 없음 -1
- 3순위 업종이 없음 -1
- 2순위 업종이 교통 -0.5
- 2순위 업종이 납부 -0.5
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위업종'] == '쇼핑':
        score += 1
    if row['_2순위업종'] == '사교활동':
        score += 1
    if row['_2순위업종'] == '교육':
        score += 0.5
    if row['_2순위업종'] == '의료':
        score += 1
    if row['_3순위업종'] == '사교활동':
        score += 1
    if row['_3순위업종'] == '의료':
        score += 1

    # 감점 조건
    if row['_2순위업종'] == '없음':
        score -= 1
    if row['_3순위업종'] == '없음':
        score -= 1
    if row['_2순위업종'] == '교통':
        score -= 0.5
    if row['_2순위업종'] == '납부':
        score -= 0.5

    return score

sales_with_seg['_n순위업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위쇼핑업종_AB' 변수 추가 (상관계수 -0.3304)

'''
- 1순위 쇼핑업종이 온라인 +1
- 2순위 쇼핑업종이 도소매 +1
- 3순위 쇼핑업종이 마트 +1

- 1순위 쇼핑업종이 없음 -1
- 2순위 쇼핑업종이 슈퍼마켓 -0.5
- 2순위 쇼핑업종이 없음 -0.5
- 3순위 쇼핑업종이 없음 -1
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위쇼핑업종'] == '온라인':
        score += 1
    if row['_2순위쇼핑업종'] == '도소매':
        score += 1
    if row['_3순위쇼핑업종'] == '마트':
        score += 1

    # 감점 조건
    if row['_1순위쇼핑업종'] == '없음':
        score -= 1
    if row['_2순위쇼핑업종'] == '슈퍼마켓':
        score -= 0.5
    if row['_2순위쇼핑업종'] == '없음':
        score -= 0.5
    if row['_3순위쇼핑업종'] == '없음':
        score -= 1

    return score

sales_with_seg['_n순위쇼핑업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위교통업종_AB' 변수 추가 (상관계수 -0.34514)

'''
- 1순위 교통업종이 주유 +1
- 2순위 교통업종이 정비 +1
- 3순위 교통업종이 택시 +0.5
- 3순위 교통업종이 정비 +0.5
- 3순위 교통업종이 철도버스 +0.5

- 1순위 교통업종이 없음 -1
- 2순위 교통업종이 없음 -1
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위교통업종'] == '주유':
        score += 1
    if row['_2순위교통업종'] == '정비':
        score += 1
    if row['_3순위교통업종'] == '택시':
        score += 0.5
    if row['_3순위교통업종'] == '정비':
        score += 0.5
    if row['_3순위교통업종'] == '철도버스':
        score += 0.5

    # 감점 조건
    if row['_1순위교통업종'] == '없음':
        score -= 1
    if row['_2순위교통업종'] == '없음':
        score -= 1

    return score

sales_with_seg['_n순위교통업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위여유업종_AB' 변수 추가 (상관계수 -0.26628)

'''
- 1순위 여유업종이 운동 +1
- 1순위 여유업종이 항공 +1
- 2순위 여유업종이 숙박 +1
- 2순위 여유업종이 공연 +1
- 3순위 여유업종이 숙박 +1
- 3순위 여유업종이 공연 +1
- 3순위 여유업종이 여유기타 +1

- 1순위 여유업종이 없음 -0.5
- 2순위 여유업종이 없음 -0.5
- 3순위 여유업종이 없음 -0.5
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위여유업종'] == '운동':
        score += 1
    if row['_1순위여유업종'] == '항공':
        score += 1
    if row['_2순위여유업종'] == '숙박':
        score += 1
    if row['_2순위여유업종'] == '공연':
        score += 1
    if row['_3순위여유업종'] == '숙박':
        score += 1
    if row['_3순위여유업종'] == '공연':
        score += 1
    if row['_3순위여유업종'] == '여유기타':
        score += 1

    # 감점 조건
    if row['_1순위여유업종'] == '없음':
        score -= 0.5
    if row['_2순위여유업종'] == '없음':
        score -= 0.5
    if row['_3순위여유업종'] == '없음':
        score -= 0.5

    return score

sales_with_seg['_n순위여유업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위납부업종_AB' 변수 추가 (상관계수 -0.2579)

'''
- 1순위 납부업종이 납부기타 +0.5
- 1순위 납부업종이 보험료 +0.5
- 1순위 납부업종이 통신비 +0.5
- 2순위 납부업종이 납부기타 +0.5
- 2순위 납부업종이 보험료 +0.5
- 2순위 납부업종이 통신비 +0.5
- 3순위 납부업종이 납부기타 +0.5

- 1순위 납부업종이 없음 -0.5
- 2순위 납부업종이 없음 -0.5
- 3순위 납부업종이 없음 -0.5
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위납부업종'] == '납부기타':
        score += 0.5
    if row['_1순위납부업종'] == '보험료':
        score += 0.5
    if row['_1순위납부업종'] == '통신비':
        score += 0.5
    if row['_2순위납부업종'] == '납부기타':
        score += 0.5
    if row['_2순위납부업종'] == '보험료':
        score += 0.5
    if row['_2순위납부업종'] == '통신비':
        score += 0.5
    if row['_3순위납부업종'] == '납부기타':
        score += 0.5

    # 감점 조건
    if row['_1순위납부업종'] == '없음':
        score -= 0.5
    if row['_2순위납부업종'] == '없음':
        score -= 0.5
    if row['_3순위납부업종'] == '없음':
        score -= 0.5

    return score

sales_with_seg['_n순위납부업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위@@업종_AB' 5개 변수 합친 변수 '_n순위업종_AB_총합' 추가 (상관계수 -0.3927)
sales_with_seg['_n순위업종_AB_총합'] = (
    sales_with_seg['_n순위업종_AB'] +
    sales_with_seg['_n순위쇼핑업종_AB'] +
    sales_with_seg['_n순위교통업종_AB'] +
    sales_with_seg['_n순위여유업종_AB'] +
    sales_with_seg['_n순위납부업종_AB']
)

# '_n순위@@업종' 변수 삭제
cols_to_drop = [
    '_1순위업종','_2순위업종','_3순위업종',
    '_1순위쇼핑업종','_2순위쇼핑업종','_3순위쇼핑업종',
    '_1순위교통업종','_2순위교통업종','_3순위교통업종',
    '_1순위여유업종','_2순위여유업종','_3순위여유업종',
    '_1순위납부업종','_2순위납부업종','_3순위납부업종'
]

sales_with_seg.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# 결측치 80% 이상인 최종카드론 관련 변수 삭제
cols_to_drop = ['최종카드론_대출일자','최종카드론_신청경로코드','최종카드론_금융상환방식코드']
sales_with_seg = sales_with_seg.drop(columns=cols_to_drop)

### 2. 자료형 변환

In [ ]:
# '이용금액대_점수' 변수 생성 (상관계수 -0.6026)
amount_mapping = {
    '01.100만원+': 18,
    '02.50만원+': 5,
    '03.30만원+': 2,
    '04.10만원+': 2,
    '05.10만원-': 1,
    '09.미사용': 1
}

sales_with_seg['이용금액대_점수'] = sales_with_seg['이용금액대'].map(amount_mapping)

# 원래 변수 '이용금액대' 삭제
sales_with_seg = sales_with_seg.drop(columns=['이용금액대'])

### 3. 상관관계 분석

In [ ]:
# 데이터 값이 모두 같은 변수 2개 제거
sales_with_seg = sales_with_seg.drop(columns=['승인거절건수_입력오류_B0M','승인거절건수_기타_B0M'])

### 변수 별로 데이터 살펴보기

In [ ]:
# '이용여부_@@' 변수 생성
sales_with_seg['이용여부_기본']   = (sales_with_seg['최종이용일자_기본'] != 10101).astype(int)
sales_with_seg['이용여부_신판']   = (sales_with_seg['최종이용일자_신판'] != 10101).astype(int)
sales_with_seg['이용여부_CA']    = (sales_with_seg['최종이용일자_CA'] != 10101).astype(int)
sales_with_seg['이용여부_카드론'] = (sales_with_seg['최종이용일자_카드론'] != 10101).astype(int)
sales_with_seg['이용여부_체크']   = (sales_with_seg['최종이용일자_체크'] != 10101).astype(int)
sales_with_seg['이용여부_일시불'] = (sales_with_seg['최종이용일자_일시불'] != 10101).astype(int)
sales_with_seg['이용여부_할부']   = (sales_with_seg['최종이용일자_할부'] != 10101).astype(int)

# 원래 '최종이용일자_@@' 변수 삭제
for i in ['최종이용일자_기본','최종이용일자_신판','최종이용일자_CA','최종이용일자_카드론','최종이용일자_체크','최종이용일자_일시불','최종이용일자_할부']:
  sales_with_seg = sales_with_seg.drop(columns=[i])

- BOM 변수

In [ ]:
##### 이용건수 #####

# 데이터 값이 모두 같은 변수 제거
sales_with_seg = sales_with_seg.drop(columns=['이용건수_부분무이자_B0M'])

# '이용건수_신용_B0M' 변수 값 보정
sales_with_seg['이용건수_신용_B0M'] = sales_with_seg['이용건수_신용_B0M'] + sales_with_seg['이용건수_카드론_B0M']



##### 이용금액 #####

# 데이터 값이 모두 같은 변수 제거
sales_with_seg = sales_with_seg.drop(columns=['이용금액_부분무이자_B0M'])

# '이용금액_신용_B0M', '이용금액_신판_B0M' 변수 생성
sales_with_seg['이용금액_신용_B0M'] = sales_with_seg['이용금액_일시불_B0M'] + sales_with_seg['이용금액_할부_B0M'] + sales_with_seg['이용금액_CA_B0M'] + sales_with_seg['이용금액_카드론_B0M']
sales_with_seg['이용금액_신판_B0M'] = sales_with_seg['이용금액_일시불_B0M'] + sales_with_seg['이용금액_할부_B0M']

In [ ]:
# 건수별평균이용금액 변수 생성
sales_with_seg['건수별평균이용금액_신용_B0M'] = np.where(
    (sales_with_seg['이용건수_신용_B0M'] == 0) | (sales_with_seg['이용금액_신용_B0M'] == 0), 0,
    sales_with_seg['이용금액_신용_B0M'] / sales_with_seg['이용건수_신용_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_B0M'] = np.where(
    (sales_with_seg['이용건수_신판_B0M'] == 0) | (sales_with_seg['이용금액_신판_B0M'] == 0), 0,
    sales_with_seg['이용금액_신판_B0M'] / sales_with_seg['이용건수_신판_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_B0M'] = np.where(
    (sales_with_seg['이용건수_일시불_B0M'] == 0) | (sales_with_seg['이용금액_일시불_B0M'] == 0), 0,
    sales_with_seg['이용금액_일시불_B0M'] / sales_with_seg['이용건수_일시불_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_B0M'] = np.where(
    (sales_with_seg['이용건수_할부_B0M'] == 0) | (sales_with_seg['이용금액_할부_B0M'] == 0), 0,
    sales_with_seg['이용금액_할부_B0M'] / sales_with_seg['이용건수_할부_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_B0M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_B0M'] == 0) | (sales_with_seg['이용금액_할부_유이자_B0M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_B0M'] / sales_with_seg['이용건수_할부_유이자_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_B0M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_B0M'] == 0) | (sales_with_seg['이용금액_할부_무이자_B0M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_B0M'] / sales_with_seg['이용건수_할부_무이자_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_B0M'] = np.where(
    (sales_with_seg['이용건수_CA_B0M'] == 0) | (sales_with_seg['이용금액_CA_B0M'] == 0), 0,
    sales_with_seg['이용금액_CA_B0M'] / sales_with_seg['이용건수_CA_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_B0M'] = np.where(
    (sales_with_seg['이용건수_체크_B0M'] == 0) | (sales_with_seg['이용금액_체크_B0M'] == 0), 0,
    sales_with_seg['이용금액_체크_B0M'] / sales_with_seg['이용건수_체크_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_B0M'] = np.where(
    (sales_with_seg['이용건수_카드론_B0M'] == 0) | (sales_with_seg['이용금액_카드론_B0M'] == 0), 0,
    sales_with_seg['이용금액_카드론_B0M'] / sales_with_seg['이용건수_카드론_B0M'].abs()
)

In [ ]:
# 최근3개월이용여부 변수 생성
sales_with_seg['최근3개월이용여부_신용'] = (sales_with_seg['이용후경과월_신용'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_신판'] = (sales_with_seg['이용후경과월_신판'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_일시불'] = (sales_with_seg['이용후경과월_일시불'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_할부'] = (sales_with_seg['이용후경과월_할부'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_할부_유이자'] = (sales_with_seg['이용후경과월_할부_유이자'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_할부_무이자'] = (sales_with_seg['이용후경과월_할부_무이자'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_CA'] = (sales_with_seg['이용후경과월_CA'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_체크'] = (sales_with_seg['이용후경과월_체크'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_카드론'] = (sales_with_seg['이용후경과월_카드론'] <= 3).astype(int)

- R12M 변수

In [ ]:
##### 이용건수 #####

# '이용건수_신용_R12M' 변수 값 보정
sales_with_seg['이용건수_신용_R12M'] = sales_with_seg['이용건수_신용_R12M'] + sales_with_seg['이용건수_카드론_R12M']


##### 이용금액 #####

# '이용금액_신용_R12M', '이용금액_신판_R12M' 변수 생성
sales_with_seg['이용금액_신용_R12M'] = sales_with_seg['이용금액_일시불_R12M'] + sales_with_seg['이용금액_할부_R12M'] + sales_with_seg['이용금액_CA_R12M'] + sales_with_seg['이용금액_카드론_R12M']
sales_with_seg['이용금액_신판_R12M'] = sales_with_seg['이용금액_일시불_R12M'] + sales_with_seg['이용금액_할부_R12M']


In [ ]:
# 건수별평균이용금액 변수 생성 (R12M 기준)
sales_with_seg['건수별평균이용금액_신용_R12M'] = np.where(
    (sales_with_seg['이용건수_신용_R12M'] == 0) | (sales_with_seg['이용금액_신용_R12M'] == 0), 0,
    sales_with_seg['이용금액_신용_R12M'] / sales_with_seg['이용건수_신용_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_R12M'] = np.where(
    (sales_with_seg['이용건수_신판_R12M'] == 0) | (sales_with_seg['이용금액_신판_R12M'] == 0), 0,
    sales_with_seg['이용금액_신판_R12M'] / sales_with_seg['이용건수_신판_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_R12M'] = np.where(
    (sales_with_seg['이용건수_일시불_R12M'] == 0) | (sales_with_seg['이용금액_일시불_R12M'] == 0), 0,
    sales_with_seg['이용금액_일시불_R12M'] / sales_with_seg['이용건수_일시불_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_R12M'] = np.where(
    (sales_with_seg['이용건수_할부_R12M'] == 0) | (sales_with_seg['이용금액_할부_R12M'] == 0), 0,
    sales_with_seg['이용금액_할부_R12M'] / sales_with_seg['이용건수_할부_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_R12M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_R12M'] == 0) | (sales_with_seg['이용금액_할부_유이자_R12M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_R12M'] / sales_with_seg['이용건수_할부_유이자_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_R12M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_R12M'] == 0) | (sales_with_seg['이용금액_할부_무이자_R12M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_R12M'] / sales_with_seg['이용건수_할부_무이자_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_R12M'] = np.where(
    (sales_with_seg['이용건수_CA_R12M'] == 0) | (sales_with_seg['이용금액_CA_R12M'] == 0), 0,
    sales_with_seg['이용금액_CA_R12M'] / sales_with_seg['이용건수_CA_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_R12M'] = np.where(
    (sales_with_seg['이용건수_체크_R12M'] == 0) | (sales_with_seg['이용금액_체크_R12M'] == 0), 0,
    sales_with_seg['이용금액_체크_R12M'] / sales_with_seg['이용건수_체크_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_R12M'] = np.where(
    (sales_with_seg['이용건수_카드론_R12M'] == 0) | (sales_with_seg['이용금액_카드론_R12M'] == 0), 0,
    sales_with_seg['이용금액_카드론_R12M'] / sales_with_seg['이용건수_카드론_R12M'].abs()
)

In [ ]:
# 이용월평균이용금액 변수 생성 (R12M 기)
sales_with_seg['이용월평균이용금액_신용_R12M'] = np.where(
    sales_with_seg['이용개월수_신용_R12M'] == 0, 0,
    sales_with_seg['이용금액_신용_R12M'] / sales_with_seg['이용개월수_신용_R12M']
)

sales_with_seg['이용월평균이용금액_신판_R12M'] = np.where(
    sales_with_seg['이용개월수_신판_R12M'] == 0, 0,
    sales_with_seg['이용금액_신판_R12M'] / sales_with_seg['이용개월수_신판_R12M']
)

sales_with_seg['이용월평균이용금액_일시불_R12M'] = np.where(
    sales_with_seg['이용개월수_일시불_R12M'] == 0, 0,
    sales_with_seg['이용금액_일시불_R12M'] / sales_with_seg['이용개월수_일시불_R12M']
)

sales_with_seg['이용월평균이용금액_할부_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_R12M'] == 0, 0,
    sales_with_seg['이용금액_할부_R12M'] / sales_with_seg['이용개월수_할부_R12M']
)

sales_with_seg['이용월평균이용금액_할부_유이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R12M'] == 0, 0,
    sales_with_seg['이용금액_할부_유이자_R12M'] / sales_with_seg['이용개월수_할부_유이자_R12M']
)

sales_with_seg['이용월평균이용금액_할부_무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R12M'] == 0, 0,
    sales_with_seg['이용금액_할부_무이자_R12M'] / sales_with_seg['이용개월수_할부_무이자_R12M']
)

sales_with_seg['이용월평균이용금액_부분무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R12M'] == 0, 0,
    sales_with_seg['이용금액_부분무이자_R12M'] / sales_with_seg['이용개월수_부분무이자_R12M']
)

sales_with_seg['이용월평균이용금액_CA_R12M'] = np.where(
    sales_with_seg['이용개월수_CA_R12M'] == 0, 0,
    sales_with_seg['이용금액_CA_R12M'] / sales_with_seg['이용개월수_CA_R12M']
)

sales_with_seg['이용월평균이용금액_체크_R12M'] = np.where(
    sales_with_seg['이용개월수_체크_R12M'] == 0, 0,
    sales_with_seg['이용금액_체크_R12M'] / sales_with_seg['이용개월수_체크_R12M']
)

sales_with_seg['이용월평균이용금액_카드론_R12M'] = np.where(
    sales_with_seg['이용개월수_카드론_R12M'] == 0, 0,
    sales_with_seg['이용금액_카드론_R12M'] / sales_with_seg['이용개월수_카드론_R12M']
)

In [ ]:
# 이용월평균이용건수 변수 생성 (R12M 기준)
sales_with_seg['이용월평균이용건수_신용_R12M'] = np.where(
    sales_with_seg['이용개월수_신용_R12M'] == 0, 0,
    sales_with_seg['이용건수_신용_R12M'] / sales_with_seg['이용개월수_신용_R12M']
)

sales_with_seg['이용월평균이용건수_신판_R12M'] = np.where(
    sales_with_seg['이용개월수_신판_R12M'] == 0, 0,
    sales_with_seg['이용건수_신판_R12M'] / sales_with_seg['이용개월수_신판_R12M']
)

sales_with_seg['이용월평균이용건수_일시불_R12M'] = np.where(
    sales_with_seg['이용개월수_일시불_R12M'] == 0, 0,
    sales_with_seg['이용건수_일시불_R12M'] / sales_with_seg['이용개월수_일시불_R12M']
)

sales_with_seg['이용월평균이용건수_할부_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_R12M'] == 0, 0,
    sales_with_seg['이용건수_할부_R12M'] / sales_with_seg['이용개월수_할부_R12M']
)

sales_with_seg['이용월평균이용건수_할부_유이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R12M'] == 0, 0,
    sales_with_seg['이용건수_할부_유이자_R12M'] / sales_with_seg['이용개월수_할부_유이자_R12M']
)

sales_with_seg['이용월평균이용건수_할부_무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R12M'] == 0, 0,
    sales_with_seg['이용건수_할부_무이자_R12M'] / sales_with_seg['이용개월수_할부_무이자_R12M']
)

sales_with_seg['이용월평균이용건수_부분무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R12M'] == 0, 0,
    sales_with_seg['이용건수_부분무이자_R12M'] / sales_with_seg['이용개월수_부분무이자_R12M']
)

sales_with_seg['이용월평균이용건수_CA_R12M'] = np.where(
    sales_with_seg['이용개월수_CA_R12M'] == 0, 0,
    sales_with_seg['이용건수_CA_R12M'] / sales_with_seg['이용개월수_CA_R12M']
)

sales_with_seg['이용월평균이용건수_체크_R12M'] = np.where(
    sales_with_seg['이용개월수_체크_R12M'] == 0, 0,
    sales_with_seg['이용건수_체크_R12M'] / sales_with_seg['이용개월수_체크_R12M']
)

sales_with_seg['이용월평균이용건수_카드론_R12M'] = np.where(
    sales_with_seg['이용개월수_카드론_R12M'] == 0, 0,
    sales_with_seg['이용건수_카드론_R12M'] / sales_with_seg['이용개월수_카드론_R12M']
)

- R6M 변수

In [ ]:
##### 이용건수 #####

# '이용건수_신용_R6M' 변수 값 보정
sales_with_seg['이용건수_신용_R6M'] = sales_with_seg['이용건수_신용_R6M'] + sales_with_seg['이용건수_카드론_R6M']


##### 이용금액 #####

# '이용금액_신용_R6M', '이용금액_신판_R6M' 변수 생성
sales_with_seg['이용금액_신용_R6M'] = sales_with_seg['이용금액_일시불_R6M'] + sales_with_seg['이용금액_할부_R6M'] + sales_with_seg['이용금액_CA_R6M'] + sales_with_seg['이용금액_카드론_R6M']
sales_with_seg['이용금액_신판_R6M'] = sales_with_seg['이용금액_일시불_R6M'] + sales_with_seg['이용금액_할부_R6M']

In [ ]:
# 건수별평균이용금액 변수 생성 (R6M 기준)
sales_with_seg['건수별평균이용금액_신용_R6M'] = np.where(
    (sales_with_seg['이용건수_신용_R6M'] == 0) | (sales_with_seg['이용금액_신용_R6M'] == 0), 0,
    sales_with_seg['이용금액_신용_R6M'] / sales_with_seg['이용건수_신용_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_R6M'] = np.where(
    (sales_with_seg['이용건수_신판_R6M'] == 0) | (sales_with_seg['이용금액_신판_R6M'] == 0), 0,
    sales_with_seg['이용금액_신판_R6M'] / sales_with_seg['이용건수_신판_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_R6M'] = np.where(
    (sales_with_seg['이용건수_일시불_R6M'] == 0) | (sales_with_seg['이용금액_일시불_R6M'] == 0), 0,
    sales_with_seg['이용금액_일시불_R6M'] / sales_with_seg['이용건수_일시불_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_R6M'] = np.where(
    (sales_with_seg['이용건수_할부_R6M'] == 0) | (sales_with_seg['이용금액_할부_R6M'] == 0), 0,
    sales_with_seg['이용금액_할부_R6M'] / sales_with_seg['이용건수_할부_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_R6M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_R6M'] == 0) | (sales_with_seg['이용금액_할부_유이자_R6M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_R6M'] / sales_with_seg['이용건수_할부_유이자_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_R6M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_R6M'] == 0) | (sales_with_seg['이용금액_할부_무이자_R6M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_R6M'] / sales_with_seg['이용건수_할부_무이자_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_부분무이자_R6M'] = np.where(
    (sales_with_seg['이용건수_부분무이자_R6M'] == 0) | (sales_with_seg['이용금액_부분무이자_R6M'] == 0), 0,
    sales_with_seg['이용금액_부분무이자_R6M'] / sales_with_seg['이용건수_부분무이자_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_R6M'] = np.where(
    (sales_with_seg['이용건수_CA_R6M'] == 0) | (sales_with_seg['이용금액_CA_R6M'] == 0), 0,
    sales_with_seg['이용금액_CA_R6M'] / sales_with_seg['이용건수_CA_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_R6M'] = np.where(
    (sales_with_seg['이용건수_체크_R6M'] == 0) | (sales_with_seg['이용금액_체크_R6M'] == 0), 0,
    sales_with_seg['이용금액_체크_R6M'] / sales_with_seg['이용건수_체크_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_R6M'] = np.where(
    (sales_with_seg['이용건수_카드론_R6M'] == 0) | (sales_with_seg['이용금액_카드론_R6M'] == 0), 0,
    sales_with_seg['이용금액_카드론_R6M'] / sales_with_seg['이용건수_카드론_R6M'].abs()
)

In [ ]:
# 이용월평균이용금액 변수 생성 (R6M 기준)
sales_with_seg['이용월평균이용금액_신용_R6M'] = np.where(
    sales_with_seg['이용개월수_신용_R6M'] == 0, 0,
    sales_with_seg['이용금액_신용_R6M'] / sales_with_seg['이용개월수_신용_R6M']
)

sales_with_seg['이용월평균이용금액_신판_R6M'] = np.where(
    sales_with_seg['이용개월수_신판_R6M'] == 0, 0,
    sales_with_seg['이용금액_신판_R6M'] / sales_with_seg['이용개월수_신판_R6M']
)

sales_with_seg['이용월평균이용금액_일시불_R6M'] = np.where(
    sales_with_seg['이용개월수_일시불_R6M'] == 0, 0,
    sales_with_seg['이용금액_일시불_R6M'] / sales_with_seg['이용개월수_일시불_R6M']
)

sales_with_seg['이용월평균이용금액_할부_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_R6M'] == 0, 0,
    sales_with_seg['이용금액_할부_R6M'] / sales_with_seg['이용개월수_할부_R6M']
)

sales_with_seg['이용월평균이용금액_할부_유이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R6M'] == 0, 0,
    sales_with_seg['이용금액_할부_유이자_R6M'] / sales_with_seg['이용개월수_할부_유이자_R6M']
)

sales_with_seg['이용월평균이용금액_할부_무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R6M'] == 0, 0,
    sales_with_seg['이용금액_할부_무이자_R6M'] / sales_with_seg['이용개월수_할부_무이자_R6M']
)

sales_with_seg['이용월평균이용금액_부분무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R6M'] == 0, 0,
    sales_with_seg['이용금액_부분무이자_R6M'] / sales_with_seg['이용개월수_부분무이자_R6M']
)

sales_with_seg['이용월평균이용금액_CA_R6M'] = np.where(
    sales_with_seg['이용개월수_CA_R6M'] == 0, 0,
    sales_with_seg['이용금액_CA_R6M'] / sales_with_seg['이용개월수_CA_R6M']
)

sales_with_seg['이용월평균이용금액_체크_R6M'] = np.where(
    sales_with_seg['이용개월수_체크_R6M'] == 0, 0,
    sales_with_seg['이용금액_체크_R6M'] / sales_with_seg['이용개월수_체크_R6M']
)

sales_with_seg['이용월평균이용금액_카드론_R6M'] = np.where(
    sales_with_seg['이용개월수_카드론_R6M'] == 0, 0,
    sales_with_seg['이용금액_카드론_R6M'] / sales_with_seg['이용개월수_카드론_R6M']
)

In [ ]:
# 이용월평균이용건수 변수 생성 (R6M 기준)
sales_with_seg['이용월평균이용건수_신용_R6M'] = np.where(
    sales_with_seg['이용개월수_신용_R6M'] == 0, 0,
    sales_with_seg['이용건수_신용_R6M'] / sales_with_seg['이용개월수_신용_R6M']
)

sales_with_seg['이용월평균이용건수_신판_R6M'] = np.where(
    sales_with_seg['이용개월수_신판_R6M'] == 0, 0,
    sales_with_seg['이용건수_신판_R6M'] / sales_with_seg['이용개월수_신판_R6M']
)

sales_with_seg['이용월평균이용건수_일시불_R6M'] = np.where(
    sales_with_seg['이용개월수_일시불_R6M'] == 0, 0,
    sales_with_seg['이용건수_일시불_R6M'] / sales_with_seg['이용개월수_일시불_R6M']
)

sales_with_seg['이용월평균이용건수_할부_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_R6M'] == 0, 0,
    sales_with_seg['이용건수_할부_R6M'] / sales_with_seg['이용개월수_할부_R6M']
)

sales_with_seg['이용월평균이용건수_할부_유이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R6M'] == 0, 0,
    sales_with_seg['이용건수_할부_유이자_R6M'] / sales_with_seg['이용개월수_할부_유이자_R6M']
)

sales_with_seg['이용월평균이용건수_할부_무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R6M'] == 0, 0,
    sales_with_seg['이용건수_할부_무이자_R6M'] / sales_with_seg['이용개월수_할부_무이자_R6M']
)

sales_with_seg['이용월평균이용건수_부분무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R6M'] == 0, 0,
    sales_with_seg['이용건수_부분무이자_R6M'] / sales_with_seg['이용개월수_부분무이자_R6M']
)

sales_with_seg['이용월평균이용건수_CA_R6M'] = np.where(
    sales_with_seg['이용개월수_CA_R6M'] == 0, 0,
    sales_with_seg['이용건수_CA_R6M'] / sales_with_seg['이용개월수_CA_R6M']
)

sales_with_seg['이용월평균이용건수_체크_R6M'] = np.where(
    sales_with_seg['이용개월수_체크_R6M'] == 0, 0,
    sales_with_seg['이용건수_체크_R6M'] / sales_with_seg['이용개월수_체크_R6M']
)

sales_with_seg['이용월평균이용건수_카드론_R6M'] = np.where(
    sales_with_seg['이용개월수_카드론_R6M'] == 0, 0,
    sales_with_seg['이용건수_카드론_R6M'] / sales_with_seg['이용개월수_카드론_R6M']
)

<ipython-input-26-14d78e973320>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_할부_유이자_R6M'] = np.where(
<ipython-input-26-14d78e973320>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_할부_무이자_R6M'] = np.where(
<ipython-input-26-14d78e973320>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fra

- R3M 변수

In [ ]:
##### 이용건수 #####

# '이용건수_신용_R3M' 변수 값 보정
sales_with_seg['이용건수_신용_R3M'] = sales_with_seg['이용건수_신용_R3M'] + sales_with_seg['이용건수_카드론_R3M']


##### 이용금액 #####

# '이용금액_신용_R3M', '이용금액_신판_R3M' 변수 생성
sales_with_seg['이용금액_신용_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M'] + sales_with_seg['이용금액_CA_R3M'] + sales_with_seg['이용금액_카드론_R3M']
sales_with_seg['이용금액_신판_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M']


<ipython-input-27-4d9390fc2cd0>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용금액_신용_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M'] + sales_with_seg['이용금액_CA_R3M'] + sales_with_seg['이용금액_카드론_R3M']
<ipython-input-27-4d9390fc2cd0>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용금액_신판_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M']


In [ ]:
# 건수별평균이용금액 변수 생성 (R3M 기준)
sales_with_seg['건수별평균이용금액_신용_R3M'] = np.where(
    (sales_with_seg['이용건수_신용_R3M'] == 0) | (sales_with_seg['이용금액_신용_R3M'] == 0), 0,
    sales_with_seg['이용금액_신용_R3M'] / sales_with_seg['이용건수_신용_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_R3M'] = np.where(
    (sales_with_seg['이용건수_신판_R3M'] == 0) | (sales_with_seg['이용금액_신판_R3M'] == 0), 0,
    sales_with_seg['이용금액_신판_R3M'] / sales_with_seg['이용건수_신판_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_R3M'] = np.where(
    (sales_with_seg['이용건수_일시불_R3M'] == 0) | (sales_with_seg['이용금액_일시불_R3M'] == 0), 0,
    sales_with_seg['이용금액_일시불_R3M'] / sales_with_seg['이용건수_일시불_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_R3M'] = np.where(
    (sales_with_seg['이용건수_할부_R3M'] == 0) | (sales_with_seg['이용금액_할부_R3M'] == 0), 0,
    sales_with_seg['이용금액_할부_R3M'] / sales_with_seg['이용건수_할부_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_R3M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_R3M'] == 0) | (sales_with_seg['이용금액_할부_유이자_R3M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_R3M'] / sales_with_seg['이용건수_할부_유이자_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_R3M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_R3M'] == 0) | (sales_with_seg['이용금액_할부_무이자_R3M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_R3M'] / sales_with_seg['이용건수_할부_무이자_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_부분무이자_R3M'] = np.where(
    (sales_with_seg['이용건수_부분무이자_R3M'] == 0) | (sales_with_seg['이용금액_부분무이자_R3M'] == 0), 0,
    sales_with_seg['이용금액_부분무이자_R3M'] / sales_with_seg['이용건수_부분무이자_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_R3M'] = np.where(
    (sales_with_seg['이용건수_CA_R3M'] == 0) | (sales_with_seg['이용금액_CA_R3M'] == 0), 0,
    sales_with_seg['이용금액_CA_R3M'] / sales_with_seg['이용건수_CA_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_R3M'] = np.where(
    (sales_with_seg['이용건수_체크_R3M'] == 0) | (sales_with_seg['이용금액_체크_R3M'] == 0), 0,
    sales_with_seg['이용금액_체크_R3M'] / sales_with_seg['이용건수_체크_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_R3M'] = np.where(
    (sales_with_seg['이용건수_카드론_R3M'] == 0) | (sales_with_seg['이용금액_카드론_R3M'] == 0), 0,
    sales_with_seg['이용금액_카드론_R3M'] / sales_with_seg['이용건수_카드론_R3M'].abs()
)

<ipython-input-28-8a8fe6265eaa>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_신용_R3M'] = np.where(
<ipython-input-28-8a8fe6265eaa>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_신판_R3M'] = np.where(
<ipython-input-28-8a8fe6265eaa>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

In [ ]:
# 이용월평균이용금액 변수 생성 (R3M 기준)
sales_with_seg['이용월평균이용금액_신용_R3M'] = np.where(
    sales_with_seg['이용개월수_신용_R3M'] == 0, 0,
    sales_with_seg['이용금액_신용_R3M'] / sales_with_seg['이용개월수_신용_R3M']
)

sales_with_seg['이용월평균이용금액_신판_R3M'] = np.where(
    sales_with_seg['이용개월수_신판_R3M'] == 0, 0,
    sales_with_seg['이용금액_신판_R3M'] / sales_with_seg['이용개월수_신판_R3M']
)

sales_with_seg['이용월평균이용금액_일시불_R3M'] = np.where(
    sales_with_seg['이용개월수_일시불_R3M'] == 0, 0,
    sales_with_seg['이용금액_일시불_R3M'] / sales_with_seg['이용개월수_일시불_R3M']
)

sales_with_seg['이용월평균이용금액_할부_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_R3M'] == 0, 0,
    sales_with_seg['이용금액_할부_R3M'] / sales_with_seg['이용개월수_할부_R3M']
)

sales_with_seg['이용월평균이용금액_할부_유이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R3M'] == 0, 0,
    sales_with_seg['이용금액_할부_유이자_R3M'] / sales_with_seg['이용개월수_할부_유이자_R3M']
)

sales_with_seg['이용월평균이용금액_할부_무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R3M'] == 0, 0,
    sales_with_seg['이용금액_할부_무이자_R3M'] / sales_with_seg['이용개월수_할부_무이자_R3M']
)

sales_with_seg['이용월평균이용금액_부분무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R3M'] == 0, 0,
    sales_with_seg['이용금액_부분무이자_R3M'] / sales_with_seg['이용개월수_부분무이자_R3M']
)

sales_with_seg['이용월평균이용금액_CA_R3M'] = np.where(
    sales_with_seg['이용개월수_CA_R3M'] == 0, 0,
    sales_with_seg['이용금액_CA_R3M'] / sales_with_seg['이용개월수_CA_R3M']
)

sales_with_seg['이용월평균이용금액_체크_R3M'] = np.where(
    sales_with_seg['이용개월수_체크_R3M'] == 0, 0,
    sales_with_seg['이용금액_체크_R3M'] / sales_with_seg['이용개월수_체크_R3M']
)

sales_with_seg['이용월평균이용금액_카드론_R3M'] = np.where(
    sales_with_seg['이용개월수_카드론_R3M'] == 0, 0,
    sales_with_seg['이용금액_카드론_R3M'] / sales_with_seg['이용개월수_카드론_R3M']
)

<ipython-input-29-c32a9b21fcfa>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용금액_신용_R3M'] = np.where(
<ipython-input-29-c32a9b21fcfa>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용금액_신판_R3M'] = np.where(
<ipython-input-29-c32a9b21fcfa>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

In [ ]:
# 이용월평균이용건수 변수 생성 (R3M 기준)
sales_with_seg['이용월평균이용건수_신용_R3M'] = np.where(
    sales_with_seg['이용개월수_신용_R3M'] == 0, 0,
    sales_with_seg['이용건수_신용_R3M'] / sales_with_seg['이용개월수_신용_R3M']
)

sales_with_seg['이용월평균이용건수_신판_R3M'] = np.where(
    sales_with_seg['이용개월수_신판_R3M'] == 0, 0,
    sales_with_seg['이용건수_신판_R3M'] / sales_with_seg['이용개월수_신판_R3M']
)

sales_with_seg['이용월평균이용건수_일시불_R3M'] = np.where(
    sales_with_seg['이용개월수_일시불_R3M'] == 0, 0,
    sales_with_seg['이용건수_일시불_R3M'] / sales_with_seg['이용개월수_일시불_R3M']
)

sales_with_seg['이용월평균이용건수_할부_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_R3M'] == 0, 0,
    sales_with_seg['이용건수_할부_R3M'] / sales_with_seg['이용개월수_할부_R3M']
)

sales_with_seg['이용월평균이용건수_할부_유이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R3M'] == 0, 0,
    sales_with_seg['이용건수_할부_유이자_R3M'] / sales_with_seg['이용개월수_할부_유이자_R3M']
)

sales_with_seg['이용월평균이용건수_할부_무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R3M'] == 0, 0,
    sales_with_seg['이용건수_할부_무이자_R3M'] / sales_with_seg['이용개월수_할부_무이자_R3M']
)

sales_with_seg['이용월평균이용건수_부분무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R3M'] == 0, 0,
    sales_with_seg['이용건수_부분무이자_R3M'] / sales_with_seg['이용개월수_부분무이자_R3M']
)

sales_with_seg['이용월평균이용건수_CA_R3M'] = np.where(
    sales_with_seg['이용개월수_CA_R3M'] == 0, 0,
    sales_with_seg['이용건수_CA_R3M'] / sales_with_seg['이용개월수_CA_R3M']
)

sales_with_seg['이용월평균이용건수_체크_R3M'] = np.where(
    sales_with_seg['이용개월수_체크_R3M'] == 0, 0,
    sales_with_seg['이용건수_체크_R3M'] / sales_with_seg['이용개월수_체크_R3M']
)

sales_with_seg['이용월평균이용건수_카드론_R3M'] = np.where(
    sales_with_seg['이용개월수_카드론_R3M'] == 0, 0,
    sales_with_seg['이용건수_카드론_R3M'] / sales_with_seg['이용개월수_카드론_R3M']
)


<ipython-input-30-cb1d1abb0035>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_신용_R3M'] = np.where(
<ipython-input-30-cb1d1abb0035>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_신판_R3M'] = np.where(
<ipython-input-30-cb1d1abb0035>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

- 쇼핑/교통/여유/납부

In [ ]:
# '쇼핑_총이용금액' 변수 생성
shopping_columns = ['쇼핑_도소매_이용금액',
                    '쇼핑_백화점_이용금액',
                    '쇼핑_마트_이용금액',
                    '쇼핑_슈퍼마켓_이용금액',
                    '쇼핑_편의점_이용금액',
                    '쇼핑_아울렛_이용금액',
                    '쇼핑_온라인_이용금액',
                    '쇼핑_기타_이용금액']
sales_with_seg['쇼핑_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

# '교통_총이용금액' 변수 생성
shopping_columns = ['교통_주유이용금액',
                    '교통_정비이용금액',
                    '교통_통행료이용금액',
                    '교통_버스지하철이용금액',
                    '교통_택시이용금액',
                    '교통_철도버스이용금액']
sales_with_seg['교통_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

# '여유_총이용금액' 변수 생성
shopping_columns = ['여유_운동이용금액',
                    '여유_Pet이용금액',
                    '여유_공연이용금액',
                    '여유_공원이용금액',
                    '여유_숙박이용금액',
                    '여유_항공이용금액',
                    '여유_기타이용금액']
sales_with_seg['여유_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

# '납부_총이용금액' 변수 생성
shopping_columns = ['납부_통신비이용금액',
                    '납부_관리비이용금액',
                    '납부_가스전기료이용금액',
                    '납부_보험료이용금액',
                    '납부_기타이용금액']
sales_with_seg['납부_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

<ipython-input-31-406ff2616aa4>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['쇼핑_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)
<ipython-input-31-406ff2616aa4>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['교통_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)
<ipython-input-31-406ff2616aa4>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once us

In [ ]:
# 상관계수 절댓값 0.3 넘는 변수 5개만 합치기
shopping_columns = ['쇼핑_도소매_이용금액',
                    '쇼핑_마트_이용금액',
                    '쇼핑_온라인_이용금액',
                    '교통_주유이용금액',
                    '납부_기타이용금액']
sales_with_seg['0.3이상_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

<ipython-input-32-b7ebe65e3d8d>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['0.3이상_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)


In [ ]:
# 쇼핑/교통/여유/납부 다 합치기
columns = ['쇼핑_총이용금액',
          '교통_총이용금액',
          '여유_총이용금액',
          '납부_총이용금액']
sales_with_seg['총이용금액_쇼핑교통여유납부'] = sales_with_seg[columns].sum(axis=1)

<ipython-input-33-22bf84db1bde>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['총이용금액_쇼핑교통여유납부'] = sales_with_seg[columns].sum(axis=1)


In [ ]:
# 기존 변수 다 삭제
columns_to_drop = [
    '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액',
    '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액',
    '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액',

    '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액',
    '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액',

    '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액',
    '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액',
    '여유_항공이용금액', '여유_기타이용금액',

    '납부_통신비이용금액', '납부_관리비이용금액', '납부_렌탈료이용금액',
    '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_유선방송이용금액',
    '납부_건강연금이용금액', '납부_기타이용금액'
]

# 변수 삭제
sales_with_seg.drop(columns=columns_to_drop, inplace=True)

-  R12M 할부건수/할부금액

In [ ]:
# 데이터 동일한 변수 4개 삭제
columns_drop = [
    '할부건수_부분_3M_R12M',
    '할부건수_부분_6M_R12M',
    '할부건수_부분_14M_R12M',
    '할부금액_부분_3M_R12M'
    ]
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 최근 1년치 총합 변수 7개 생성 (할부건수/할부금액)

# 1. 할부
sales_with_seg['할부건수총합_R12M'] = (
    sales_with_seg['할부건수_3M_R12M'] +
    sales_with_seg['할부건수_6M_R12M'] +
    sales_with_seg['할부건수_12M_R12M']
)

sales_with_seg['할부금액총합_R12M'] = (
    sales_with_seg['할부금액_3M_R12M'] +
    sales_with_seg['할부금액_6M_R12M'] +
    sales_with_seg['할부금액_12M_R12M']
)

# 2. 유이자 할부
sales_with_seg['할부건수총합_유이자_R12M'] = (
    sales_with_seg['할부건수_유이자_3M_R12M'] +
    sales_with_seg['할부건수_유이자_6M_R12M'] +
    sales_with_seg['할부건수_유이자_12M_R12M']
)

sales_with_seg['할부금액총합_유이자_R12M'] = (
    sales_with_seg['할부금액_유이자_3M_R12M'] +
    sales_with_seg['할부금액_유이자_6M_R12M'] +
    sales_with_seg['할부금액_유이자_12M_R12M']
)

# 3. 무이자 할부
sales_with_seg['할부건수총합_무이자_R12M'] = (
    sales_with_seg['할부건수_무이자_3M_R12M'] +
    sales_with_seg['할부건수_무이자_6M_R12M'] +
    sales_with_seg['할부건수_무이자_12M_R12M']
)

sales_with_seg['할부금액총합_무이자_R12M'] = (
    sales_with_seg['할부금액_무이자_3M_R12M'] +
    sales_with_seg['할부금액_무이자_6M_R12M'] +
    sales_with_seg['할부금액_무이자_12M_R12M']
)

# 3. 부분 할부
# '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M'가 다 0이라서 '할부건수_부분_12M_R12M'가 1년치 총합과 동일함
sales_with_seg['할부금액총합_부분_R12M'] = (
    sales_with_seg['할부금액_부분_6M_R12M'] +
    sales_with_seg['할부금액_부분_12M_R12M']
)

<ipython-input-36-a57ce0275030>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['할부건수총합_R12M'] = (
<ipython-input-36-a57ce0275030>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['할부금액총합_R12M'] = (
<ipython-input-36-a57ce0275030>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fram

In [ ]:
# 할부 건수 별 금액 변수 4개 생성
#  - 3M 기준이 제일 상관계수가 좋음
#  - 부분 무이자 할부는 할부건수가 12M 밖에 없어서 12M 기준으로 계산

# 1. 할부
sales_with_seg['건수별할부금액_3M'] = np.where(
    (sales_with_seg['할부금액_3M_R12M'] == 0) | (sales_with_seg['할부건수_3M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_3M_R12M'] / sales_with_seg['할부건수_3M_R12M']
)

# 2. 유이자 할부
sales_with_seg['건수별할부금액_유이자_3M'] = np.where(
    (sales_with_seg['할부금액_유이자_3M_R12M'] == 0) | (sales_with_seg['할부건수_유이자_3M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_유이자_3M_R12M'] / sales_with_seg['할부건수_유이자_3M_R12M']
)

# 3. 무이자 할부
sales_with_seg['건수별할부금액_무이자_3M'] = np.where(
    (sales_with_seg['할부금액_무이자_3M_R12M'] == 0) | (sales_with_seg['할부건수_무이자_3M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_무이자_3M_R12M'] / sales_with_seg['할부건수_무이자_3M_R12M']
)

# 4. 부분 무이자 할부
sales_with_seg['건수별할부금액_부분_12M'] = np.where(
    (sales_with_seg['할부금액_부분_12M_R12M'] == 0) | (sales_with_seg['할부건수_부분_12M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_부분_12M_R12M'] / sales_with_seg['할부건수_부분_12M_R12M']
)

<ipython-input-37-fbf2bfb45a75>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별할부금액_3M'] = np.where(
<ipython-input-37-fbf2bfb45a75>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별할부금액_유이자_3M'] = np.where(
<ipython-input-37-fbf2bfb45a75>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

- RP 관련 변수들

In [ ]:
# 데이터 동일한 변수 6개 삭제
columns_drop = ['RP건수_유선방송_B0M',
                'RP건수_건강_B0M',
                'RP후경과월_유선방송',
                'RP후경과월_건강',
                '증감_RP건수_유선방송_전월',
                '증감_RP건수_건강_전월']
sales_with_seg.drop(columns=columns_drop, inplace=True)

# '최근3개월이용여부_RP' 변수 생성
sales_with_seg['최근3개월이용여부_RP'] = (sales_with_seg['RP후경과월'] <= 3).astype(int)

# 'RP_이용확대여부' 변수 생성
delta_cols = [col for col in sales_with_seg.columns if col.startswith('증감_RP건수_')]
sales_with_seg['RP건수_증감총합'] = sales_with_seg[delta_cols].sum(axis=1)
sales_with_seg['RP_이용확대여부'] = (sales_with_seg['RP건수_증감총합'] > 0).astype(int) # 방향성: 양수면 확장, 음수면 축소
sales_with_seg.drop(columns=['RP건수_증감총합'], inplace=True)

<ipython-input-38-071d93968994>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['최근3개월이용여부_RP'] = (sales_with_seg['RP후경과월'] <= 3).astype(int)
<ipython-input-38-071d93968994>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['RP건수_증감총합'] = sales_with_seg[delta_cols].sum(axis=1)
<ipython-input-38-071d93968994>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

In [ ]:
# 'RP활성업종수' 변수 생성
rp_cols = [
    'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M',
    'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M',
    'RP건수_학습비_B0M', 'RP건수_교통_B0M'
]
sales_with_seg['RP활성업종수'] = sales_with_seg[rp_cols].gt(0).sum(axis=1)

<ipython-input-39-a8a35245fb88>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['RP활성업종수'] = sales_with_seg[rp_cols].gt(0).sum(axis=1)


- 카드론 관련 변수들

In [ ]:
# '카드론이용여부' 변수 생성
sales_with_seg['카드론이용여부'] = (
    (sales_with_seg['최초카드론이용경과월'] != 999) &
    (sales_with_seg['최종카드론이용경과월'] != 999)
).astype(int)

<ipython-input-40-781516af5b95>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['카드론이용여부'] = (


- 이용개월수/이용금액/이용건수 - 온라인/오프라인

In [ ]:
# 건수별평균금액 변수 6개 생성

# R6M(최근 6개월)
sales_with_seg['건수별평균이용금액_온라인_R6M'] = np.where(
    (sales_with_seg['이용금액_온라인_R6M'] == 0) | (sales_with_seg['이용건수_온라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_온라인_R6M'] / sales_with_seg['이용건수_온라인_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_오프라인_R6M'] = np.where(
    (sales_with_seg['이용금액_오프라인_R6M'] == 0) | (sales_with_seg['이용건수_오프라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_오프라인_R6M'] / sales_with_seg['이용건수_오프라인_R6M'].abs()
)

# R3M(최근 3개월)
sales_with_seg['건수별평균이용금액_온라인_R3M'] = np.where(
    (sales_with_seg['이용금액_온라인_R3M'] == 0) | (sales_with_seg['이용건수_온라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_온라인_R3M'] / sales_with_seg['이용건수_온라인_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_오프라인_R3M'] = np.where(
    (sales_with_seg['이용금액_오프라인_R3M'] == 0) | (sales_with_seg['이용건수_오프라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_오프라인_R3M'] / sales_with_seg['이용건수_오프라인_R3M'].abs()
)


# B0M(당월)
sales_with_seg['건수별평균이용금액_온라인_B0M'] = np.where(
    (sales_with_seg['이용금액_온라인_B0M'] == 0) | (sales_with_seg['이용건수_온라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_온라인_B0M'] / sales_with_seg['이용건수_온라인_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_오프라인_B0M'] = np.where(
    (sales_with_seg['이용금액_오프라인_B0M'] == 0) | (sales_with_seg['이용건수_오프라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_오프라인_B0M'] / sales_with_seg['이용건수_오프라인_B0M'].abs()
)

<ipython-input-41-9fcf63bd8b3f>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_온라인_R6M'] = np.where(
<ipython-input-41-9fcf63bd8b3f>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_오프라인_R6M'] = np.where(
<ipython-input-41-9fcf63bd8b3f>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmente

- 이용개월수/이용금액/이용건수 - 페이_온라인/오프라인

In [ ]:
# 건수별평균이용금액 변수 생성

# R6M (최근 6개월)
sales_with_seg['건수별평균이용금액_페이_온라인_R6M'] = np.where(
    (sales_with_seg['이용금액_페이_온라인_R6M'] == 0) | (sales_with_seg['이용건수_페이_온라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_페이_온라인_R6M'] / sales_with_seg['이용건수_페이_온라인_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_페이_오프라인_R6M'] = np.where(
    (sales_with_seg['이용금액_페이_오프라인_R6M'] == 0) | (sales_with_seg['이용건수_페이_오프라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_페이_오프라인_R6M'] / sales_with_seg['이용건수_페이_오프라인_R6M'].abs()
)

# R3M (최근 3개월)
sales_with_seg['건수별평균이용금액_페이_온라인_R3M'] = np.where(
    (sales_with_seg['이용금액_페이_온라인_R3M'] == 0) | (sales_with_seg['이용건수_페이_온라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_페이_온라인_R3M'] / sales_with_seg['이용건수_페이_온라인_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_페이_오프라인_R3M'] = np.where(
    (sales_with_seg['이용금액_페이_오프라인_R3M'] == 0) | (sales_with_seg['이용건수_페이_오프라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_페이_오프라인_R3M'] / sales_with_seg['이용건수_페이_오프라인_R3M'].abs()
)

# B0M (당월)
sales_with_seg['건수별평균이용금액_페이_온라인_B0M'] = np.where(
    (sales_with_seg['이용금액_페이_온라인_B0M'] == 0) | (sales_with_seg['이용건수_페이_온라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_페이_온라인_B0M'] / sales_with_seg['이용건수_페이_온라인_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_페이_오프라인_B0M'] = np.where(
    (sales_with_seg['이용금액_페이_오프라인_B0M'] == 0) | (sales_with_seg['이용건수_페이_오프라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_페이_오프라인_B0M'] / sales_with_seg['이용건수_페이_오프라인_B0M'].abs()
)

<ipython-input-42-1b4362d6e054>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_페이_온라인_R6M'] = np.where(
<ipython-input-42-1b4362d6e054>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_페이_오프라인_R6M'] = np.where(
<ipython-input-42-1b4362d6e054>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fra

- 이용개월수/이용금액/이용건수 -  간편결제/당사페이/당사기타/A,B,C,D페이

- R6M

In [ ]:
# 데이터 동일한 변수 5개 삭제
columns_drop = ['이용개월수_당사페이_R6M',
                '이용금액_당사페이_R6M',
                '이용금액_당사기타_R6M',
                '이용건수_당사페이_R6M',
                '이용건수_당사기타_R6M']
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 건수별평균이용금액 변수 5개 생성 (R6M 기준)

# 간편결제
sales_with_seg['건수별평균이용금액_간편결제_R6M'] = np.where(
    (sales_with_seg['이용금액_간편결제_R6M'] == 0) | (sales_with_seg['이용건수_간편결제_R6M'] == 0),
    0,
    sales_with_seg['이용금액_간편결제_R6M'] / sales_with_seg['이용건수_간편결제_R6M'].abs()
)

# A페이
sales_with_seg['건수별평균이용금액_A페이_R6M'] = np.where(
    (sales_with_seg['이용금액_A페이_R6M'] == 0) | (sales_with_seg['이용건수_A페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_A페이_R6M'] / sales_with_seg['이용건수_A페이_R6M'].abs()
)

# B페이
sales_with_seg['건수별평균이용금액_B페이_R6M'] = np.where(
    (sales_with_seg['이용금액_B페이_R6M'] == 0) | (sales_with_seg['이용건수_B페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_B페이_R6M'] / sales_with_seg['이용건수_B페이_R6M'].abs()
)

# C페이
sales_with_seg['건수별평균이용금액_C페이_R6M'] = np.where(
    (sales_with_seg['이용금액_C페이_R6M'] == 0) | (sales_with_seg['이용건수_C페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_C페이_R6M'] / sales_with_seg['이용건수_C페이_R6M'].abs()
)

# D페이
sales_with_seg['건수별평균이용금액_D페이_R6M'] = np.where(
    (sales_with_seg['이용금액_D페이_R6M'] == 0) | (sales_with_seg['이용건수_D페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_D페이_R6M'] / sales_with_seg['이용건수_D페이_R6M'].abs()
)

<ipython-input-44-154095dbc84a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_간편결제_R6M'] = np.where(
<ipython-input-44-154095dbc84a>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_A페이_R6M'] = np.where(
<ipython-input-44-154095dbc84a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmente

- R3M

In [ ]:
# 데이터 동일한 변수 4개 삭제
columns_drop = ['이용금액_당사페이_R3M',
                '이용금액_당사기타_R3M',
                '이용건수_당사페이_R3M',
                '이용건수_당사기타_R3M']
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 건수별평균이용금액 변수 5개 생성 (R3M 기준)

# 간편결제
sales_with_seg['건수별평균이용금액_간편결제_R3M'] = np.where(
    (sales_with_seg['이용금액_간편결제_R3M'] == 0) | (sales_with_seg['이용건수_간편결제_R3M'] == 0),
    0,
    sales_with_seg['이용금액_간편결제_R3M'] / sales_with_seg['이용건수_간편결제_R3M'].abs()
)

# A페이
sales_with_seg['건수별평균이용금액_A페이_R3M'] = np.where(
    (sales_with_seg['이용금액_A페이_R3M'] == 0) | (sales_with_seg['이용건수_A페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_A페이_R3M'] / sales_with_seg['이용건수_A페이_R3M'].abs()
)

# B페이
sales_with_seg['건수별평균이용금액_B페이_R3M'] = np.where(
    (sales_with_seg['이용금액_B페이_R3M'] == 0) | (sales_with_seg['이용건수_B페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_B페이_R3M'] / sales_with_seg['이용건수_B페이_R3M'].abs()
)

# C페이
sales_with_seg['건수별평균이용금액_C페이_R3M'] = np.where(
    (sales_with_seg['이용금액_C페이_R3M'] == 0) | (sales_with_seg['이용건수_C페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_C페이_R3M'] / sales_with_seg['이용건수_C페이_R3M'].abs()
)

# D페이
sales_with_seg['건수별평균이용금액_D페이_R3M'] = np.where(
    (sales_with_seg['이용금액_D페이_R3M'] == 0) | (sales_with_seg['이용건수_D페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_D페이_R3M'] / sales_with_seg['이용건수_D페이_R3M'].abs()
)

<ipython-input-46-c3a97c739420>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_간편결제_R3M'] = np.where(
<ipython-input-46-c3a97c739420>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_A페이_R3M'] = np.where(
<ipython-input-46-c3a97c739420>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmente

- B0M

In [ ]:
# 데이터 동일한 변수 4개 삭제
columns_drop = ['이용금액_당사페이_B0M',
                '이용금액_당사기타_B0M',
                '이용건수_당사페이_B0M',
                '이용건수_당사기타_B0M']
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 건수별평균이용금액 변수 5개 생성 (B0M 기준)

# 간편결제
sales_with_seg['건수별평균이용금액_간편결제_B0M'] = np.where(
    (sales_with_seg['이용금액_간편결제_B0M'] == 0) | (sales_with_seg['이용건수_간편결제_B0M'] == 0),
    0,
    sales_with_seg['이용금액_간편결제_B0M'] / sales_with_seg['이용건수_간편결제_B0M'].abs()
)

# A페이
sales_with_seg['건수별평균이용금액_A페이_B0M'] = np.where(
    (sales_with_seg['이용금액_A페이_B0M'] == 0) | (sales_with_seg['이용건수_A페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_A페이_B0M'] / sales_with_seg['이용건수_A페이_B0M'].abs()
)

# B페이
sales_with_seg['건수별평균이용금액_B페이_B0M'] = np.where(
    (sales_with_seg['이용금액_B페이_B0M'] == 0) | (sales_with_seg['이용건수_B페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_B페이_B0M'] / sales_with_seg['이용건수_B페이_B0M'].abs()
)

# C페이
sales_with_seg['건수별평균이용금액_C페이_B0M'] = np.where(
    (sales_with_seg['이용금액_C페이_B0M'] == 0) | (sales_with_seg['이용건수_C페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_C페이_B0M'] / sales_with_seg['이용건수_C페이_B0M'].abs()
)

# D페이
sales_with_seg['건수별평균이용금액_D페이_B0M'] = np.where(
    (sales_with_seg['이용금액_D페이_B0M'] == 0) | (sales_with_seg['이용건수_D페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_D페이_B0M'] / sales_with_seg['이용건수_D페이_B0M'].abs()
)

<ipython-input-48-51b69cc8c592>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_간편결제_B0M'] = np.where(
<ipython-input-48-51b69cc8c592>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_A페이_B0M'] = np.where(
<ipython-input-48-51b69cc8c592>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmente

- 선결제

In [ ]:
# 횟수별/건수별 평균이용금액

# R6M
sales_with_seg['횟수별평균이용금액_선결제_R6M'] = np.where(
    (sales_with_seg['이용금액_선결제_R6M'] == 0) | (sales_with_seg['이용횟수_선결제_R6M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R6M'] / sales_with_seg['이용횟수_선결제_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_선결제_R6M'] = np.where(
    (sales_with_seg['이용금액_선결제_R6M'] == 0) | (sales_with_seg['이용건수_선결제_R6M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R6M'] / sales_with_seg['이용건수_선결제_R6M'].abs()
)

# R3M
sales_with_seg['횟수별평균이용금액_선결제_R3M'] = np.where(
    (sales_with_seg['이용금액_선결제_R3M'] == 0) | (sales_with_seg['이용횟수_선결제_R3M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R3M'] / sales_with_seg['이용횟수_선결제_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_선결제_R3M'] = np.where(
    (sales_with_seg['이용금액_선결제_R3M'] == 0) | (sales_with_seg['이용건수_선결제_R3M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R3M'] / sales_with_seg['이용건수_선결제_R3M'].abs()
)

# B0M
sales_with_seg['횟수별평균이용금액_선결제_B0M'] = np.where(
    (sales_with_seg['이용금액_선결제_B0M'] == 0) | (sales_with_seg['이용횟수_선결제_B0M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_B0M'] / sales_with_seg['이용횟수_선결제_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_선결제_B0M'] = np.where(
    (sales_with_seg['이용금액_선결제_B0M'] == 0) | (sales_with_seg['이용건수_선결제_B0M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_B0M'] / sales_with_seg['이용건수_선결제_B0M'].abs()
)

<ipython-input-49-f74a7dbb1d59>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['횟수별평균이용금액_선결제_R6M'] = np.where(
<ipython-input-49-f74a7dbb1d59>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_선결제_R6M'] = np.where(
<ipython-input-49-f74a7dbb1d59>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

- 연체

In [ ]:
# 횟수별평균이용금액 변수 생성

# R6M
sales_with_seg['횟수별평균이용금액_연체_R6M'] = np.where(
    (sales_with_seg['이용금액_연체_R6M'] == 0) | (sales_with_seg['이용횟수_연체_R6M'] == 0),
    0,
    sales_with_seg['이용금액_연체_R6M'] / sales_with_seg['이용횟수_연체_R6M'].abs()
)

# R3M
sales_with_seg['횟수별평균이용금액_연체_R3M'] = np.where(
    (sales_with_seg['이용금액_연체_R3M'] == 0) | (sales_with_seg['이용횟수_연체_R3M'] == 0),
    0,
    sales_with_seg['이용금액_연체_R3M'] / sales_with_seg['이용횟수_연체_R3M'].abs()
)

# B0M
sales_with_seg['횟수별평균이용금액_연체_B0M'] = np.where(
    (sales_with_seg['이용금액_연체_B0M'] == 0) | (sales_with_seg['이용횟수_연체_B0M'] == 0),
    0,
    sales_with_seg['이용금액_연체_B0M'] / sales_with_seg['이용횟수_연체_B0M'].abs()
)

<ipython-input-50-d7558ba4bb37>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['횟수별평균이용금액_연체_R6M'] = np.where(
<ipython-input-50-d7558ba4bb37>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['횟수별평균이용금액_연체_R3M'] = np.where(
<ipython-input-50-d7558ba4bb37>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

- 입금원금 관련 변수

In [ ]:
# 총납부금 변수 및 납부비율 변수 생성
for month in ['B0M', 'B2M', 'B5M']:
    sales_with_seg[f'총납부금_{month}'] = (
        sales_with_seg[f'선입금원금_{month}'] +
        sales_with_seg[f'정상입금원금_{month}'] +
        sales_with_seg[f'연체입금원금_{month}']
    )

    sales_with_seg[f'납부비율_{month}'] = np.where(
        sales_with_seg[f'정상청구원금_{month}'] == 0,
        0,
        sales_with_seg[f'총납부금_{month}'] / sales_with_seg[f'정상청구원금_{month}']
    )

<ipython-input-51-a792eb1a16b8>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'총납부금_{month}'] = (
<ipython-input-51-a792eb1a16b8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'납부비율_{month}'] = np.where(
<ipython-input-51-a792eb1a16b8>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

In [ ]:
# 정시납부금 및 정시납부비율 변수 생성
for month in ['B0M', 'B2M', 'B5M']:
    sales_with_seg[f'정시납부금_{month}'] = (
        sales_with_seg[f'선입금원금_{month}'] +
        sales_with_seg[f'정상입금원금_{month}']
    )

    sales_with_seg[f'정시납부비율_{month}'] = np.where(
        sales_with_seg[f'정상청구원금_{month}'] == 0,
        0,
        sales_with_seg[f'정시납부금_{month}'] / sales_with_seg[f'정상청구원금_{month}']
    )

<ipython-input-52-d151c7f70dda>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'정시납부금_{month}'] = (
<ipython-input-52-d151c7f70dda>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'정시납부비율_{month}'] = np.where(
<ipython-input-52-d151c7f70dda>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `ne

- 건수 관련 변수 등

In [ ]:
# 건수별평균금액 변수 만들기
# R6M
sales_with_seg['건수별평균금액_할부전환_R6M'] = np.where(
    (sales_with_seg['금액_할부전환_R6M'] == 0) | (sales_with_seg['건수_할부전환_R6M'] == 0),
    0,
    sales_with_seg['금액_할부전환_R6M'] / sales_with_seg['건수_할부전환_R6M'].abs()
)

# R3M
sales_with_seg['건수별평균금액_할부전환_R3M'] = np.where(
    (sales_with_seg['금액_할부전환_R3M'] == 0) | (sales_with_seg['건수_할부전환_R3M'] == 0),
    0,
    sales_with_seg['금액_할부전환_R3M'] / sales_with_seg['건수_할부전환_R3M'].abs()
)

<ipython-input-53-a4a144698f85>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균금액_할부전환_R6M'] = np.where(
<ipython-input-53-a4a144698f85>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균금액_할부전환_R3M'] = np.where(


- 파일 저장: sales_train_cleaned (2400000, 523)

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in sales_with_seg.select_dtypes(include='int64').columns:
    if sales_with_seg[col].max() < 2_147_483_647:
        sales_with_seg[col] = sales_with_seg[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in sales_with_seg.select_dtypes(include='float64').columns:
    sales_with_seg[col] = sales_with_seg[col].astype('float32')

sales_with_seg = sales_with_seg.drop(columns=['Segment'])
sales_with_seg.to_parquet("/content/drive/MyDrive/Dacon/sales_train_cleaned.parquet")

## 청구입금정보 (billing)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = '/content/drive/MyDrive/Data'

# 데이터 분할(폴더) 구분
data_splits = ["train"] # train 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

billing_train_07 is loaded from /content/drive/MyDrive/Data/train/4.청구입금정보/201807_train_청구정보.parquet
billing_train_08 is loaded from /content/drive/MyDrive/Data/train/4.청구입금정보/201808_train_청구정보.parquet
billing_train_09 is loaded from /content/drive/MyDrive/Data/train/4.청구입금정보/201809_train_청구정보.parquet
billing_train_10 is loaded from /content/drive/MyDrive/Data/train/4.청구입금정보/201810_train_청구정보.parquet
billing_train_11 is loaded from /content/drive/MyDrive/Data/train/4.청구입금정보/201811_train_청구정보.parquet
billing_train_12 is loaded from /content/drive/MyDrive/Data/train/4.청구입금정보/201812_train_청구정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["billing"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

billing_train_df  = train_dfs["billing_train_df"]

gc.collect()

billing_train_df is created with shape: (2400000, 46)


0

- 전처리

In [ ]:
# 대표결제일이 10인 경우: D, E만 있다 -> 대표결제일 10 여부 변수 추가
billing_train_df['대표결제일_10여부'] = np.where(billing_train_df['대표결제일'] == 10, 1, 0)

# 대표결제일이 21인 경우: C, D, E만 있다 -> 대표결제일 21 여부 변수 추가
billing_train_df['대표결제일_21여부'] = np.where(billing_train_df['대표결제일'] == 21, 1, 0)

billing_train_df = billing_train_df.drop(['대표결제일'], axis=1) #원래 대표결제일 변수 삭제

# 청구서수령방법이 당사멤버십인 고객: C, D, E만 있다 -> 청구서수령방법 당사멤버십 여부 변수 추가
billing_train_df['청구서수령방법_당사멤버십여부'] = np.where(billing_train_df['청구서수령방법'] == '당사멤버십', 1, 0)
billing_train_df = billing_train_df.drop(['청구서수령방법'], axis=1) # 원래 청구서수령방법 변수 삭제

# 대표결제방법코드: 모두 '자동이체'로 동일 -> 삭제
# 대표청구서수령지구분코드, 대표청구지고객주소구분코드: 청구서수령방법에 대해 세분화한 것 -> 삭제
# 청구서발송여부_B0, 청구서발송여부_R3M, 청구서발송여부_R6M: 0,1로 구성되어있으며 청구금액이 0이면 미발송 -> 청구금액 변수와 겹치므로 삭제
billing_train_df = billing_train_df.drop(['대표결제방법코드', '대표청구서수령지구분코드', '대표청구지고객주소구분코드', '청구서발송여부_B0',
                                          '청구서발송여부_R3M', '청구서발송여부_R6M'], axis=1)

# 문자열 인코딩
billing_train_df['할인건수_R3M'] = billing_train_df['할인건수_R3M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2,'30회 이상': 3, '40회 이상': 4})
billing_train_df['할인건수_B0M'] = billing_train_df['할인건수_B0M'].map({'1회 이상': 0, '10회 이상': 1})

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in billing_train_df.select_dtypes(include='int64').columns:
    if billing_train_df[col].max() < 2_147_483_647:
        billing_train_df[col] = billing_train_df[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in billing_train_df.select_dtypes(include='float64').columns:
    billing_train_df[col] = billing_train_df[col].astype('float32')

billing_train_df.to_parquet("/content/drive/MyDrive/Dacon/billing_train_cleaned.parquet")

## 잔액정보 (balance)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

balance_train_07 is loaded from /content/drive/MyDrive/Data/train/5.잔액정보/201807_train_잔액정보.parquet
balance_train_08 is loaded from /content/drive/MyDrive/Data/train/5.잔액정보/201808_train_잔액정보.parquet
balance_train_09 is loaded from /content/drive/MyDrive/Data/train/5.잔액정보/201809_train_잔액정보.parquet
balance_train_10 is loaded from /content/drive/MyDrive/Data/train/5.잔액정보/201810_train_잔액정보.parquet
balance_train_11 is loaded from /content/drive/MyDrive/Data/train/5.잔액정보/201811_train_잔액정보.parquet
balance_train_12 is loaded from /content/drive/MyDrive/Data/train/5.잔액정보/201812_train_잔액정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["balance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

balance_train_df  = train_dfs["balance_train_df"]
gc.collect()

balance_train_df is created with shape: (2400000, 82)


0

In [ ]:
# 연체일자_B0M: 99.7%가 결측값 -> 연체일수 변수도 있기 때문에 삭제
# 카드론잔액_최종경과월, 최종연체개월수_R15M, RV잔액이월횟수_R6M, RV잔액이월횟수_R3M, 연체잔액_일시불_해외_B0M, 연체잔액_RV일시불_해외_B0M, 연체잔액_할부_해외_B0M, 연체잔액_CA_해외_B0M : 모두 0 → 삭제
balance_train_df = balance_train_df.drop(['연체일자_B0M', '카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M',
                                          'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M',
                                          '연체잔액_CA_해외_B0M'], axis=1)

#최종연체회차: -99, 0으로 구성됨 -> 0,1로 바꿔서 인코딩
balance_train_df['최종연체회차'] = balance_train_df['최종연체회차'].map({-99: 0, 0: 1})

# 연체일수_B1M, 연체일수_B2M, 연체일수_최근: -999999, 0, 1로 구성됨, 연체일수 1은 C,D,E뿐 -> 연체일수 1이면 1, 아니면 0으로 인코딩
balance_train_df['연체일수_B1M'] = np.where(balance_train_df['연체일수_B1M'] == 1, 1, 0)
balance_train_df['연체일수_B2M'] = np.where(balance_train_df['연체일수_B2M'] == 1, 1, 0)
balance_train_df['연체일수_최근'] = np.where(balance_train_df['연체일수_최근'] == 1, 1, 0)

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in balance_train_df.select_dtypes(include='int64').columns:
    if balance_train_df[col].max() < 2_147_483_647:
        balance_train_df[col] = balance_train_df[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in balance_train_df.select_dtypes(include='float64').columns:
    balance_train_df[col] = balance_train_df[col].astype('float32')

balance_train_df.to_parquet("/content/drive/MyDrive/Dacon/balance_train_cleaned.parquet")

## 채널정보 (channel)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

channel_train_07 is loaded from /content/drive/MyDrive/Data/train/6.채널정보/201807_train_채널정보.parquet
channel_train_08 is loaded from /content/drive/MyDrive/Data/train/6.채널정보/201808_train_채널정보.parquet
channel_train_09 is loaded from /content/drive/MyDrive/Data/train/6.채널정보/201809_train_채널정보.parquet
channel_train_10 is loaded from /content/drive/MyDrive/Data/train/6.채널정보/201810_train_채널정보.parquet
channel_train_11 is loaded from /content/drive/MyDrive/Data/train/6.채널정보/201811_train_채널정보.parquet
channel_train_12 is loaded from /content/drive/MyDrive/Data/train/6.채널정보/201812_train_채널정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["channel"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

channel_train_df  = train_dfs["channel_train_df"]

gc.collect()

channel_train_df is created with shape: (2400000, 105)


0

In [ ]:
# 인입횟수~당사PAY 방문월수: 모두 0 -> 삭제
# 인입불만후경과월_IB_R6M: 모두 6 -> 삭제
# OS구분코드: Android, OS로 구성되어 있으나 결측치가 많음 -> 삭제
channel_train_df = channel_train_df.drop(['인입횟수_금융_IB_R6M', '인입불만횟수_IB_R6M', '인입불만일수_IB_R6M', '인입불만월수_IB_R6M',
                                          '인입불만횟수_IB_B0M', '인입불만일수_IB_B0M', 'IB문의건수_한도_B0M', 'IB문의건수_결제_B0M',
                                          'IB문의건수_할부_B0M', 'IB문의건수_정보변경_B0M', 'IB문의건수_결제일변경_B0M', 'IB문의건수_명세서_B0M',
                                          'IB문의건수_비밀번호_B0M', 'IB문의건수_SMS_B0M', 'IB문의건수_APP_B0M', 'IB문의건수_부대서비스_B0M',
                                          'IB문의건수_포인트_B0M', 'IB문의건수_BL_B0M', 'IB문의건수_분실도난_B0M', 'IB문의건수_CA_B0M',
                                          'IB상담건수_VOC_B0M', 'IB상담건수_VOC민원_B0M', 'IB상담건수_VOC불만_B0M', 'IB상담건수_금감원_B0M',
                                          'IB문의건수_명세서_R6M', 'IB문의건수_APP_R6M', 'IB상담건수_VOC_R6M', 'IB상담건수_VOC민원_R6M',
                                          'IB상담건수_VOC불만_R6M', 'IB상담건수_금감원_R6M', '불만제기건수_B0M', '불만제기건수_R12M',
                                          '당사PAY_방문횟수_B0M', '당사PAY_방문횟수_R6M', '당사PAY_방문월수_R6M','인입불만후경과월_IB_R6M',
                                          'OS구분코드'], axis=1)
# 문자열 인코딩
channel_train_df['인입횟수_ARS_R6M'] = channel_train_df['인입횟수_ARS_R6M'].map({'1회 이상': 0, '10회 이상': 1})
channel_train_df['이용메뉴건수_ARS_R6M'] = channel_train_df['이용메뉴건수_ARS_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3})
channel_train_df['방문횟수_PC_R6M'] = channel_train_df['방문횟수_PC_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3, '40회 이상': 4})
channel_train_df['방문일수_PC_R6M'] = channel_train_df['방문일수_PC_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3})
channel_train_df['방문횟수_앱_R6M'] = channel_train_df['방문횟수_앱_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3, '40회 이상': 4,
                                                                     '50회 이상': 5, '60회 이상': 6, '70회 이상': 7, '80회 이상': 8})

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in channel_train_df.select_dtypes(include='int64').columns:
    if channel_train_df[col].max() < 2_147_483_647:
        channel_train_df[col] = channel_train_df[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in channel_train_df.select_dtypes(include='float64').columns:
    channel_train_df[col] = channel_train_df[col].astype('float32')

channel_train_df.to_parquet("/content/drive/MyDrive/Dacon/channel_train_cleaned.parquet")

## 마케팅정보 (marketing)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

marketing_train_07 is loaded from /content/drive/MyDrive/Data/train/7.마케팅정보/201807_train_마케팅정보.parquet
marketing_train_08 is loaded from /content/drive/MyDrive/Data/train/7.마케팅정보/201808_train_마케팅정보.parquet
marketing_train_09 is loaded from /content/drive/MyDrive/Data/train/7.마케팅정보/201809_train_마케팅정보.parquet
marketing_train_10 is loaded from /content/drive/MyDrive/Data/train/7.마케팅정보/201810_train_마케팅정보.parquet
marketing_train_11 is loaded from /content/drive/MyDrive/Data/train/7.마케팅정보/201811_train_마케팅정보.parquet
marketing_train_12 is loaded from /content/drive/MyDrive/Data/train/7.마케팅정보/201812_train_마케팅정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["marketing"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")

marketing_train_df= train_dfs["marketing_train_df"]

gc.collect()

marketing_train_df is created with shape: (2400000, 64)


0

In [ ]:
# 문자열 인코딩
marketing_train_df['캠페인접촉건수_R12M'] = marketing_train_df['캠페인접촉건수_R12M'].map({'1회 이상': 0, '5회 이상': 1,
                                                                             '10회 이상': 2,'15회 이상': 3, '20회 이상': 4, '25회 이상': 5})
marketing_train_df['캠페인접촉일수_R12M'] = marketing_train_df['캠페인접촉일수_R12M'].map({'1일 이상': 0, '5일 이상': 1, '10일 이상': 2,
                                                                             '15일 이상': 3, '20일 이상': 4})
# 모두 0이므로 삭제
marketing_train_df = marketing_train_df.drop(['컨택건수_CA_TM_B0M', '컨택건수_포인트소진_TM_B0M', '컨택건수_CA_EM_B0M', '컨택건수_리볼빙_EM_B0M',
                                              '컨택건수_리볼빙_청구서_B0M', '컨택건수_카드론_인터넷_B0M', '컨택건수_CA_인터넷_B0M', '컨택건수_리볼빙_인터넷_B0M',
                                              '컨택건수_카드론_당사앱_B0M', '컨택건수_CA_당사앱_B0M', '컨택건수_리볼빙_당사앱_B0M', '컨택건수_CA_EM_R6M',
                                              '컨택건수_리볼빙_EM_R6M', '컨택건수_리볼빙_청구서_R6M', '컨택건수_카드론_인터넷_R6M', '컨택건수_CA_인터넷_R6M',
                                              '컨택건수_리볼빙_인터넷_R6M', '컨택건수_카드론_당사앱_R6M', '컨택건수_CA_당사앱_R6M', '컨택건수_리볼빙_당사앱_R6M',
                                              '컨택건수_FDS_B0M', '컨택건수_FDS_R6M'], axis=1)

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in marketing_train_df.select_dtypes(include='int64').columns:
    if marketing_train_df[col].max() < 2_147_483_647:
        marketing_train_df[col] = marketing_train_df[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in marketing_train_df.select_dtypes(include='float64').columns:
    marketing_train_df[col] = marketing_train_df[col].astype('float32')

marketing_train_df.to_parquet("/content/drive/MyDrive/Dacon/marketing_train_cleaned.parquet")

## 성과정보 (performance)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["train"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

performance_train_07 is loaded from /content/drive/MyDrive/Data/train/8.성과정보/201807_train_성과정보.parquet
performance_train_08 is loaded from /content/drive/MyDrive/Data/train/8.성과정보/201808_train_성과정보.parquet
performance_train_09 is loaded from /content/drive/MyDrive/Data/train/8.성과정보/201809_train_성과정보.parquet
performance_train_10 is loaded from /content/drive/MyDrive/Data/train/8.성과정보/201810_train_성과정보.parquet
performance_train_11 is loaded from /content/drive/MyDrive/Data/train/8.성과정보/201811_train_성과정보.parquet
performance_train_12 is loaded from /content/drive/MyDrive/Data/train/8.성과정보/201812_train_성과정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


performance_train_df = train_dfs["performance_train_df"]

gc.collect()

performance_train_df is created with shape: (2400000, 49)


0

In [ ]:
# 결측치를 중앙값으로 대체
performance_train_df["혜택수혜율_B0M"] = performance_train_df["혜택수혜율_B0M"].fillna(performance_train_df["혜택수혜율_B0M"].median())
performance_train_df["혜택수혜율_R3M"] = performance_train_df["혜택수혜율_R3M"].fillna(performance_train_df["혜택수혜율_R3M"].median())

In [ ]:
# int64 -> int32로 변경 (메모리 줄이기 위함)
for col in performance_train_df.select_dtypes(include='int64').columns:
    if performance_train_df[col].max() < 2_147_483_647:
        performance_train_df[col] = performance_train_df[col].astype('int32')

# float64 -> float 32로 변경 (메모리 줄이기 위함)
for col in performance_train_df.select_dtypes(include='float64').columns:
    performance_train_df[col] = performance_train_df[col].astype('float32')

performance_train_df.to_parquet("/content/drive/MyDrive/Dacon/performance_train_cleaned.parquet")

# Data load & Preprocessing (2) - test data

## 회원정보 (customer)

customer_test_df 생성

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"] # test 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            #file_path = f"./{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"

            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"

            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

customer_test_07 is loaded from /content/drive/MyDrive/Data/test/1.회원정보/201807_test_회원정보.parquet
customer_test_08 is loaded from /content/drive/MyDrive/Data/test/1.회원정보/201808_test_회원정보.parquet
customer_test_09 is loaded from /content/drive/MyDrive/Data/test/1.회원정보/201809_test_회원정보.parquet
customer_test_10 is loaded from /content/drive/MyDrive/Data/test/1.회원정보/201810_test_회원정보.parquet
customer_test_11 is loaded from /content/drive/MyDrive/Data/test/1.회원정보/201811_test_회원정보.parquet
customer_test_12 is loaded from /content/drive/MyDrive/Data/test/1.회원정보/201812_test_회원정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["customer"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
test_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")
    # (600000, 77)

customer_test_df = test_dfs["customer_test_df"]

gc.collect()

customer_test_df is created with shape: (600000, 77)


0

전처리

In [ ]:
##### 1. 결측치 처리
# '_1순위신용체크구분', '_2순위신용체크구분’
#    → 결측치 '기타'로 채우기
#    → '_1순위신용체크구분_인코딩', '_2순위신용체크구분_인코딩’
customer_test_df['_1순위신용체크구분'] = customer_test_df['_1순위신용체크구분'].fillna('기타')
customer_test_df['_2순위신용체크구분'] = customer_test_df['_2순위신용체크구분'].fillna('기타')
mapping = {'신용': 1, '체크': 0, '기타': -1}
customer_test_df['1순위신용체크구분_인코딩'] = customer_test_df['_1순위신용체크구분'].map(mapping)
customer_test_df['2순위신용체크구분_인코딩'] = customer_test_df['_2순위신용체크구분'].map(mapping)
customer_test_df.drop(columns=['_1순위신용체크구분','_2순위신용체크구분'], inplace=True)

# 가입통신회사코드 → 가입통신회사_S사여부
customer_test_df['가입통신회사_S사여부'] = (customer_test_df['가입통신회사코드'] == 'S사').astype(int)
customer_test_df = customer_test_df.drop(columns=['가입통신회사코드'])

# 직장시도명 → 직장_수도권여부
customer_test_df['직장_수도권여부'] = customer_test_df['직장시도명'].isin(['서울', '경기']).astype(int)
customer_test_df = customer_test_df.drop(columns=['직장시도명'])

# 결측치 많은 변수 삭제
customer_test_df = customer_test_df.drop(columns=['최종카드발급일자', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용'])


##### 2. 자료형 변환
# 거주시도명 → 거주지_수도권여부
customer_test_df['거주지_수도권여부'] = customer_test_df['거주시도명'].isin(['서울', '경기']).astype(int)
customer_test_df = customer_test_df.drop(columns=['거주시도명'])

# 연회비발생카드수_B0M → 연회비발생카드수_B0M_이진
customer_test_df['연회비발생카드수_B0M_이진'] = customer_test_df['연회비발생카드수_B0M'].isin(['1개이상']).astype(int)
customer_test_df = customer_test_df.drop(columns=['연회비발생카드수_B0M'])

# 데이터 다 동일한 변수 4개 삭제!
columns_drop = ['상품관련면제카드수_B0M','임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']
customer_test_df = customer_test_df.drop(columns=columns_drop)

# Life_Stage: 파생변수(Life_Stage_자녀성장_여부) 생성 후 변수 삭제
customer_test_df['Life_Stage_자녀성장_여부'] = customer_test_df['Life_Stage'].isin(['자녀성장(1)', '자녀성장(2)']).astype(int)
customer_test_df = customer_test_df.drop(columns=['Life_Stage'])

# 연령에서 숫자만 추출 후 int형으로 변환
customer_test_df['연령'] = customer_test_df['연령'].str.extract(r'(\d+)').astype(float).astype('Int64')


##### 3. 상관관계 분석
# 데이터 다 동일한 변수 4개 삭제
columns_drop = ['이용금액_R3M_체크_가족','연회비할인카드수_B0M','할인금액_기본연회비_B0M','할인금액_제휴연회비_B0M']
customer_test_df = customer_test_df.drop(columns=columns_drop)


##### 4. 변수 별로 확인
# 총 4개의 변수 삭제
# '입회일자_신용' 삭제 (입회경과개월수_신용 있음)
# 데이터 다 동일한 변수 '이용카드수_체크_가족' 삭제!
# '청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M’삭제 (의미 같은 변수 있음)
columns_drop = ['입회일자_신용','이용카드수_체크_가족','청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M']
customer_test_df = customer_test_df.drop(columns=columns_drop)

In [ ]:
customer_test_df.to_parquet("/content/drive/MyDrive/Dacon/customer_test_cleaned.parquet")

## 신용정보 (credit)

credit_test_df 생성

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"] # test 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            #file_path = f"./{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"

            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"

            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

credit_test_07 is loaded from /content/drive/MyDrive/Data/test/2.신용정보/201807_test_신용정보.parquet
credit_test_08 is loaded from /content/drive/MyDrive/Data/test/2.신용정보/201808_test_신용정보.parquet
credit_test_09 is loaded from /content/drive/MyDrive/Data/test/2.신용정보/201809_test_신용정보.parquet
credit_test_10 is loaded from /content/drive/MyDrive/Data/test/2.신용정보/201810_test_신용정보.parquet
credit_test_11 is loaded from /content/drive/MyDrive/Data/test/2.신용정보/201811_test_신용정보.parquet
credit_test_12 is loaded from /content/drive/MyDrive/Data/test/2.신용정보/201812_test_신용정보.parquet


18

In [ ]:
# 데이터 유형별 설정
info_categories = ["credit"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
test_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")
    # (600000, 42)

credit_with_seg = test_dfs["credit_test_df"]

gc.collect()

credit_test_df is created with shape: (600000, 42)


0

전처리

In [ ]:
##### 1. 결측치 처리
# 'RV신청일자' 변수 삭제
credit_with_seg = credit_with_seg.drop(columns=['RV신청일자'])

# 'RV전환가능여부' → 'RV전환불가능여부'
credit_with_seg['RV_전환가능여부_이진'] = (credit_with_seg['RV전환가능여부'] == 'N').astype(int)
credit_with_seg = credit_with_seg.drop(columns='RV전환가능여부')



##### 2. 자료형 변환
# 자발한도감액횟수_R12M
# '0회' → 0, '1회' → 1, '2회' → 2
credit_with_seg['자발한도감액횟수_R12M'] = (
    credit_with_seg['자발한도감액횟수_R12M']
    .str.replace('회', '', regex=False)  # '회' 제거
    .astype(int)  # 바로 int로 변환!
)

#‘한도증액횟수_R12M’ → '한도증액_R12M_여부'
# '0회' → 0, '1회이상' → 1
credit_with_seg['한도증액_R12M_여부'] = credit_with_seg['한도증액횟수_R12M'].map({
    '0회': 0,
    '1회이상': 1
    }).astype(int)
credit_with_seg.drop(columns=['한도증액횟수_R12M'], inplace=True)

#‘카드론동의여부’
# 'Y' → 1, 'N' → 0
credit_with_seg['카드론동의여부'] = credit_with_seg['카드론동의여부'].map({
    'Y': 1,
    'N': 0
    }).astype(int)

#‘한도심사요청건수’ → ‘한도심사요청여부’
# '0회' → 0, '1회이상' → 1
credit_with_seg['한도심사요청여부'] = credit_with_seg['한도심사요청건수'].map({
    '0회': 0,
    '1회이상': 1
    }).astype(int)
credit_with_seg.drop(columns=['한도심사요청건수'], inplace=True)



##### 3. 상관관계 분석
# 데이터 모두 동일한 변수 1개 삭제
credit_with_seg = credit_with_seg.drop(columns=['시장연체상환여부_R3M'])



##### 4. 변수 별로 데이터 살펴보기
# 'RV실사용여부' 변수 추가
credit_with_seg['RV실사용여부'] = (credit_with_seg['RV약정청구율'] > 0).astype(int)

# '강제한도감액횟수_2회이상여부' 변수 추가
credit_with_seg['강제한도감액횟수_2회이상여부'] = (credit_with_seg['강제한도감액횟수_R12M'] > 1).astype(int)

# '강제한도감액금액_R12M_3이상여부' 변수 추가
credit_with_seg['강제한도감액금액_R12M_3이상여부'] = (credit_with_seg['강제한도감액금액_R12M'] > 2).astype(int)

# '상향가능CA한도금액_1여부' 변수 추가
credit_with_seg['상향가능CA한도금액_1여부'] = (credit_with_seg['상향가능CA한도금액'] == 1).astype(int)

# '카드이용한도금액_A수준복합' 변수 추가
def classify_dual_limit(row):
    b1 = row['카드이용한도금액_B1M'] >= 170344
    b2 = row['카드이용한도금액_B2M'] >= 170344
    if b1 and b2:
        return 2     # 둘 다 170,344 이상
    elif b1 or b2:
        return 1     # 하나만 170,344 이상
    else:
        return 0    # 둘 다 170,344 미만

credit_with_seg['카드이용한도금액_A수준복합'] = credit_with_seg.apply(classify_dual_limit, axis=1)

In [ ]:
credit_with_seg.to_parquet("/content/drive/MyDrive/Dacon/credit_test_cleaned.parquet")

## 승인매출정보 (sales)

sales_test_df 생성

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"] # test 만!

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            #file_path = f"./{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"

            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"

            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

sales_test_07 is loaded from /content/drive/MyDrive/Data/test/3.승인매출정보/201807_test_승인매출정보.parquet
sales_test_08 is loaded from /content/drive/MyDrive/Data/test/3.승인매출정보/201808_test_승인매출정보.parquet
sales_test_09 is loaded from /content/drive/MyDrive/Data/test/3.승인매출정보/201809_test_승인매출정보.parquet
sales_test_10 is loaded from /content/drive/MyDrive/Data/test/3.승인매출정보/201810_test_승인매출정보.parquet
sales_test_11 is loaded from /content/drive/MyDrive/Data/test/3.승인매출정보/201811_test_승인매출정보.parquet
sales_test_12 is loaded from /content/drive/MyDrive/Data/test/3.승인매출정보/201812_test_승인매출정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["sales"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
test_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")
    # (600000, 406)

sales_with_seg = test_dfs["sales_test_df"]

gc.collect()

sales_test_df is created with shape: (600000, 406)


0

전처리

### 결측치 처리

- '_n순위@@업종' 변수 - AB

In [ ]:
# '_n순위@@업종' 변수(총 15개 변수) 결측치는 '없음'으로 채우기
upjong_cols = [
    '_1순위업종','_2순위업종','_3순위업종',
    '_1순위쇼핑업종','_2순위쇼핑업종','_3순위쇼핑업종',
    '_1순위교통업종','_2순위교통업종','_3순위교통업종',
    '_1순위여유업종','_2순위여유업종','_3순위여유업종',
    '_1순위납부업종','_2순위납부업종','_3순위납부업종'
]

sales_with_seg[upjong_cols] = sales_with_seg[upjong_cols].fillna('없음')

In [ ]:
# '_n순위업종_AB' 변수 추가 (상관계수 -0.3030)

'''
- 1순위 업종이 쇼핑	+1
- 2순위 업종이 사교활동	+1
- 2순위 업종이 교육	+0.5
- 2순위 업종이 의료	+1.5 (A, B에서 더 강하게 나타나므로)
- 3순위 업종이 사교활동	+1
- 3순위 업종이 의료	+1.5 (A, B에서 더 강하게 나타나므로)

- 2순위 업종이 없음 -1
- 3순위 업종이 없음 -1
- 2순위 업종이 교통 -0.5
- 2순위 업종이 납부 -0.5
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위업종'] == '쇼핑':
        score += 1
    if row['_2순위업종'] == '사교활동':
        score += 1
    if row['_2순위업종'] == '교육':
        score += 0.5
    if row['_2순위업종'] == '의료':
        score += 1
    if row['_3순위업종'] == '사교활동':
        score += 1
    if row['_3순위업종'] == '의료':
        score += 1

    # 감점 조건
    if row['_2순위업종'] == '없음':
        score -= 1
    if row['_3순위업종'] == '없음':
        score -= 1
    if row['_2순위업종'] == '교통':
        score -= 0.5
    if row['_2순위업종'] == '납부':
        score -= 0.5

    return score

sales_with_seg['_n순위업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위쇼핑업종_AB' 변수 추가 (상관계수 -0.3304)

'''
- 1순위 쇼핑업종이 온라인 +1
- 2순위 쇼핑업종이 도소매 +1
- 3순위 쇼핑업종이 마트 +1

- 1순위 쇼핑업종이 없음 -1
- 2순위 쇼핑업종이 슈퍼마켓 -0.5
- 2순위 쇼핑업종이 없음 -0.5
- 3순위 쇼핑업종이 없음 -1
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위쇼핑업종'] == '온라인':
        score += 1
    if row['_2순위쇼핑업종'] == '도소매':
        score += 1
    if row['_3순위쇼핑업종'] == '마트':
        score += 1

    # 감점 조건
    if row['_1순위쇼핑업종'] == '없음':
        score -= 1
    if row['_2순위쇼핑업종'] == '슈퍼마켓':
        score -= 0.5
    if row['_2순위쇼핑업종'] == '없음':
        score -= 0.5
    if row['_3순위쇼핑업종'] == '없음':
        score -= 1

    return score

sales_with_seg['_n순위쇼핑업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위교통업종_AB' 변수 추가 (상관계수 -0.34514)

'''
- 1순위 교통업종이 주유 +1
- 2순위 교통업종이 정비 +1
- 3순위 교통업종이 택시 +0.5
- 3순위 교통업종이 정비 +0.5
- 3순위 교통업종이 철도버스 +0.5

- 1순위 교통업종이 없음 -1
- 2순위 교통업종이 없음 -1
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위교통업종'] == '주유':
        score += 1
    if row['_2순위교통업종'] == '정비':
        score += 1
    if row['_3순위교통업종'] == '택시':
        score += 0.5
    if row['_3순위교통업종'] == '정비':
        score += 0.5
    if row['_3순위교통업종'] == '철도버스':
        score += 0.5

    # 감점 조건
    if row['_1순위교통업종'] == '없음':
        score -= 1
    if row['_2순위교통업종'] == '없음':
        score -= 1

    return score

sales_with_seg['_n순위교통업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위여유업종_AB' 변수 추가 (상관계수 -0.26628)

'''
- 1순위 여유업종이 운동 +1
- 1순위 여유업종이 항공 +1
- 2순위 여유업종이 숙박 +1
- 2순위 여유업종이 공연 +1
- 3순위 여유업종이 숙박 +1
- 3순위 여유업종이 공연 +1
- 3순위 여유업종이 여유기타 +1

- 1순위 여유업종이 없음 -0.5
- 2순위 여유업종이 없음 -0.5
- 3순위 여유업종이 없음 -0.5
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위여유업종'] == '운동':
        score += 1
    if row['_1순위여유업종'] == '항공':
        score += 1
    if row['_2순위여유업종'] == '숙박':
        score += 1
    if row['_2순위여유업종'] == '공연':
        score += 1
    if row['_3순위여유업종'] == '숙박':
        score += 1
    if row['_3순위여유업종'] == '공연':
        score += 1
    if row['_3순위여유업종'] == '여유기타':
        score += 1

    # 감점 조건
    if row['_1순위여유업종'] == '없음':
        score -= 0.5
    if row['_2순위여유업종'] == '없음':
        score -= 0.5
    if row['_3순위여유업종'] == '없음':
        score -= 0.5

    return score

sales_with_seg['_n순위여유업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위납부업종_AB' 변수 추가 (상관계수 -0.2579)

'''
- 1순위 납부업종이 납부기타 +0.5
- 1순위 납부업종이 보험료 +0.5
- 1순위 납부업종이 통신비 +0.5
- 2순위 납부업종이 납부기타 +0.5
- 2순위 납부업종이 보험료 +0.5
- 2순위 납부업종이 통신비 +0.5
- 3순위 납부업종이 납부기타 +0.5

- 1순위 납부업종이 없음 -0.5
- 2순위 납부업종이 없음 -0.5
- 3순위 납부업종이 없음 -0.5
'''

def compute_ab_score(row):
    score = 0
    # 가점 조건
    if row['_1순위납부업종'] == '납부기타':
        score += 0.5
    if row['_1순위납부업종'] == '보험료':
        score += 0.5
    if row['_1순위납부업종'] == '통신비':
        score += 0.5
    if row['_2순위납부업종'] == '납부기타':
        score += 0.5
    if row['_2순위납부업종'] == '보험료':
        score += 0.5
    if row['_2순위납부업종'] == '통신비':
        score += 0.5
    if row['_3순위납부업종'] == '납부기타':
        score += 0.5

    # 감점 조건
    if row['_1순위납부업종'] == '없음':
        score -= 0.5
    if row['_2순위납부업종'] == '없음':
        score -= 0.5
    if row['_3순위납부업종'] == '없음':
        score -= 0.5

    return score

sales_with_seg['_n순위납부업종_AB'] = sales_with_seg.apply(compute_ab_score, axis=1)

In [ ]:
# '_n순위@@업종_AB' 5개 변수 합친 변수 '_n순위업종_AB_총합' 추가 (상관계수 -0.3927)
sales_with_seg['_n순위업종_AB_총합'] = (
    sales_with_seg['_n순위업종_AB'] +
    sales_with_seg['_n순위쇼핑업종_AB'] +
    sales_with_seg['_n순위교통업종_AB'] +
    sales_with_seg['_n순위여유업종_AB'] +
    sales_with_seg['_n순위납부업종_AB']
)

In [ ]:
# '_n순위@@업종' 변수 삭제
cols_to_drop = [
    '_1순위업종','_2순위업종','_3순위업종',
    '_1순위쇼핑업종','_2순위쇼핑업종','_3순위쇼핑업종',
    '_1순위교통업종','_2순위교통업종','_3순위교통업종',
    '_1순위여유업종','_2순위여유업종','_3순위여유업종',
    '_1순위납부업종','_2순위납부업종','_3순위납부업종'
]

sales_with_seg.drop(columns=cols_to_drop, inplace=True)

- 결측치 80% 이상인 최종카드론 관련 변수들

In [ ]:
# 변수 삭제
cols_to_drop = ['최종카드론_대출일자','최종카드론_신청경로코드','최종카드론_금융상환방식코드']
sales_with_seg = sales_with_seg.drop(columns=cols_to_drop)

### 자료형 변환

In [ ]:
# '이용금액대_점수' 변수 생성 (상관계수 -0.6026)
amount_mapping = {
    '01.100만원+': 18,
    '02.50만원+': 5,
    '03.30만원+': 2,
    '04.10만원+': 2,
    '05.10만원-': 1,
    '09.미사용': 1
}

sales_with_seg['이용금액대_점수'] = sales_with_seg['이용금액대'].map(amount_mapping)

In [ ]:
# 원래 변수 '이용금액대' 삭제
sales_with_seg = sales_with_seg.drop(columns=['이용금액대'])

### 상관관계 분석

In [ ]:
# 데이터 값이 모두 같은 변수 2개 제거
sales_with_seg = sales_with_seg.drop(columns=['승인거절건수_입력오류_B0M','승인거절건수_기타_B0M'])

### 변수 별로 데이터 살펴보기

In [ ]:
# '이용여부_@@' 변수 생성
sales_with_seg['이용여부_기본']   = (sales_with_seg['최종이용일자_기본'] != 10101).astype(int)
sales_with_seg['이용여부_신판']   = (sales_with_seg['최종이용일자_신판'] != 10101).astype(int)
sales_with_seg['이용여부_CA']    = (sales_with_seg['최종이용일자_CA'] != 10101).astype(int)
sales_with_seg['이용여부_카드론'] = (sales_with_seg['최종이용일자_카드론'] != 10101).astype(int)
sales_with_seg['이용여부_체크']   = (sales_with_seg['최종이용일자_체크'] != 10101).astype(int)
sales_with_seg['이용여부_일시불'] = (sales_with_seg['최종이용일자_일시불'] != 10101).astype(int)
sales_with_seg['이용여부_할부']   = (sales_with_seg['최종이용일자_할부'] != 10101).astype(int)

# 원래 '최종이용일자_@@' 변수 삭제
for i in ['최종이용일자_기본','최종이용일자_신판','최종이용일자_CA','최종이용일자_카드론','최종이용일자_체크','최종이용일자_일시불','최종이용일자_할부']:
  sales_with_seg = sales_with_seg.drop(columns=[i])

- BOM 변수

In [ ]:
##### 이용건수 #####

# 데이터 값이 모두 같은 변수 제거
sales_with_seg = sales_with_seg.drop(columns=['이용건수_부분무이자_B0M'])

# '이용건수_신용_B0M' 변수 값 보정
sales_with_seg['이용건수_신용_B0M'] = sales_with_seg['이용건수_신용_B0M'] + sales_with_seg['이용건수_카드론_B0M']



##### 이용금액 #####

# 데이터 값이 모두 같은 변수 제거
sales_with_seg = sales_with_seg.drop(columns=['이용금액_부분무이자_B0M'])

# '이용금액_신용_B0M', '이용금액_신판_B0M' 변수 생성
sales_with_seg['이용금액_신용_B0M'] = sales_with_seg['이용금액_일시불_B0M'] + sales_with_seg['이용금액_할부_B0M'] + sales_with_seg['이용금액_CA_B0M'] + sales_with_seg['이용금액_카드론_B0M']
sales_with_seg['이용금액_신판_B0M'] = sales_with_seg['이용금액_일시불_B0M'] + sales_with_seg['이용금액_할부_B0M']

In [ ]:
# 건수별평균이용금액 변수 생성
sales_with_seg['건수별평균이용금액_신용_B0M'] = np.where(
    (sales_with_seg['이용건수_신용_B0M'] == 0) | (sales_with_seg['이용금액_신용_B0M'] == 0), 0,
    sales_with_seg['이용금액_신용_B0M'] / sales_with_seg['이용건수_신용_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_B0M'] = np.where(
    (sales_with_seg['이용건수_신판_B0M'] == 0) | (sales_with_seg['이용금액_신판_B0M'] == 0), 0,
    sales_with_seg['이용금액_신판_B0M'] / sales_with_seg['이용건수_신판_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_B0M'] = np.where(
    (sales_with_seg['이용건수_일시불_B0M'] == 0) | (sales_with_seg['이용금액_일시불_B0M'] == 0), 0,
    sales_with_seg['이용금액_일시불_B0M'] / sales_with_seg['이용건수_일시불_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_B0M'] = np.where(
    (sales_with_seg['이용건수_할부_B0M'] == 0) | (sales_with_seg['이용금액_할부_B0M'] == 0), 0,
    sales_with_seg['이용금액_할부_B0M'] / sales_with_seg['이용건수_할부_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_B0M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_B0M'] == 0) | (sales_with_seg['이용금액_할부_유이자_B0M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_B0M'] / sales_with_seg['이용건수_할부_유이자_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_B0M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_B0M'] == 0) | (sales_with_seg['이용금액_할부_무이자_B0M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_B0M'] / sales_with_seg['이용건수_할부_무이자_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_B0M'] = np.where(
    (sales_with_seg['이용건수_CA_B0M'] == 0) | (sales_with_seg['이용금액_CA_B0M'] == 0), 0,
    sales_with_seg['이용금액_CA_B0M'] / sales_with_seg['이용건수_CA_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_B0M'] = np.where(
    (sales_with_seg['이용건수_체크_B0M'] == 0) | (sales_with_seg['이용금액_체크_B0M'] == 0), 0,
    sales_with_seg['이용금액_체크_B0M'] / sales_with_seg['이용건수_체크_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_B0M'] = np.where(
    (sales_with_seg['이용건수_카드론_B0M'] == 0) | (sales_with_seg['이용금액_카드론_B0M'] == 0), 0,
    sales_with_seg['이용금액_카드론_B0M'] / sales_with_seg['이용건수_카드론_B0M'].abs()
)

In [ ]:
# 최근3개월이용여부 변수 생성
sales_with_seg['최근3개월이용여부_신용'] = (sales_with_seg['이용후경과월_신용'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_신판'] = (sales_with_seg['이용후경과월_신판'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_일시불'] = (sales_with_seg['이용후경과월_일시불'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_할부'] = (sales_with_seg['이용후경과월_할부'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_할부_유이자'] = (sales_with_seg['이용후경과월_할부_유이자'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_할부_무이자'] = (sales_with_seg['이용후경과월_할부_무이자'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_CA'] = (sales_with_seg['이용후경과월_CA'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_체크'] = (sales_with_seg['이용후경과월_체크'] <= 3).astype(int)
sales_with_seg['최근3개월이용여부_카드론'] = (sales_with_seg['이용후경과월_카드론'] <= 3).astype(int)

- R12M 변수

In [ ]:
##### 이용건수 #####

# '이용건수_신용_R12M' 변수 값 보정
sales_with_seg['이용건수_신용_R12M'] = sales_with_seg['이용건수_신용_R12M'] + sales_with_seg['이용건수_카드론_R12M']


##### 이용금액 #####

# '이용금액_신용_R12M', '이용금액_신판_R12M' 변수 생성
sales_with_seg['이용금액_신용_R12M'] = sales_with_seg['이용금액_일시불_R12M'] + sales_with_seg['이용금액_할부_R12M'] + sales_with_seg['이용금액_CA_R12M'] + sales_with_seg['이용금액_카드론_R12M']
sales_with_seg['이용금액_신판_R12M'] = sales_with_seg['이용금액_일시불_R12M'] + sales_with_seg['이용금액_할부_R12M']


In [ ]:
# 건수별평균이용금액 변수 생성 (R12M 기준)
sales_with_seg['건수별평균이용금액_신용_R12M'] = np.where(
    (sales_with_seg['이용건수_신용_R12M'] == 0) | (sales_with_seg['이용금액_신용_R12M'] == 0), 0,
    sales_with_seg['이용금액_신용_R12M'] / sales_with_seg['이용건수_신용_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_R12M'] = np.where(
    (sales_with_seg['이용건수_신판_R12M'] == 0) | (sales_with_seg['이용금액_신판_R12M'] == 0), 0,
    sales_with_seg['이용금액_신판_R12M'] / sales_with_seg['이용건수_신판_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_R12M'] = np.where(
    (sales_with_seg['이용건수_일시불_R12M'] == 0) | (sales_with_seg['이용금액_일시불_R12M'] == 0), 0,
    sales_with_seg['이용금액_일시불_R12M'] / sales_with_seg['이용건수_일시불_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_R12M'] = np.where(
    (sales_with_seg['이용건수_할부_R12M'] == 0) | (sales_with_seg['이용금액_할부_R12M'] == 0), 0,
    sales_with_seg['이용금액_할부_R12M'] / sales_with_seg['이용건수_할부_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_R12M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_R12M'] == 0) | (sales_with_seg['이용금액_할부_유이자_R12M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_R12M'] / sales_with_seg['이용건수_할부_유이자_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_R12M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_R12M'] == 0) | (sales_with_seg['이용금액_할부_무이자_R12M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_R12M'] / sales_with_seg['이용건수_할부_무이자_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_R12M'] = np.where(
    (sales_with_seg['이용건수_CA_R12M'] == 0) | (sales_with_seg['이용금액_CA_R12M'] == 0), 0,
    sales_with_seg['이용금액_CA_R12M'] / sales_with_seg['이용건수_CA_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_R12M'] = np.where(
    (sales_with_seg['이용건수_체크_R12M'] == 0) | (sales_with_seg['이용금액_체크_R12M'] == 0), 0,
    sales_with_seg['이용금액_체크_R12M'] / sales_with_seg['이용건수_체크_R12M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_R12M'] = np.where(
    (sales_with_seg['이용건수_카드론_R12M'] == 0) | (sales_with_seg['이용금액_카드론_R12M'] == 0), 0,
    sales_with_seg['이용금액_카드론_R12M'] / sales_with_seg['이용건수_카드론_R12M'].abs()
)

In [ ]:
# 이용월평균이용금액 변수 생성 (R12M 기)
sales_with_seg['이용월평균이용금액_신용_R12M'] = np.where(
    sales_with_seg['이용개월수_신용_R12M'] == 0, 0,
    sales_with_seg['이용금액_신용_R12M'] / sales_with_seg['이용개월수_신용_R12M']
)

sales_with_seg['이용월평균이용금액_신판_R12M'] = np.where(
    sales_with_seg['이용개월수_신판_R12M'] == 0, 0,
    sales_with_seg['이용금액_신판_R12M'] / sales_with_seg['이용개월수_신판_R12M']
)

sales_with_seg['이용월평균이용금액_일시불_R12M'] = np.where(
    sales_with_seg['이용개월수_일시불_R12M'] == 0, 0,
    sales_with_seg['이용금액_일시불_R12M'] / sales_with_seg['이용개월수_일시불_R12M']
)

sales_with_seg['이용월평균이용금액_할부_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_R12M'] == 0, 0,
    sales_with_seg['이용금액_할부_R12M'] / sales_with_seg['이용개월수_할부_R12M']
)

sales_with_seg['이용월평균이용금액_할부_유이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R12M'] == 0, 0,
    sales_with_seg['이용금액_할부_유이자_R12M'] / sales_with_seg['이용개월수_할부_유이자_R12M']
)

sales_with_seg['이용월평균이용금액_할부_무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R12M'] == 0, 0,
    sales_with_seg['이용금액_할부_무이자_R12M'] / sales_with_seg['이용개월수_할부_무이자_R12M']
)

sales_with_seg['이용월평균이용금액_부분무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R12M'] == 0, 0,
    sales_with_seg['이용금액_부분무이자_R12M'] / sales_with_seg['이용개월수_부분무이자_R12M']
)

sales_with_seg['이용월평균이용금액_CA_R12M'] = np.where(
    sales_with_seg['이용개월수_CA_R12M'] == 0, 0,
    sales_with_seg['이용금액_CA_R12M'] / sales_with_seg['이용개월수_CA_R12M']
)

sales_with_seg['이용월평균이용금액_체크_R12M'] = np.where(
    sales_with_seg['이용개월수_체크_R12M'] == 0, 0,
    sales_with_seg['이용금액_체크_R12M'] / sales_with_seg['이용개월수_체크_R12M']
)

sales_with_seg['이용월평균이용금액_카드론_R12M'] = np.where(
    sales_with_seg['이용개월수_카드론_R12M'] == 0, 0,
    sales_with_seg['이용금액_카드론_R12M'] / sales_with_seg['이용개월수_카드론_R12M']
)

In [ ]:
# 이용월평균이용건수 변수 생성 (R12M 기준)
sales_with_seg['이용월평균이용건수_신용_R12M'] = np.where(
    sales_with_seg['이용개월수_신용_R12M'] == 0, 0,
    sales_with_seg['이용건수_신용_R12M'] / sales_with_seg['이용개월수_신용_R12M']
)

sales_with_seg['이용월평균이용건수_신판_R12M'] = np.where(
    sales_with_seg['이용개월수_신판_R12M'] == 0, 0,
    sales_with_seg['이용건수_신판_R12M'] / sales_with_seg['이용개월수_신판_R12M']
)

sales_with_seg['이용월평균이용건수_일시불_R12M'] = np.where(
    sales_with_seg['이용개월수_일시불_R12M'] == 0, 0,
    sales_with_seg['이용건수_일시불_R12M'] / sales_with_seg['이용개월수_일시불_R12M']
)

sales_with_seg['이용월평균이용건수_할부_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_R12M'] == 0, 0,
    sales_with_seg['이용건수_할부_R12M'] / sales_with_seg['이용개월수_할부_R12M']
)

sales_with_seg['이용월평균이용건수_할부_유이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R12M'] == 0, 0,
    sales_with_seg['이용건수_할부_유이자_R12M'] / sales_with_seg['이용개월수_할부_유이자_R12M']
)

sales_with_seg['이용월평균이용건수_할부_무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R12M'] == 0, 0,
    sales_with_seg['이용건수_할부_무이자_R12M'] / sales_with_seg['이용개월수_할부_무이자_R12M']
)

sales_with_seg['이용월평균이용건수_부분무이자_R12M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R12M'] == 0, 0,
    sales_with_seg['이용건수_부분무이자_R12M'] / sales_with_seg['이용개월수_부분무이자_R12M']
)

sales_with_seg['이용월평균이용건수_CA_R12M'] = np.where(
    sales_with_seg['이용개월수_CA_R12M'] == 0, 0,
    sales_with_seg['이용건수_CA_R12M'] / sales_with_seg['이용개월수_CA_R12M']
)

sales_with_seg['이용월평균이용건수_체크_R12M'] = np.where(
    sales_with_seg['이용개월수_체크_R12M'] == 0, 0,
    sales_with_seg['이용건수_체크_R12M'] / sales_with_seg['이용개월수_체크_R12M']
)

sales_with_seg['이용월평균이용건수_카드론_R12M'] = np.where(
    sales_with_seg['이용개월수_카드론_R12M'] == 0, 0,
    sales_with_seg['이용건수_카드론_R12M'] / sales_with_seg['이용개월수_카드론_R12M']
)

- R6M 변수

In [ ]:
##### 이용건수 #####

# '이용건수_신용_R6M' 변수 값 보정
sales_with_seg['이용건수_신용_R6M'] = sales_with_seg['이용건수_신용_R6M'] + sales_with_seg['이용건수_카드론_R6M']


##### 이용금액 #####

# '이용금액_신용_R6M', '이용금액_신판_R6M' 변수 생성
sales_with_seg['이용금액_신용_R6M'] = sales_with_seg['이용금액_일시불_R6M'] + sales_with_seg['이용금액_할부_R6M'] + sales_with_seg['이용금액_CA_R6M'] + sales_with_seg['이용금액_카드론_R6M']
sales_with_seg['이용금액_신판_R6M'] = sales_with_seg['이용금액_일시불_R6M'] + sales_with_seg['이용금액_할부_R6M']

In [ ]:
# 건수별평균이용금액 변수 생성 (R6M 기준)
sales_with_seg['건수별평균이용금액_신용_R6M'] = np.where(
    (sales_with_seg['이용건수_신용_R6M'] == 0) | (sales_with_seg['이용금액_신용_R6M'] == 0), 0,
    sales_with_seg['이용금액_신용_R6M'] / sales_with_seg['이용건수_신용_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_R6M'] = np.where(
    (sales_with_seg['이용건수_신판_R6M'] == 0) | (sales_with_seg['이용금액_신판_R6M'] == 0), 0,
    sales_with_seg['이용금액_신판_R6M'] / sales_with_seg['이용건수_신판_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_R6M'] = np.where(
    (sales_with_seg['이용건수_일시불_R6M'] == 0) | (sales_with_seg['이용금액_일시불_R6M'] == 0), 0,
    sales_with_seg['이용금액_일시불_R6M'] / sales_with_seg['이용건수_일시불_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_R6M'] = np.where(
    (sales_with_seg['이용건수_할부_R6M'] == 0) | (sales_with_seg['이용금액_할부_R6M'] == 0), 0,
    sales_with_seg['이용금액_할부_R6M'] / sales_with_seg['이용건수_할부_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_R6M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_R6M'] == 0) | (sales_with_seg['이용금액_할부_유이자_R6M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_R6M'] / sales_with_seg['이용건수_할부_유이자_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_R6M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_R6M'] == 0) | (sales_with_seg['이용금액_할부_무이자_R6M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_R6M'] / sales_with_seg['이용건수_할부_무이자_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_부분무이자_R6M'] = np.where(
    (sales_with_seg['이용건수_부분무이자_R6M'] == 0) | (sales_with_seg['이용금액_부분무이자_R6M'] == 0), 0,
    sales_with_seg['이용금액_부분무이자_R6M'] / sales_with_seg['이용건수_부분무이자_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_R6M'] = np.where(
    (sales_with_seg['이용건수_CA_R6M'] == 0) | (sales_with_seg['이용금액_CA_R6M'] == 0), 0,
    sales_with_seg['이용금액_CA_R6M'] / sales_with_seg['이용건수_CA_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_R6M'] = np.where(
    (sales_with_seg['이용건수_체크_R6M'] == 0) | (sales_with_seg['이용금액_체크_R6M'] == 0), 0,
    sales_with_seg['이용금액_체크_R6M'] / sales_with_seg['이용건수_체크_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_R6M'] = np.where(
    (sales_with_seg['이용건수_카드론_R6M'] == 0) | (sales_with_seg['이용금액_카드론_R6M'] == 0), 0,
    sales_with_seg['이용금액_카드론_R6M'] / sales_with_seg['이용건수_카드론_R6M'].abs()
)

<ipython-input-106-f2f64e88dced>:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_체크_R6M'] = np.where(
<ipython-input-106-f2f64e88dced>:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_카드론_R6M'] = np.where(


In [ ]:
# 이용월평균이용금액 변수 생성 (R6M 기준)
sales_with_seg['이용월평균이용금액_신용_R6M'] = np.where(
    sales_with_seg['이용개월수_신용_R6M'] == 0, 0,
    sales_with_seg['이용금액_신용_R6M'] / sales_with_seg['이용개월수_신용_R6M']
)

sales_with_seg['이용월평균이용금액_신판_R6M'] = np.where(
    sales_with_seg['이용개월수_신판_R6M'] == 0, 0,
    sales_with_seg['이용금액_신판_R6M'] / sales_with_seg['이용개월수_신판_R6M']
)

sales_with_seg['이용월평균이용금액_일시불_R6M'] = np.where(
    sales_with_seg['이용개월수_일시불_R6M'] == 0, 0,
    sales_with_seg['이용금액_일시불_R6M'] / sales_with_seg['이용개월수_일시불_R6M']
)

sales_with_seg['이용월평균이용금액_할부_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_R6M'] == 0, 0,
    sales_with_seg['이용금액_할부_R6M'] / sales_with_seg['이용개월수_할부_R6M']
)

sales_with_seg['이용월평균이용금액_할부_유이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R6M'] == 0, 0,
    sales_with_seg['이용금액_할부_유이자_R6M'] / sales_with_seg['이용개월수_할부_유이자_R6M']
)

sales_with_seg['이용월평균이용금액_할부_무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R6M'] == 0, 0,
    sales_with_seg['이용금액_할부_무이자_R6M'] / sales_with_seg['이용개월수_할부_무이자_R6M']
)

sales_with_seg['이용월평균이용금액_부분무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R6M'] == 0, 0,
    sales_with_seg['이용금액_부분무이자_R6M'] / sales_with_seg['이용개월수_부분무이자_R6M']
)

sales_with_seg['이용월평균이용금액_CA_R6M'] = np.where(
    sales_with_seg['이용개월수_CA_R6M'] == 0, 0,
    sales_with_seg['이용금액_CA_R6M'] / sales_with_seg['이용개월수_CA_R6M']
)

sales_with_seg['이용월평균이용금액_체크_R6M'] = np.where(
    sales_with_seg['이용개월수_체크_R6M'] == 0, 0,
    sales_with_seg['이용금액_체크_R6M'] / sales_with_seg['이용개월수_체크_R6M']
)

sales_with_seg['이용월평균이용금액_카드론_R6M'] = np.where(
    sales_with_seg['이용개월수_카드론_R6M'] == 0, 0,
    sales_with_seg['이용금액_카드론_R6M'] / sales_with_seg['이용개월수_카드론_R6M']
)

<ipython-input-107-aaa73e740e06>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용금액_신용_R6M'] = np.where(
<ipython-input-107-aaa73e740e06>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용금액_신판_R6M'] = np.where(
<ipython-input-107-aaa73e740e06>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

In [ ]:
# 이용월평균이용건수 변수 생성 (R6M 기준)
sales_with_seg['이용월평균이용건수_신용_R6M'] = np.where(
    sales_with_seg['이용개월수_신용_R6M'] == 0, 0,
    sales_with_seg['이용건수_신용_R6M'] / sales_with_seg['이용개월수_신용_R6M']
)

sales_with_seg['이용월평균이용건수_신판_R6M'] = np.where(
    sales_with_seg['이용개월수_신판_R6M'] == 0, 0,
    sales_with_seg['이용건수_신판_R6M'] / sales_with_seg['이용개월수_신판_R6M']
)

sales_with_seg['이용월평균이용건수_일시불_R6M'] = np.where(
    sales_with_seg['이용개월수_일시불_R6M'] == 0, 0,
    sales_with_seg['이용건수_일시불_R6M'] / sales_with_seg['이용개월수_일시불_R6M']
)

sales_with_seg['이용월평균이용건수_할부_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_R6M'] == 0, 0,
    sales_with_seg['이용건수_할부_R6M'] / sales_with_seg['이용개월수_할부_R6M']
)

sales_with_seg['이용월평균이용건수_할부_유이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R6M'] == 0, 0,
    sales_with_seg['이용건수_할부_유이자_R6M'] / sales_with_seg['이용개월수_할부_유이자_R6M']
)

sales_with_seg['이용월평균이용건수_할부_무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R6M'] == 0, 0,
    sales_with_seg['이용건수_할부_무이자_R6M'] / sales_with_seg['이용개월수_할부_무이자_R6M']
)

sales_with_seg['이용월평균이용건수_부분무이자_R6M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R6M'] == 0, 0,
    sales_with_seg['이용건수_부분무이자_R6M'] / sales_with_seg['이용개월수_부분무이자_R6M']
)

sales_with_seg['이용월평균이용건수_CA_R6M'] = np.where(
    sales_with_seg['이용개월수_CA_R6M'] == 0, 0,
    sales_with_seg['이용건수_CA_R6M'] / sales_with_seg['이용개월수_CA_R6M']
)

sales_with_seg['이용월평균이용건수_체크_R6M'] = np.where(
    sales_with_seg['이용개월수_체크_R6M'] == 0, 0,
    sales_with_seg['이용건수_체크_R6M'] / sales_with_seg['이용개월수_체크_R6M']
)

sales_with_seg['이용월평균이용건수_카드론_R6M'] = np.where(
    sales_with_seg['이용개월수_카드론_R6M'] == 0, 0,
    sales_with_seg['이용건수_카드론_R6M'] / sales_with_seg['이용개월수_카드론_R6M']
)

<ipython-input-108-14d78e973320>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_신용_R6M'] = np.where(
<ipython-input-108-14d78e973320>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_신판_R6M'] = np.where(
<ipython-input-108-14d78e973320>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

- R3M 변수

In [ ]:
##### 이용건수 #####

# '이용건수_신용_R3M' 변수 값 보정
sales_with_seg['이용건수_신용_R3M'] = sales_with_seg['이용건수_신용_R3M'] + sales_with_seg['이용건수_카드론_R3M']


##### 이용금액 #####

# '이용금액_신용_R3M', '이용금액_신판_R3M' 변수 생성
sales_with_seg['이용금액_신용_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M'] + sales_with_seg['이용금액_CA_R3M'] + sales_with_seg['이용금액_카드론_R3M']
sales_with_seg['이용금액_신판_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M']


<ipython-input-109-4d9390fc2cd0>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용금액_신용_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M'] + sales_with_seg['이용금액_CA_R3M'] + sales_with_seg['이용금액_카드론_R3M']
<ipython-input-109-4d9390fc2cd0>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용금액_신판_R3M'] = sales_with_seg['이용금액_일시불_R3M'] + sales_with_seg['이용금액_할부_R3M']


In [ ]:
# 건수별평균이용금액 변수 생성 (R3M 기준)
sales_with_seg['건수별평균이용금액_신용_R3M'] = np.where(
    (sales_with_seg['이용건수_신용_R3M'] == 0) | (sales_with_seg['이용금액_신용_R3M'] == 0), 0,
    sales_with_seg['이용금액_신용_R3M'] / sales_with_seg['이용건수_신용_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_신판_R3M'] = np.where(
    (sales_with_seg['이용건수_신판_R3M'] == 0) | (sales_with_seg['이용금액_신판_R3M'] == 0), 0,
    sales_with_seg['이용금액_신판_R3M'] / sales_with_seg['이용건수_신판_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_일시불_R3M'] = np.where(
    (sales_with_seg['이용건수_일시불_R3M'] == 0) | (sales_with_seg['이용금액_일시불_R3M'] == 0), 0,
    sales_with_seg['이용금액_일시불_R3M'] / sales_with_seg['이용건수_일시불_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_R3M'] = np.where(
    (sales_with_seg['이용건수_할부_R3M'] == 0) | (sales_with_seg['이용금액_할부_R3M'] == 0), 0,
    sales_with_seg['이용금액_할부_R3M'] / sales_with_seg['이용건수_할부_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_유이자_R3M'] = np.where(
    (sales_with_seg['이용건수_할부_유이자_R3M'] == 0) | (sales_with_seg['이용금액_할부_유이자_R3M'] == 0), 0,
    sales_with_seg['이용금액_할부_유이자_R3M'] / sales_with_seg['이용건수_할부_유이자_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_할부_무이자_R3M'] = np.where(
    (sales_with_seg['이용건수_할부_무이자_R3M'] == 0) | (sales_with_seg['이용금액_할부_무이자_R3M'] == 0), 0,
    sales_with_seg['이용금액_할부_무이자_R3M'] / sales_with_seg['이용건수_할부_무이자_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_부분무이자_R3M'] = np.where(
    (sales_with_seg['이용건수_부분무이자_R3M'] == 0) | (sales_with_seg['이용금액_부분무이자_R3M'] == 0), 0,
    sales_with_seg['이용금액_부분무이자_R3M'] / sales_with_seg['이용건수_부분무이자_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_CA_R3M'] = np.where(
    (sales_with_seg['이용건수_CA_R3M'] == 0) | (sales_with_seg['이용금액_CA_R3M'] == 0), 0,
    sales_with_seg['이용금액_CA_R3M'] / sales_with_seg['이용건수_CA_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_체크_R3M'] = np.where(
    (sales_with_seg['이용건수_체크_R3M'] == 0) | (sales_with_seg['이용금액_체크_R3M'] == 0), 0,
    sales_with_seg['이용금액_체크_R3M'] / sales_with_seg['이용건수_체크_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_카드론_R3M'] = np.where(
    (sales_with_seg['이용건수_카드론_R3M'] == 0) | (sales_with_seg['이용금액_카드론_R3M'] == 0), 0,
    sales_with_seg['이용금액_카드론_R3M'] / sales_with_seg['이용건수_카드론_R3M'].abs()
)

<ipython-input-110-8a8fe6265eaa>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_신용_R3M'] = np.where(
<ipython-input-110-8a8fe6265eaa>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_신판_R3M'] = np.where(
<ipython-input-110-8a8fe6265eaa>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

In [ ]:
# 이용월평균이용금액 변수 생성 (R3M 기준)
sales_with_seg['이용월평균이용금액_신용_R3M'] = np.where(
    sales_with_seg['이용개월수_신용_R3M'] == 0, 0,
    sales_with_seg['이용금액_신용_R3M'] / sales_with_seg['이용개월수_신용_R3M']
)

sales_with_seg['이용월평균이용금액_신판_R3M'] = np.where(
    sales_with_seg['이용개월수_신판_R3M'] == 0, 0,
    sales_with_seg['이용금액_신판_R3M'] / sales_with_seg['이용개월수_신판_R3M']
)

sales_with_seg['이용월평균이용금액_일시불_R3M'] = np.where(
    sales_with_seg['이용개월수_일시불_R3M'] == 0, 0,
    sales_with_seg['이용금액_일시불_R3M'] / sales_with_seg['이용개월수_일시불_R3M']
)

sales_with_seg['이용월평균이용금액_할부_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_R3M'] == 0, 0,
    sales_with_seg['이용금액_할부_R3M'] / sales_with_seg['이용개월수_할부_R3M']
)

sales_with_seg['이용월평균이용금액_할부_유이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R3M'] == 0, 0,
    sales_with_seg['이용금액_할부_유이자_R3M'] / sales_with_seg['이용개월수_할부_유이자_R3M']
)

sales_with_seg['이용월평균이용금액_할부_무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R3M'] == 0, 0,
    sales_with_seg['이용금액_할부_무이자_R3M'] / sales_with_seg['이용개월수_할부_무이자_R3M']
)

sales_with_seg['이용월평균이용금액_부분무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R3M'] == 0, 0,
    sales_with_seg['이용금액_부분무이자_R3M'] / sales_with_seg['이용개월수_부분무이자_R3M']
)

sales_with_seg['이용월평균이용금액_CA_R3M'] = np.where(
    sales_with_seg['이용개월수_CA_R3M'] == 0, 0,
    sales_with_seg['이용금액_CA_R3M'] / sales_with_seg['이용개월수_CA_R3M']
)

sales_with_seg['이용월평균이용금액_체크_R3M'] = np.where(
    sales_with_seg['이용개월수_체크_R3M'] == 0, 0,
    sales_with_seg['이용금액_체크_R3M'] / sales_with_seg['이용개월수_체크_R3M']
)

sales_with_seg['이용월평균이용금액_카드론_R3M'] = np.where(
    sales_with_seg['이용개월수_카드론_R3M'] == 0, 0,
    sales_with_seg['이용금액_카드론_R3M'] / sales_with_seg['이용개월수_카드론_R3M']
)

<ipython-input-111-c32a9b21fcfa>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용금액_신용_R3M'] = np.where(
<ipython-input-111-c32a9b21fcfa>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용금액_신판_R3M'] = np.where(
<ipython-input-111-c32a9b21fcfa>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

In [ ]:
# 이용월평균이용건수 변수 생성 (R3M 기준)
sales_with_seg['이용월평균이용건수_신용_R3M'] = np.where(
    sales_with_seg['이용개월수_신용_R3M'] == 0, 0,
    sales_with_seg['이용건수_신용_R3M'] / sales_with_seg['이용개월수_신용_R3M']
)

sales_with_seg['이용월평균이용건수_신판_R3M'] = np.where(
    sales_with_seg['이용개월수_신판_R3M'] == 0, 0,
    sales_with_seg['이용건수_신판_R3M'] / sales_with_seg['이용개월수_신판_R3M']
)

sales_with_seg['이용월평균이용건수_일시불_R3M'] = np.where(
    sales_with_seg['이용개월수_일시불_R3M'] == 0, 0,
    sales_with_seg['이용건수_일시불_R3M'] / sales_with_seg['이용개월수_일시불_R3M']
)

sales_with_seg['이용월평균이용건수_할부_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_R3M'] == 0, 0,
    sales_with_seg['이용건수_할부_R3M'] / sales_with_seg['이용개월수_할부_R3M']
)

sales_with_seg['이용월평균이용건수_할부_유이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_유이자_R3M'] == 0, 0,
    sales_with_seg['이용건수_할부_유이자_R3M'] / sales_with_seg['이용개월수_할부_유이자_R3M']
)

sales_with_seg['이용월평균이용건수_할부_무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_할부_무이자_R3M'] == 0, 0,
    sales_with_seg['이용건수_할부_무이자_R3M'] / sales_with_seg['이용개월수_할부_무이자_R3M']
)

sales_with_seg['이용월평균이용건수_부분무이자_R3M'] = np.where(
    sales_with_seg['이용개월수_부분무이자_R3M'] == 0, 0,
    sales_with_seg['이용건수_부분무이자_R3M'] / sales_with_seg['이용개월수_부분무이자_R3M']
)

sales_with_seg['이용월평균이용건수_CA_R3M'] = np.where(
    sales_with_seg['이용개월수_CA_R3M'] == 0, 0,
    sales_with_seg['이용건수_CA_R3M'] / sales_with_seg['이용개월수_CA_R3M']
)

sales_with_seg['이용월평균이용건수_체크_R3M'] = np.where(
    sales_with_seg['이용개월수_체크_R3M'] == 0, 0,
    sales_with_seg['이용건수_체크_R3M'] / sales_with_seg['이용개월수_체크_R3M']
)

sales_with_seg['이용월평균이용건수_카드론_R3M'] = np.where(
    sales_with_seg['이용개월수_카드론_R3M'] == 0, 0,
    sales_with_seg['이용건수_카드론_R3M'] / sales_with_seg['이용개월수_카드론_R3M']
)


<ipython-input-112-cb1d1abb0035>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_신용_R3M'] = np.where(
<ipython-input-112-cb1d1abb0035>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['이용월평균이용건수_신판_R3M'] = np.where(
<ipython-input-112-cb1d1abb0035>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

- 쇼핑/교통/여유/납부

In [ ]:
# '쇼핑_총이용금액' 변수 생성
shopping_columns = ['쇼핑_도소매_이용금액',
                    '쇼핑_백화점_이용금액',
                    '쇼핑_마트_이용금액',
                    '쇼핑_슈퍼마켓_이용금액',
                    '쇼핑_편의점_이용금액',
                    '쇼핑_아울렛_이용금액',
                    '쇼핑_온라인_이용금액',
                    '쇼핑_기타_이용금액']
sales_with_seg['쇼핑_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

# '교통_총이용금액' 변수 생성
shopping_columns = ['교통_주유이용금액',
                    '교통_정비이용금액',
                    '교통_통행료이용금액',
                    '교통_버스지하철이용금액',
                    '교통_택시이용금액',
                    '교통_철도버스이용금액']
sales_with_seg['교통_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

# '여유_총이용금액' 변수 생성
shopping_columns = ['여유_운동이용금액',
                    '여유_Pet이용금액',
                    '여유_공연이용금액',
                    '여유_공원이용금액',
                    '여유_숙박이용금액',
                    '여유_항공이용금액',
                    '여유_기타이용금액']
sales_with_seg['여유_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

# '납부_총이용금액' 변수 생성
shopping_columns = ['납부_통신비이용금액',
                    '납부_관리비이용금액',
                    '납부_가스전기료이용금액',
                    '납부_보험료이용금액',
                    '납부_기타이용금액']
sales_with_seg['납부_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

<ipython-input-113-406ff2616aa4>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['쇼핑_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)
<ipython-input-113-406ff2616aa4>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['교통_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)
<ipython-input-113-406ff2616aa4>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [ ]:
# 상관계수 절댓값 0.3 넘는 변수 5개만 합치기
shopping_columns = ['쇼핑_도소매_이용금액',
                    '쇼핑_마트_이용금액',
                    '쇼핑_온라인_이용금액',
                    '교통_주유이용금액',
                    '납부_기타이용금액']
sales_with_seg['0.3이상_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)

<ipython-input-114-b7ebe65e3d8d>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['0.3이상_총이용금액'] = sales_with_seg[shopping_columns].sum(axis=1)


In [ ]:
# 쇼핑/교통/여유/납부 다 합치기
columns = ['쇼핑_총이용금액',
          '교통_총이용금액',
          '여유_총이용금액',
          '납부_총이용금액']
sales_with_seg['총이용금액_쇼핑교통여유납부'] = sales_with_seg[columns].sum(axis=1)

<ipython-input-115-22bf84db1bde>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['총이용금액_쇼핑교통여유납부'] = sales_with_seg[columns].sum(axis=1)


In [ ]:
# 기존 변수 다 삭제
columns_to_drop = [
    '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액',
    '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액',
    '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액',

    '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액',
    '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액',

    '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액',
    '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액',
    '여유_항공이용금액', '여유_기타이용금액',

    '납부_통신비이용금액', '납부_관리비이용금액', '납부_렌탈료이용금액',
    '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_유선방송이용금액',
    '납부_건강연금이용금액', '납부_기타이용금액'
]

# 변수 삭제
sales_with_seg.drop(columns=columns_to_drop, inplace=True)

-  R12M 할부건수/할부금액

In [ ]:
# 데이터 동일한 변수 4개 삭제
columns_drop = [
    '할부건수_부분_3M_R12M',
    '할부건수_부분_6M_R12M',
    '할부건수_부분_14M_R12M',
    '할부금액_부분_3M_R12M'
    ]
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 최근 1년치 총합 변수 7개 생성 (할부건수/할부금액)

# 1. 할부
sales_with_seg['할부건수총합_R12M'] = (
    sales_with_seg['할부건수_3M_R12M'] +
    sales_with_seg['할부건수_6M_R12M'] +
    sales_with_seg['할부건수_12M_R12M']
)

sales_with_seg['할부금액총합_R12M'] = (
    sales_with_seg['할부금액_3M_R12M'] +
    sales_with_seg['할부금액_6M_R12M'] +
    sales_with_seg['할부금액_12M_R12M']
)

# 2. 유이자 할부
sales_with_seg['할부건수총합_유이자_R12M'] = (
    sales_with_seg['할부건수_유이자_3M_R12M'] +
    sales_with_seg['할부건수_유이자_6M_R12M'] +
    sales_with_seg['할부건수_유이자_12M_R12M']
)

sales_with_seg['할부금액총합_유이자_R12M'] = (
    sales_with_seg['할부금액_유이자_3M_R12M'] +
    sales_with_seg['할부금액_유이자_6M_R12M'] +
    sales_with_seg['할부금액_유이자_12M_R12M']
)

# 3. 무이자 할부
sales_with_seg['할부건수총합_무이자_R12M'] = (
    sales_with_seg['할부건수_무이자_3M_R12M'] +
    sales_with_seg['할부건수_무이자_6M_R12M'] +
    sales_with_seg['할부건수_무이자_12M_R12M']
)

sales_with_seg['할부금액총합_무이자_R12M'] = (
    sales_with_seg['할부금액_무이자_3M_R12M'] +
    sales_with_seg['할부금액_무이자_6M_R12M'] +
    sales_with_seg['할부금액_무이자_12M_R12M']
)

# 3. 부분 할부
# '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M'가 다 0이라서 '할부건수_부분_12M_R12M'가 1년치 총합과 동일함
sales_with_seg['할부금액총합_부분_R12M'] = (
    sales_with_seg['할부금액_부분_6M_R12M'] +
    sales_with_seg['할부금액_부분_12M_R12M']
)

<ipython-input-118-a57ce0275030>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['할부건수총합_R12M'] = (
<ipython-input-118-a57ce0275030>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['할부금액총합_R12M'] = (
<ipython-input-118-a57ce0275030>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

In [ ]:
# 할부 건수 별 금액 변수 4개 생성
#  - 3M 기준이 제일 상관계수가 좋음
#  - 부분 무이자 할부는 할부건수가 12M 밖에 없어서 12M 기준으로 계산

# 1. 할부
sales_with_seg['건수별할부금액_3M'] = np.where(
    (sales_with_seg['할부금액_3M_R12M'] == 0) | (sales_with_seg['할부건수_3M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_3M_R12M'] / sales_with_seg['할부건수_3M_R12M']
)

# 2. 유이자 할부
sales_with_seg['건수별할부금액_유이자_3M'] = np.where(
    (sales_with_seg['할부금액_유이자_3M_R12M'] == 0) | (sales_with_seg['할부건수_유이자_3M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_유이자_3M_R12M'] / sales_with_seg['할부건수_유이자_3M_R12M']
)

# 3. 무이자 할부
sales_with_seg['건수별할부금액_무이자_3M'] = np.where(
    (sales_with_seg['할부금액_무이자_3M_R12M'] == 0) | (sales_with_seg['할부건수_무이자_3M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_무이자_3M_R12M'] / sales_with_seg['할부건수_무이자_3M_R12M']
)

# 4. 부분 무이자 할부
sales_with_seg['건수별할부금액_부분_12M'] = np.where(
    (sales_with_seg['할부금액_부분_12M_R12M'] == 0) | (sales_with_seg['할부건수_부분_12M_R12M'] == 0),
    0,
    sales_with_seg['할부금액_부분_12M_R12M'] / sales_with_seg['할부건수_부분_12M_R12M']
)

<ipython-input-119-fbf2bfb45a75>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별할부금액_3M'] = np.where(
<ipython-input-119-fbf2bfb45a75>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별할부금액_유이자_3M'] = np.where(
<ipython-input-119-fbf2bfb45a75>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame,

- RP 관련 변수들

In [ ]:
# 데이터 동일한 변수 6개 삭제
columns_drop = ['RP건수_유선방송_B0M',
                'RP건수_건강_B0M',
                'RP후경과월_유선방송',
                'RP후경과월_건강',
                '증감_RP건수_유선방송_전월',
                '증감_RP건수_건강_전월']
sales_with_seg.drop(columns=columns_drop, inplace=True)

# '최근3개월이용여부_RP' 변수 생성
sales_with_seg['최근3개월이용여부_RP'] = (sales_with_seg['RP후경과월'] <= 3).astype(int)

# 'RP_이용확대여부' 변수 생성
delta_cols = [col for col in sales_with_seg.columns if col.startswith('증감_RP건수_')]
sales_with_seg['RP건수_증감총합'] = sales_with_seg[delta_cols].sum(axis=1)
sales_with_seg['RP_이용확대여부'] = (sales_with_seg['RP건수_증감총합'] > 0).astype(int) # 방향성: 양수면 확장, 음수면 축소
sales_with_seg.drop(columns=['RP건수_증감총합'], inplace=True)

<ipython-input-120-071d93968994>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['최근3개월이용여부_RP'] = (sales_with_seg['RP후경과월'] <= 3).astype(int)
<ipython-input-120-071d93968994>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['RP건수_증감총합'] = sales_with_seg[delta_cols].sum(axis=1)
<ipython-input-120-071d93968994>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [ ]:
# 'RP활성업종수' 변수 생성
rp_cols = [
    'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M',
    'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M',
    'RP건수_학습비_B0M', 'RP건수_교통_B0M'
]
sales_with_seg['RP활성업종수'] = sales_with_seg[rp_cols].gt(0).sum(axis=1)

<ipython-input-121-a8a35245fb88>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['RP활성업종수'] = sales_with_seg[rp_cols].gt(0).sum(axis=1)


- 카드론 관련 변수들

In [ ]:
# '카드론이용여부' 변수 생성
sales_with_seg['카드론이용여부'] = (
    (sales_with_seg['최초카드론이용경과월'] != 999) &
    (sales_with_seg['최종카드론이용경과월'] != 999)
).astype(int)

<ipython-input-122-781516af5b95>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['카드론이용여부'] = (


- 이용개월수/이용금액/이용건수-  온라인/오프라인

In [ ]:
# 건수별평균금액 변수 6개 생성

# R6M(최근 6개월)
sales_with_seg['건수별평균이용금액_온라인_R6M'] = np.where(
    (sales_with_seg['이용금액_온라인_R6M'] == 0) | (sales_with_seg['이용건수_온라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_온라인_R6M'] / sales_with_seg['이용건수_온라인_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_오프라인_R6M'] = np.where(
    (sales_with_seg['이용금액_오프라인_R6M'] == 0) | (sales_with_seg['이용건수_오프라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_오프라인_R6M'] / sales_with_seg['이용건수_오프라인_R6M'].abs()
)

# R3M(최근 3개월)
sales_with_seg['건수별평균이용금액_온라인_R3M'] = np.where(
    (sales_with_seg['이용금액_온라인_R3M'] == 0) | (sales_with_seg['이용건수_온라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_온라인_R3M'] / sales_with_seg['이용건수_온라인_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_오프라인_R3M'] = np.where(
    (sales_with_seg['이용금액_오프라인_R3M'] == 0) | (sales_with_seg['이용건수_오프라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_오프라인_R3M'] / sales_with_seg['이용건수_오프라인_R3M'].abs()
)


# B0M(당월)
sales_with_seg['건수별평균이용금액_온라인_B0M'] = np.where(
    (sales_with_seg['이용금액_온라인_B0M'] == 0) | (sales_with_seg['이용건수_온라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_온라인_B0M'] / sales_with_seg['이용건수_온라인_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_오프라인_B0M'] = np.where(
    (sales_with_seg['이용금액_오프라인_B0M'] == 0) | (sales_with_seg['이용건수_오프라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_오프라인_B0M'] / sales_with_seg['이용건수_오프라인_B0M'].abs()
)

<ipython-input-123-9fcf63bd8b3f>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_온라인_R6M'] = np.where(
<ipython-input-123-9fcf63bd8b3f>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_오프라인_R6M'] = np.where(
<ipython-input-123-9fcf63bd8b3f>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

- 이용개월수/이용금액/이용건수 - 페이_온라인/오프라인

In [ ]:
# 건수별평균이용금액 변수 생성

# R6M (최근 6개월)
sales_with_seg['건수별평균이용금액_페이_온라인_R6M'] = np.where(
    (sales_with_seg['이용금액_페이_온라인_R6M'] == 0) | (sales_with_seg['이용건수_페이_온라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_페이_온라인_R6M'] / sales_with_seg['이용건수_페이_온라인_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_페이_오프라인_R6M'] = np.where(
    (sales_with_seg['이용금액_페이_오프라인_R6M'] == 0) | (sales_with_seg['이용건수_페이_오프라인_R6M'] == 0),
    0,
    sales_with_seg['이용금액_페이_오프라인_R6M'] / sales_with_seg['이용건수_페이_오프라인_R6M'].abs()
)

# R3M (최근 3개월)
sales_with_seg['건수별평균이용금액_페이_온라인_R3M'] = np.where(
    (sales_with_seg['이용금액_페이_온라인_R3M'] == 0) | (sales_with_seg['이용건수_페이_온라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_페이_온라인_R3M'] / sales_with_seg['이용건수_페이_온라인_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_페이_오프라인_R3M'] = np.where(
    (sales_with_seg['이용금액_페이_오프라인_R3M'] == 0) | (sales_with_seg['이용건수_페이_오프라인_R3M'] == 0),
    0,
    sales_with_seg['이용금액_페이_오프라인_R3M'] / sales_with_seg['이용건수_페이_오프라인_R3M'].abs()
)

# B0M (당월)
sales_with_seg['건수별평균이용금액_페이_온라인_B0M'] = np.where(
    (sales_with_seg['이용금액_페이_온라인_B0M'] == 0) | (sales_with_seg['이용건수_페이_온라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_페이_온라인_B0M'] / sales_with_seg['이용건수_페이_온라인_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_페이_오프라인_B0M'] = np.where(
    (sales_with_seg['이용금액_페이_오프라인_B0M'] == 0) | (sales_with_seg['이용건수_페이_오프라인_B0M'] == 0),
    0,
    sales_with_seg['이용금액_페이_오프라인_B0M'] / sales_with_seg['이용건수_페이_오프라인_B0M'].abs()
)

<ipython-input-124-1b4362d6e054>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_페이_온라인_R6M'] = np.where(
<ipython-input-124-1b4362d6e054>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_페이_오프라인_R6M'] = np.where(
<ipython-input-124-1b4362d6e054>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

- 이용개월수/이용금액/이용건수 - 간편결제/당사페이/당사기타/A,B,C,D페이

- R6M

In [ ]:
# 데이터 동일한 변수 5개 삭제
columns_drop = ['이용개월수_당사페이_R6M',
                '이용금액_당사페이_R6M',
                '이용금액_당사기타_R6M',
                '이용건수_당사페이_R6M',
                '이용건수_당사기타_R6M']
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 건수별평균이용금액 변수 5개 생성 (R6M 기준)

# 간편결제
sales_with_seg['건수별평균이용금액_간편결제_R6M'] = np.where(
    (sales_with_seg['이용금액_간편결제_R6M'] == 0) | (sales_with_seg['이용건수_간편결제_R6M'] == 0),
    0,
    sales_with_seg['이용금액_간편결제_R6M'] / sales_with_seg['이용건수_간편결제_R6M'].abs()
)

# A페이
sales_with_seg['건수별평균이용금액_A페이_R6M'] = np.where(
    (sales_with_seg['이용금액_A페이_R6M'] == 0) | (sales_with_seg['이용건수_A페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_A페이_R6M'] / sales_with_seg['이용건수_A페이_R6M'].abs()
)

# B페이
sales_with_seg['건수별평균이용금액_B페이_R6M'] = np.where(
    (sales_with_seg['이용금액_B페이_R6M'] == 0) | (sales_with_seg['이용건수_B페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_B페이_R6M'] / sales_with_seg['이용건수_B페이_R6M'].abs()
)

# C페이
sales_with_seg['건수별평균이용금액_C페이_R6M'] = np.where(
    (sales_with_seg['이용금액_C페이_R6M'] == 0) | (sales_with_seg['이용건수_C페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_C페이_R6M'] / sales_with_seg['이용건수_C페이_R6M'].abs()
)

# D페이
sales_with_seg['건수별평균이용금액_D페이_R6M'] = np.where(
    (sales_with_seg['이용금액_D페이_R6M'] == 0) | (sales_with_seg['이용건수_D페이_R6M'] == 0),
    0,
    sales_with_seg['이용금액_D페이_R6M'] / sales_with_seg['이용건수_D페이_R6M'].abs()
)

<ipython-input-126-154095dbc84a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_간편결제_R6M'] = np.where(
<ipython-input-126-154095dbc84a>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_A페이_R6M'] = np.where(
<ipython-input-126-154095dbc84a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

- R3M

In [ ]:
# 데이터 동일한 변수 4개 삭제
columns_drop = ['이용금액_당사페이_R3M',
                '이용금액_당사기타_R3M',
                '이용건수_당사페이_R3M',
                '이용건수_당사기타_R3M']
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 건수별평균이용금액 변수 5개 생성 (R3M 기준)

# 간편결제
sales_with_seg['건수별평균이용금액_간편결제_R3M'] = np.where(
    (sales_with_seg['이용금액_간편결제_R3M'] == 0) | (sales_with_seg['이용건수_간편결제_R3M'] == 0),
    0,
    sales_with_seg['이용금액_간편결제_R3M'] / sales_with_seg['이용건수_간편결제_R3M'].abs()
)

# A페이
sales_with_seg['건수별평균이용금액_A페이_R3M'] = np.where(
    (sales_with_seg['이용금액_A페이_R3M'] == 0) | (sales_with_seg['이용건수_A페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_A페이_R3M'] / sales_with_seg['이용건수_A페이_R3M'].abs()
)

# B페이
sales_with_seg['건수별평균이용금액_B페이_R3M'] = np.where(
    (sales_with_seg['이용금액_B페이_R3M'] == 0) | (sales_with_seg['이용건수_B페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_B페이_R3M'] / sales_with_seg['이용건수_B페이_R3M'].abs()
)

# C페이
sales_with_seg['건수별평균이용금액_C페이_R3M'] = np.where(
    (sales_with_seg['이용금액_C페이_R3M'] == 0) | (sales_with_seg['이용건수_C페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_C페이_R3M'] / sales_with_seg['이용건수_C페이_R3M'].abs()
)

# D페이
sales_with_seg['건수별평균이용금액_D페이_R3M'] = np.where(
    (sales_with_seg['이용금액_D페이_R3M'] == 0) | (sales_with_seg['이용건수_D페이_R3M'] == 0),
    0,
    sales_with_seg['이용금액_D페이_R3M'] / sales_with_seg['이용건수_D페이_R3M'].abs()
)

<ipython-input-128-c3a97c739420>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_간편결제_R3M'] = np.where(
<ipython-input-128-c3a97c739420>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_A페이_R3M'] = np.where(
<ipython-input-128-c3a97c739420>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

- B0M

In [ ]:
# 데이터 동일한 변수 4개 삭제
columns_drop = ['이용금액_당사페이_B0M',
                '이용금액_당사기타_B0M',
                '이용건수_당사페이_B0M',
                '이용건수_당사기타_B0M']
sales_with_seg.drop(columns=columns_drop, inplace=True)

In [ ]:
# 건수별평균이용금액 변수 5개 생성 (B0M 기준)

# 간편결제
sales_with_seg['건수별평균이용금액_간편결제_B0M'] = np.where(
    (sales_with_seg['이용금액_간편결제_B0M'] == 0) | (sales_with_seg['이용건수_간편결제_B0M'] == 0),
    0,
    sales_with_seg['이용금액_간편결제_B0M'] / sales_with_seg['이용건수_간편결제_B0M'].abs()
)

# A페이
sales_with_seg['건수별평균이용금액_A페이_B0M'] = np.where(
    (sales_with_seg['이용금액_A페이_B0M'] == 0) | (sales_with_seg['이용건수_A페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_A페이_B0M'] / sales_with_seg['이용건수_A페이_B0M'].abs()
)

# B페이
sales_with_seg['건수별평균이용금액_B페이_B0M'] = np.where(
    (sales_with_seg['이용금액_B페이_B0M'] == 0) | (sales_with_seg['이용건수_B페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_B페이_B0M'] / sales_with_seg['이용건수_B페이_B0M'].abs()
)

# C페이
sales_with_seg['건수별평균이용금액_C페이_B0M'] = np.where(
    (sales_with_seg['이용금액_C페이_B0M'] == 0) | (sales_with_seg['이용건수_C페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_C페이_B0M'] / sales_with_seg['이용건수_C페이_B0M'].abs()
)

# D페이
sales_with_seg['건수별평균이용금액_D페이_B0M'] = np.where(
    (sales_with_seg['이용금액_D페이_B0M'] == 0) | (sales_with_seg['이용건수_D페이_B0M'] == 0),
    0,
    sales_with_seg['이용금액_D페이_B0M'] / sales_with_seg['이용건수_D페이_B0M'].abs()
)

<ipython-input-130-51b69cc8c592>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_간편결제_B0M'] = np.where(
<ipython-input-130-51b69cc8c592>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_A페이_B0M'] = np.where(
<ipython-input-130-51b69cc8c592>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

- 선결제

In [ ]:
# 횟수별/건수별 평균이용금액

# R6M
sales_with_seg['횟수별평균이용금액_선결제_R6M'] = np.where(
    (sales_with_seg['이용금액_선결제_R6M'] == 0) | (sales_with_seg['이용횟수_선결제_R6M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R6M'] / sales_with_seg['이용횟수_선결제_R6M'].abs()
)

sales_with_seg['건수별평균이용금액_선결제_R6M'] = np.where(
    (sales_with_seg['이용금액_선결제_R6M'] == 0) | (sales_with_seg['이용건수_선결제_R6M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R6M'] / sales_with_seg['이용건수_선결제_R6M'].abs()
)

# R3M
sales_with_seg['횟수별평균이용금액_선결제_R3M'] = np.where(
    (sales_with_seg['이용금액_선결제_R3M'] == 0) | (sales_with_seg['이용횟수_선결제_R3M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R3M'] / sales_with_seg['이용횟수_선결제_R3M'].abs()
)

sales_with_seg['건수별평균이용금액_선결제_R3M'] = np.where(
    (sales_with_seg['이용금액_선결제_R3M'] == 0) | (sales_with_seg['이용건수_선결제_R3M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_R3M'] / sales_with_seg['이용건수_선결제_R3M'].abs()
)

# B0M
sales_with_seg['횟수별평균이용금액_선결제_B0M'] = np.where(
    (sales_with_seg['이용금액_선결제_B0M'] == 0) | (sales_with_seg['이용횟수_선결제_B0M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_B0M'] / sales_with_seg['이용횟수_선결제_B0M'].abs()
)

sales_with_seg['건수별평균이용금액_선결제_B0M'] = np.where(
    (sales_with_seg['이용금액_선결제_B0M'] == 0) | (sales_with_seg['이용건수_선결제_B0M'] == 0),
    0,
    sales_with_seg['이용금액_선결제_B0M'] / sales_with_seg['이용건수_선결제_B0M'].abs()
)

<ipython-input-131-f74a7dbb1d59>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['횟수별평균이용금액_선결제_R6M'] = np.where(
<ipython-input-131-f74a7dbb1d59>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균이용금액_선결제_R6M'] = np.where(
<ipython-input-131-f74a7dbb1d59>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmen

- 연체

In [ ]:
# 횟수별평균이용금액 변수 생성

# R6M
sales_with_seg['횟수별평균이용금액_연체_R6M'] = np.where(
    (sales_with_seg['이용금액_연체_R6M'] == 0) | (sales_with_seg['이용횟수_연체_R6M'] == 0),
    0,
    sales_with_seg['이용금액_연체_R6M'] / sales_with_seg['이용횟수_연체_R6M'].abs()
)

# R3M
sales_with_seg['횟수별평균이용금액_연체_R3M'] = np.where(
    (sales_with_seg['이용금액_연체_R3M'] == 0) | (sales_with_seg['이용횟수_연체_R3M'] == 0),
    0,
    sales_with_seg['이용금액_연체_R3M'] / sales_with_seg['이용횟수_연체_R3M'].abs()
)

# B0M
sales_with_seg['횟수별평균이용금액_연체_B0M'] = np.where(
    (sales_with_seg['이용금액_연체_B0M'] == 0) | (sales_with_seg['이용횟수_연체_B0M'] == 0),
    0,
    sales_with_seg['이용금액_연체_B0M'] / sales_with_seg['이용횟수_연체_B0M'].abs()
)

<ipython-input-132-d7558ba4bb37>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['횟수별평균이용금액_연체_R6M'] = np.where(
<ipython-input-132-d7558ba4bb37>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['횟수별평균이용금액_연체_R3M'] = np.where(
<ipython-input-132-d7558ba4bb37>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmente

- 입금원금 관련 변수

In [ ]:
# 총납부금 변수 및 납부비율 변수 생성
for month in ['B0M', 'B2M', 'B5M']:
    sales_with_seg[f'총납부금_{month}'] = (
        sales_with_seg[f'선입금원금_{month}'] +
        sales_with_seg[f'정상입금원금_{month}'] +
        sales_with_seg[f'연체입금원금_{month}']
    )

    sales_with_seg[f'납부비율_{month}'] = np.where(
        sales_with_seg[f'정상청구원금_{month}'] == 0,
        0,
        sales_with_seg[f'총납부금_{month}'] / sales_with_seg[f'정상청구원금_{month}']
    )

<ipython-input-133-a792eb1a16b8>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'총납부금_{month}'] = (
<ipython-input-133-a792eb1a16b8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'납부비율_{month}'] = np.where(
<ipython-input-133-a792eb1a16b8>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `ne

In [ ]:
# 정시납부금 및 정시납부비율 변수 생성
for month in ['B0M', 'B2M', 'B5M']:
    sales_with_seg[f'정시납부금_{month}'] = (
        sales_with_seg[f'선입금원금_{month}'] +
        sales_with_seg[f'정상입금원금_{month}']
    )

    sales_with_seg[f'정시납부비율_{month}'] = np.where(
        sales_with_seg[f'정상청구원금_{month}'] == 0,
        0,
        sales_with_seg[f'정시납부금_{month}'] / sales_with_seg[f'정상청구원금_{month}']
    )

<ipython-input-134-d151c7f70dda>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'정시납부금_{month}'] = (
<ipython-input-134-d151c7f70dda>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg[f'정시납부비율_{month}'] = np.where(
<ipython-input-134-d151c7f70dda>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

- 건수 관련 변수 등

In [ ]:
# 건수별평균금액 변수 만들기
# R6M
sales_with_seg['건수별평균금액_할부전환_R6M'] = np.where(
    (sales_with_seg['금액_할부전환_R6M'] == 0) | (sales_with_seg['건수_할부전환_R6M'] == 0),
    0,
    sales_with_seg['금액_할부전환_R6M'] / sales_with_seg['건수_할부전환_R6M'].abs()
)

# R3M
sales_with_seg['건수별평균금액_할부전환_R3M'] = np.where(
    (sales_with_seg['금액_할부전환_R3M'] == 0) | (sales_with_seg['건수_할부전환_R3M'] == 0),
    0,
    sales_with_seg['금액_할부전환_R3M'] / sales_with_seg['건수_할부전환_R3M'].abs()
)

<ipython-input-135-a4a144698f85>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균금액_할부전환_R6M'] = np.where(
<ipython-input-135-a4a144698f85>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_with_seg['건수별평균금액_할부전환_R3M'] = np.where(


- 파일 저장: sales_test_cleaned (600000, 523)

In [ ]:
sales_with_seg.to_parquet("/content/drive/MyDrive/Dacon/sales_test_cleaned.parquet")

## 청구입금정보 (billing)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

billing_test_07 is loaded from /content/drive/MyDrive/Data/test/4.청구입금정보/201807_test_청구정보.parquet
billing_test_08 is loaded from /content/drive/MyDrive/Data/test/4.청구입금정보/201808_test_청구정보.parquet
billing_test_09 is loaded from /content/drive/MyDrive/Data/test/4.청구입금정보/201809_test_청구정보.parquet
billing_test_10 is loaded from /content/drive/MyDrive/Data/test/4.청구입금정보/201810_test_청구정보.parquet
billing_test_11 is loaded from /content/drive/MyDrive/Data/test/4.청구입금정보/201811_test_청구정보.parquet
billing_test_12 is loaded from /content/drive/MyDrive/Data/test/4.청구입금정보/201812_test_청구정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["billing"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")

billing_test_df  = test_dfs["billing_test_df"]

gc.collect()

billing_test_df is created with shape: (600000, 46)


0

In [ ]:
# test 데이터에 대해 train과 동일한 방법 적용
billing_test_df['대표결제일_10여부'] = np.where(billing_test_df['대표결제일'] == 10, 1, 0)
billing_test_df['대표결제일_21여부'] = np.where(billing_test_df['대표결제일'] == 21, 1, 0)
billing_test_df = billing_test_df.drop(['대표결제일'], axis=1)
billing_test_df['청구서수령방법_당사멤버십여부'] = np.where(billing_test_df['청구서수령방법'] == '당사멤버십', 1, 0)
billing_test_df = billing_test_df.drop(['청구서수령방법'], axis=1)
billing_test_df = billing_test_df.drop(['대표결제방법코드', '대표청구서수령지구분코드', '대표청구지고객주소구분코드', '청구서발송여부_B0',
                                        '청구서발송여부_R3M', '청구서발송여부_R6M'], axis=1)
billing_test_df['할인건수_R3M'] = billing_test_df['할인건수_R3M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2,'30회 이상': 3, '40회 이상': 4})
billing_test_df['할인건수_B0M'] = billing_test_df['할인건수_B0M'].map({'1회 이상': 0, '10회 이상': 1})

In [ ]:
billing_test_df.to_parquet("/content/drive/MyDrive/Dacon/billing_test_cleaned.parquet")

## 잔액정보 (balance)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"}
    }

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

balance_test_07 is loaded from /content/drive/MyDrive/Data/test/5.잔액정보/201807_test_잔액정보.parquet
balance_test_08 is loaded from /content/drive/MyDrive/Data/test/5.잔액정보/201808_test_잔액정보.parquet
balance_test_09 is loaded from /content/drive/MyDrive/Data/test/5.잔액정보/201809_test_잔액정보.parquet
balance_test_10 is loaded from /content/drive/MyDrive/Data/test/5.잔액정보/201810_test_잔액정보.parquet
balance_test_11 is loaded from /content/drive/MyDrive/Data/test/5.잔액정보/201811_test_잔액정보.parquet
balance_test_12 is loaded from /content/drive/MyDrive/Data/test/5.잔액정보/201812_test_잔액정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["balance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")

balance_test_df  = test_dfs["balance_test_df"]

gc.collect()

balance_test_df is created with shape: (600000, 82)


0

In [ ]:
# test 데이터에 대해 train과 동일한 방법 적용
balance_test_df = balance_test_df.drop(['연체일자_B0M', '카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M',
                                        'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M',
                                        '연체잔액_CA_해외_B0M'], axis=1)
balance_test_df['최종연체회차'] = balance_test_df['최종연체회차'].map({-99: 0, 0: 1})
balance_test_df['연체일수_B1M'] = np.where(balance_test_df['연체일수_B1M'] == 1, 1, 0)
balance_test_df['연체일수_B2M'] = np.where(balance_test_df['연체일수_B2M'] == 1, 1, 0)
balance_test_df['연체일수_최근'] = np.where(balance_test_df['연체일수_최근'] == 1, 1, 0)

In [ ]:
balance_test_df.to_parquet("/content/drive/MyDrive/Dacon/balance_test_cleaned.parquet")

## 채널정보 (channel)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

channel_test_07 is loaded from /content/drive/MyDrive/Data/test/6.채널정보/201807_test_채널정보.parquet
channel_test_08 is loaded from /content/drive/MyDrive/Data/test/6.채널정보/201808_test_채널정보.parquet
channel_test_09 is loaded from /content/drive/MyDrive/Data/test/6.채널정보/201809_test_채널정보.parquet
channel_test_10 is loaded from /content/drive/MyDrive/Data/test/6.채널정보/201810_test_채널정보.parquet
channel_test_11 is loaded from /content/drive/MyDrive/Data/test/6.채널정보/201811_test_채널정보.parquet
channel_test_12 is loaded from /content/drive/MyDrive/Data/test/6.채널정보/201812_test_채널정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["channel"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")

channel_test_df  = test_dfs["channel_test_df"]

gc.collect()

channel_test_df is created with shape: (600000, 105)


0

In [ ]:
# test 데이터에 대해 train과 동일한 방법 적용
channel_test_df = channel_test_df.drop(['인입횟수_금융_IB_R6M', '인입불만횟수_IB_R6M', '인입불만일수_IB_R6M', '인입불만월수_IB_R6M',
                                        '인입불만횟수_IB_B0M', '인입불만일수_IB_B0M', 'IB문의건수_한도_B0M', 'IB문의건수_결제_B0M',
                                        'IB문의건수_할부_B0M', 'IB문의건수_정보변경_B0M', 'IB문의건수_결제일변경_B0M', 'IB문의건수_명세서_B0M',
                                        'IB문의건수_비밀번호_B0M', 'IB문의건수_SMS_B0M', 'IB문의건수_APP_B0M', 'IB문의건수_부대서비스_B0M',
                                        'IB문의건수_포인트_B0M', 'IB문의건수_BL_B0M', 'IB문의건수_분실도난_B0M', 'IB문의건수_CA_B0M',
                                        'IB상담건수_VOC_B0M', 'IB상담건수_VOC민원_B0M', 'IB상담건수_VOC불만_B0M', 'IB상담건수_금감원_B0M',
                                        'IB문의건수_명세서_R6M', 'IB문의건수_APP_R6M', 'IB상담건수_VOC_R6M', 'IB상담건수_VOC민원_R6M',
                                        'IB상담건수_VOC불만_R6M', 'IB상담건수_금감원_R6M', '불만제기건수_B0M', '불만제기건수_R12M',
                                        '당사PAY_방문횟수_B0M', '당사PAY_방문횟수_R6M', '당사PAY_방문월수_R6M','인입불만후경과월_IB_R6M',
                                        'OS구분코드'], axis=1)
channel_test_df['인입횟수_ARS_R6M'] = channel_test_df['인입횟수_ARS_R6M'].map({'1회 이상': 0, '10회 이상': 1})
channel_test_df['이용메뉴건수_ARS_R6M'] = channel_test_df['이용메뉴건수_ARS_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3})
channel_test_df['방문횟수_PC_R6M'] = channel_test_df['방문횟수_PC_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3, '40회 이상': 4})
channel_test_df['방문일수_PC_R6M'] = channel_test_df['방문일수_PC_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3})
channel_test_df['방문횟수_앱_R6M'] = channel_test_df['방문횟수_앱_R6M'].map({'1회 이상': 0, '10회 이상': 1, '20회 이상': 2, '30회 이상': 3, '40회 이상': 4,
                                                                   '50회 이상': 5, '60회 이상': 6, '70회 이상': 7, '80회 이상': 8})

In [ ]:
channel_test_df.to_parquet("/content/drive/MyDrive/Dacon/channel_test_cleaned.parquet")

## 마케팅정보 (marketing)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

marketing_test_07 is loaded from /content/drive/MyDrive/Data/test/7.마케팅정보/201807_test_마케팅정보.parquet
marketing_test_08 is loaded from /content/drive/MyDrive/Data/test/7.마케팅정보/201808_test_마케팅정보.parquet
marketing_test_09 is loaded from /content/drive/MyDrive/Data/test/7.마케팅정보/201809_test_마케팅정보.parquet
marketing_test_10 is loaded from /content/drive/MyDrive/Data/test/7.마케팅정보/201810_test_마케팅정보.parquet
marketing_test_11 is loaded from /content/drive/MyDrive/Data/test/7.마케팅정보/201811_test_마케팅정보.parquet
marketing_test_12 is loaded from /content/drive/MyDrive/Data/test/7.마케팅정보/201812_test_마케팅정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["marketing"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


marketing_test_df= test_dfs["marketing_test_df"]

gc.collect()

marketing_test_df is created with shape: (600000, 64)


0

In [ ]:
# test 데이터에 대해 train과 동일한 방법 적용
marketing_test_df['캠페인접촉건수_R12M'] = marketing_test_df['캠페인접촉건수_R12M'].map({'1회 이상': 0, '5회 이상': 1,
                                                                           '10회 이상': 2,'15회 이상': 3, '20회 이상': 4, '25회 이상': 5})
marketing_test_df['캠페인접촉일수_R12M'] = marketing_test_df['캠페인접촉일수_R12M'].map({'1일 이상': 0, '5일 이상': 1, '10일 이상': 2,
                                                                           '15일 이상': 3, '20일 이상': 4})
marketing_test_df = marketing_test_df.drop(['컨택건수_CA_TM_B0M', '컨택건수_포인트소진_TM_B0M', '컨택건수_CA_EM_B0M', '컨택건수_리볼빙_EM_B0M',
                                            '컨택건수_리볼빙_청구서_B0M', '컨택건수_카드론_인터넷_B0M', '컨택건수_CA_인터넷_B0M', '컨택건수_리볼빙_인터넷_B0M',
                                            '컨택건수_카드론_당사앱_B0M', '컨택건수_CA_당사앱_B0M', '컨택건수_리볼빙_당사앱_B0M', '컨택건수_CA_EM_R6M',
                                            '컨택건수_리볼빙_EM_R6M', '컨택건수_리볼빙_청구서_R6M', '컨택건수_카드론_인터넷_R6M', '컨택건수_CA_인터넷_R6M',
                                            '컨택건수_리볼빙_인터넷_R6M', '컨택건수_카드론_당사앱_R6M', '컨택건수_CA_당사앱_R6M', '컨택건수_리볼빙_당사앱_R6M',
                                            '컨택건수_FDS_B0M', '컨택건수_FDS_R6M'], axis=1)

In [ ]:
marketing_test_df.to_parquet("/content/drive/MyDrive/Dacon/marketing_test_cleaned.parquet")

## 성과정보 (performance)

In [ ]:
# Google Drive에 있는 데이터 경로
base_dir = "/content/drive/MyDrive/Data"

# 데이터 분할(폴더) 구분
data_splits = ["test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]

        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"{base_dir}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

performance_test_07 is loaded from /content/drive/MyDrive/Data/test/8.성과정보/201807_test_성과정보.parquet
performance_test_08 is loaded from /content/drive/MyDrive/Data/test/8.성과정보/201808_test_성과정보.parquet
performance_test_09 is loaded from /content/drive/MyDrive/Data/test/8.성과정보/201809_test_성과정보.parquet
performance_test_10 is loaded from /content/drive/MyDrive/Data/test/8.성과정보/201810_test_성과정보.parquet
performance_test_11 is loaded from /content/drive/MyDrive/Data/test/8.성과정보/201811_test_성과정보.parquet
performance_test_12 is loaded from /content/drive/MyDrive/Data/test/8.성과정보/201812_test_성과정보.parquet


0

In [ ]:
# 데이터 유형별 설정
info_categories = ["performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")

performance_test_df = test_dfs["performance_test_df"]

gc.collect()

performance_test_df is created with shape: (600000, 49)


0

In [ ]:
# test 데이터에 대해 train과 동일한 방법 적용
performance_test_df["혜택수혜율_B0M"] = performance_test_df["혜택수혜율_B0M"].fillna(performance_test_df["혜택수혜율_B0M"].median())
performance_test_df["혜택수혜율_R3M"] = performance_test_df["혜택수혜율_R3M"].fillna(performance_test_df["혜택수혜율_R3M"].median())

In [ ]:
performance_test_df.to_parquet("/content/drive/MyDrive/Dacon/performance_test_cleaned.parquet")

# Data Preprocessing(3) - concat & merge

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# train data
customer_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/customer_train_cleaned.parquet")
credit_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/credit_train_cleaned.parquet")
sales_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/sales_train_cleaned.parquet")
billing_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/billing_train_cleaned.parquet")
balance_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/balance_train_cleaned.parquet")
channel_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/channel_train_cleaned.parquet")
marketing_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/marketing_train_cleaned.parquet")
performance_train_df = pd.read_parquet("/content/drive/MyDrive/Dacon/performance_train_cleaned.parquet")

In [ ]:
train_df = customer_train_df.merge(credit_train_df, on=['기준년월', 'ID'], how='left')
train_df = train_df.merge(sales_train_df, on=['기준년월', 'ID'], how='left')
train_df = train_df.merge(billing_train_df, on=['기준년월', 'ID'], how='left')
train_df = train_df.merge(balance_train_df, on=['기준년월', 'ID'], how='left')
train_df = train_df.merge(channel_train_df, on=['기준년월', 'ID'], how='left')
train_df = train_df.merge(marketing_train_df, on=['기준년월', 'ID'], how='left')
train_df = train_df.merge(performance_train_df, on=['기준년월', 'ID'], how='left')

In [ ]:
for col in train_df.select_dtypes(include='int64').columns:
    if train_df[col].max() < 2_147_483_647:
        train_df[col] = train_df[col].astype('int32') # 메모리 줄이기 위해 int64 ->int32

In [ ]:
train_df.to_parquet('/content/drive/MyDrive/Data/train_df_cleaned.parquet')

In [ ]:
# test data
customer_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/customer_test_cleaned.parquet")
credit_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/credit_test_cleaned.parquet")
sales_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/sales_test_cleaned.parquet")
billing_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/billing_test_cleaned.parquet")
balance_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/balance_test_cleaned.parquet")
channel_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/channel_test_cleaned.parquet")
marketing_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/marketing_test_cleaned.parquet")
performance_test_df = pd.read_parquet("/content/drive/MyDrive/Dacon/performance_test_cleaned.parquet")

In [ ]:
test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
test_df = test_df.merge(sales_test_df, on=['기준년월', 'ID'], how='left')
test_df = test_df.merge(billing_test_df, on=['기준년월', 'ID'], how='left')
test_df = test_df.merge(balance_test_df, on=['기준년월', 'ID'], how='left')
test_df = test_df.merge(channel_test_df, on=['기준년월', 'ID'], how='left')
test_df = test_df.merge(marketing_test_df, on=['기준년월', 'ID'], how='left')
test_df = test_df.merge(performance_test_df, on=['기준년월', 'ID'], how='left')

for col in test_df.select_dtypes(include='int64').columns:
    if test_df[col].max() < 2_147_483_647:
        test_df[col] = test_df[col].astype('int32')

for col in test_df.select_dtypes(include='float64').columns:
    test_df[col] = test_df[col].astype('float32')

In [ ]:
test_df.to_parquet('/content/drive/MyDrive/Data/test_df_cleaned.parquet')

# Modeling(1) - feature importance

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

import sklearn
from sklearn.utils.class_weight import compute_class_weight
import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_parquet('/content/drive/MyDrive/Data/train_df_cleaned.parquet')

feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]
X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()
y = y.map({'A':0, 'B':1,'C':2,'D':3,'E':4})

del train_df
gc.collect()

# 클래스 weight 계산
classes = np.unique(y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights = dict(zip(classes, weights))

# 각 샘플에 대해 weight 매핑
w_train = pd.Series(y).map(class_weights)

# 전체 feature로 XGBoost 학습 (변수 중요도 추출용)
temp_model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=5,
    eval_metric='mlogloss',
    n_estimators=700,
    tree_method='hist',
    device='cuda',
    random_state=42
    )

temp_model.fit(X, y, sample_weight = w_train, verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=700,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

In [ ]:
# XGBoost 기준 중요도 상위 300개 변수 추출
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': temp_model.feature_importances_
}).sort_values(by='importance', ascending=False)

top300_features = importance_df.head(300)['feature'].tolist()

print(top300_features)

top300_df = pd.DataFrame({'feature': top300_features})
top300_df.to_csv(
    "/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv",
    index=False,
    encoding="utf-8-sig"
)

['카드이용한도금액_A수준복합', '평잔_CA_해외_3M', '이용금액_선결제_B0M', '할부금액_무이자_3M_R12M', '정상청구원금_B5M', '이용금액_신용_R12M', '정상청구원금_B0M', '이용금액_체크_B0M', '이용월평균이용금액_체크_R3M', '할인금액_청구서_B0M', '카드이용한도금액_B1M', '총납부금_B5M', 'CA한도금액', '잔액_리볼빙CA이월_B0M', '이용월평균이용금액_체크_R12M', '카드이용한도금액_B2M', '할부금액총합_무이자_R12M', 'RP건수_학습비_B0M', '이용개월수_카드론_R6M', '평잔_CA_해외_6M', '총납부금_B2M', '탈회횟수_발급1년이내', '방문일수_앱_B0M', '연체입금원금_B5M', '월중평잔_할부_B0M', '건수별평균이용금액_CA_R6M', '이용월평균이용금액_할부_유이자_R6M', 'RP후경과월_아파트', 'RP건수_아파트_B0M', '이용금액_체크_R12M', '이용금액_할부_무이자_B0M', '이용개월수_체크_R3M', 'rv최초시작후경과일', '이용여부_CA', '건수별평균이용금액_체크_R3M', '건수별평균이용금액_신용_R12M', '정상청구원금_B2M', '이용금액_해외', '이용개월수_할부_무이자_R12M', '이용금액_신용_B0M', '최대이용금액_체크_R12M', '이용금액_오프라인_B0M', '입회경과개월수_신용', '정시납부금_B2M', '월중평잔_CA_B0M', '이용금액_신판_R12M', '할인금액_청구서_R3M', '최종카드론_대출이율', '연속유실적개월수_기본_24M_카드', '이용금액_체크_R6M', '카드이용한도금액', '포인트_마일리지_건별_R3M', '증감_RP건수_전기_전월', '이용건수_카드론_R12M', '건수별평균이용금액_체크_R6M', '여유_총이용금액', '총납부금_B0M', '이용금액_할부_R12M', '이용월평균이용금액_신용_R12M', '포인트_마일리지_건별_B0M', '이용개월수_결제일_R6M', 'CA이자율_할인전'

# Modeling(2) - final model train

## XGBoost 모델 학습

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

import sklearn
from sklearn.utils.class_weight import compute_class_weight
import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_parquet('/content/drive/MyDrive/Data/train_df_cleaned.parquet')

feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]
X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()
y = y.map({'A':0, 'B':1,'C':2,'D':3,'E':4})
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}

# 변수 300개 사용
top300_df = pd.read_csv("/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv")
top300_features = top300_df['feature'].tolist()
X_top300 = X[top300_features]

# 오버샘플링
smote = SMOTE(sampling_strategy={0: 30000, 1: 30000, 2: 250000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_top300, y)

# 클래스별 weight 계산
classes = np.unique(y_resampled)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_resampled)
class_weights = dict(zip(classes, weights))
sample_weights = pd.Series(y_resampled).map(class_weights)

for cls in sorted(class_weights):
    print(f"클래스 {cls}: weight = {class_weights[cls]:.2f}")

클래스 0: weight = 17.21
클래스 1: weight = 17.21
클래스 2: weight = 2.07
클래스 3: weight = 1.48
클래스 4: weight = 0.27


In [ ]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=5,
    eval_metric='mlogloss',
    n_estimators=5000,
    tree_method='hist',
    device='cuda',
    random_state=42
    )

# 모델 학습 (검증 없이 전체 데이터 사용)
xgb_model.fit(
    X_resampled, y_resampled,
    sample_weight=sample_weights,
    verbose=False
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5000,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

In [ ]:
xgb_model.save_model('/content/drive/MyDrive/Dacon/softvoting_xgb_xgb변수_3_3_25.json')

## LightGBM 모델 학습

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

# LightGBM을 위한 GPU driver 설치 코드
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!sudo apt install nvidia-driver-460 nvidia-cuda-toolkit clinfo
!apt-get update --fix-missing
!pip install -q  lightgbm==4.1.0 \
  --config-settings=cmake.define.USE_GPU=ON \
  --config-settings=cmake.define.OpenCL_INCLUDE_DIR="/usr/local/cuda/include/" \
  --config-settings=cmake.define.OpenCL_LIBRARY="/usr/local/cuda/lib64/libOpenCL.so"

import sklearn
from sklearn.utils.class_weight import compute_class_weight
import imblearn
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, LGBMClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
clinfo is already the newest version (3.0.21.02.21-1).
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 libnvidia-compute-510 : Depends: libnvidia-compute-535 but it is not installable
 nvidia-cuda-dev : Breaks: libcuda1 (< 495)
                   Recommends: libnvcuvid1 but it is not installable
E: Unable to correct problems, you have held broken packages.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project

In [ ]:
train_df = pd.read_parquet('/content/drive/MyDrive/Data/train_df_cleaned.parquet')

feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]
X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()
y = y.map({'A':0, 'B':1,'C':2,'D':3,'E':4})
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}

# 변수 300개 사용
top300_df = pd.read_csv("/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv")
top300_features = top300_df['feature'].tolist()
X_top300 = X[top300_features]

# 오버샘플링
smote = SMOTE(sampling_strategy={0: 30000, 1: 30000, 2: 250000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_top300, y)

# 클래스별 weight 계산
classes = np.unique(y_resampled)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_resampled)
class_weights = dict(zip(classes, weights))
sample_weights = pd.Series(y_resampled).map(class_weights)

for cls in sorted(class_weights):
    print(f"클래스 {cls}: weight = {class_weights[cls]:.2f}")

클래스 0: weight = 17.21
클래스 1: weight = 17.21
클래스 2: weight = 2.07
클래스 3: weight = 1.48
클래스 4: weight = 0.27


In [ ]:
lgb_model = LGBMClassifier(
    objective='multiclass',
    boosting_type = 'gbdt',
    metric = 'multi_logloss',
    verbosity = -1,
    random_state = 42,
    device = 'gpu',
    num_class = 5,
    gpu_platform_id = 0,
    gpu_device_id = 0,

    num_boost_round = 3348, # n_estimators
    learning_rate = 0.16304239034369372,
    num_leaves = 260,
    max_depth = 6,
    min_data_in_leaf = 1200,
    bagging_fraction = 0.8999999999999999,
    bagging_freq = 4,
    feature_fraction = 0.8,
    lambda_l1 = 4.4610495762623494e-05,
    lambda_l2 = 0.6898684039866835,
    min_gain_to_split = 0.3944497642389666
    )

# 모델 학습 (검증 없이 전체 데이터 사용)
lgb_model.fit(
    X_resampled, y_resampled,
    sample_weight=sample_weights,
    callbacks=[
        log_evaluation(period=0)  # ✅ verbose_eval=False랑 같음!
        ]
)

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LGBMClassifier(bagging_fraction=0.8999999999999999, bagging_freq=4,
               device='gpu', feature_fraction=0.8, gpu_device_id=0,
               gpu_platform_id=0, lambda_l1=4.4610495762623494e-05,
               lambda_l2=0.6898684039866835, learning_rate=0.16304239034369372,
               max_depth=6, metric='multi_logloss', min_data_in_leaf=1200,
               min_gain_to_split=0.3944497642389666, num_boost_round=3348,
               num_class=5, num_leaves=260, objective='multiclass',
               random_state=42, verbosity=-1)

In [ ]:
lgb_model.booster_.save_model('/content/drive/MyDrive/Dacon/softvoting_lgbm_xgb변수_3_3_25.txt')

## CatBoost 모델 학습

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

# catboost 설치
!pip install catboost

import sklearn
from sklearn.utils.class_weight import compute_class_weight
import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 23.7 MB/s eta 0:00:00


In [ ]:
train_df = pd.read_parquet('/content/drive/MyDrive/Data/train_df_cleaned.parquet')

feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]
X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()
y = y.map({'A':0, 'B':1,'C':2,'D':3,'E':4})
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}

# 변수 300개 사용
top300_df = pd.read_csv("/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv")
top300_features = top300_df['feature'].tolist()
X_top300 = X[top300_features]

# 오버샘플링
smote = SMOTE(sampling_strategy={0: 60000, 1: 120000, 2: 240000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_top300, y)

# 클래스별 weight 계산
classes = np.unique(y_resampled)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_resampled)
class_weights = dict(zip(classes, weights))
sample_weights = pd.Series(y_resampled).map(class_weights)

for cls in sorted(class_weights):
    print(f"클래스 {cls}: weight = {class_weights[cls]:.2f}")

클래스 0: weight = 8.97
클래스 1: weight = 4.49
클래스 2: weight = 2.24
클래스 3: weight = 1.54
클래스 4: weight = 0.28


In [ ]:
cat_model = CatBoostClassifier(
    task_type='GPU',
    devices='0',
    random_state=42,
    class_weights=weights.tolist(),
    iterations = 5000
)

# 모델 학습
cat_model.fit(X_resampled, y_resampled)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0:	learn: 1.4404799	total: 570ms	remaining: 47m 31s
1:	learn: 1.3145128	total: 727ms	remaining: 30m 17s
2:	learn: 1.2159606	total: 855ms	remaining: 23m 43s
3:	learn: 1.1369381	total: 969ms	remaining: 20m 10s
4:	learn: 1.0668382	total: 1.09s	remaining: 18m 8s
5:	learn: 1.0107412	total: 1.19s	remaining: 16m 33s
6:	learn: 0.9593553	total: 1.3s	remaining: 15m 29s
7:	learn: 0.9158603	total: 1.41s	remaining: 14m 40s
8:	learn: 0.8774866	total: 1.53s	remaining: 14m 6s
9:	learn: 0.8432782	total: 1.64s	remaining: 13m 39s
10:	learn: 0.8111483	total: 1.77s	remaining: 13m 23s
11:	learn: 0.7833441	total: 1.9s	remaining: 13m 9s
12:	learn: 0.7580101	total: 2.02s	remaining: 12m 54s
13:	learn: 0.7334378	total: 2.15s	remaining: 12m 44s
14:	learn: 0.7117192	total: 2.26s	remaining: 12m 32s
15:	learn: 0.6930203	total: 2.38s	remaining: 12m 20s
16:	learn: 0.6742999	total: 2.5s	remaining: 12m 13s
17:	learn: 0.6578933	total: 2.63s	remaining: 12m 7s
18:	learn: 0.6423834	total:

In [ ]:
cat_model.save_model('/content/drive/MyDrive/Dacon/softvoting_cat_xgb변수_6_12_24.cbm')

# Predict - Soft voting

In [ ]:
import pandas as pd
import numpy as np
import gc

from google.colab import drive
drive.mount('/content/drive')

# LightGBM을 위한 GPU driver 설치 코드
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!sudo apt install nvidia-driver-460 nvidia-cuda-toolkit clinfo
!apt-get update --fix-missing
!pip install -q  lightgbm==4.1.0 \
  --config-settings=cmake.define.USE_GPU=ON \
  --config-settings=cmake.define.OpenCL_INCLUDE_DIR="/usr/local/cuda/include/" \
  --config-settings=cmake.define.OpenCL_LIBRARY="/usr/local/cuda/lib64/libOpenCL.so"

# catboost 설치
!pip install catboost

import sklearn
from sklearn.utils.class_weight import compute_class_weight
import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, LGBMClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
clinfo is already the newest version (3.0.21.02.21-1).
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 libnvidia-compute-510 : Depends: libnvidia-compute-535 but it is not installable
 nvidia-cuda-dev : Breaks: libcuda1 (< 495)
                   Recommends: libnvcuvid1 but it is not installable
E: Unable to correct problems, you have held broken packages.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project

In [ ]:
test_df = pd.read_parquet("/content/drive/MyDrive/Data/test_df_cleaned.parquet")

# 변수 300개 사용
top300_df = pd.read_csv("/content/drive/MyDrive/Dacon/top300_features_XGB_balanced.csv")
top300_features = top300_df['feature'].tolist()

In [ ]:
# 학습한 모델들 불러오기
xgb_model_loaded = xgb.XGBClassifier()
xgb_model_loaded.load_model('/content/drive/MyDrive/Dacon/softvoting_xgb_xgb변수_3_3_25.json')

lgb_model_loaded = lgb.Booster(model_file='/content/drive/MyDrive/Dacon/softvoting_lgbm_xgb변수_3_3_25.txt')

cat_model_loaded = CatBoostClassifier()
cat_model_loaded.load_model('/content/drive/MyDrive/Dacon/softvoting_cat_xgb변수_6_12_24.cbm')

In [ ]:
# test 데이터 준비
X_test = test_df[top300_features]

# XGB predict_proba (iteration_range)
proba_xgb = xgb_model_loaded.predict_proba(X_test, iteration_range=(0, 5000))

# LGB Booster (predict가 proba)
proba_lgb = lgb_model_loaded.predict(X_test, num_iteration=3348)

# CAT predict_proba
proba_cat = cat_model_loaded.predict_proba(X_test)

# soft voting
ensemble_proba = (0.4 * proba_xgb) + (0.3 * proba_lgb) + (0.3 * proba_cat)
ensemble_preds = np.argmax(ensemble_proba, axis=1)
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
ensemble_preds_label = pd.Series(ensemble_preds).map(inverse_label_map)

In [ ]:
# 예측값 test_df에 추가
test_df["pred_label"] = ensemble_preds_label

# ID별 mode aggregation (가장 많이 나온 label)
submission = test_df.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

# 컬럼명 정리
submission.columns = ["ID", "Segment"]

# 저장
submission.to_csv(
    "/content/drive/MyDrive/Dacon/submission_final.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ 결과 저장 완료!")

✅ 결과 저장 완료!
